# 2-electron wave functions and term energies

In [21]:
%load_ext autoreload
%autoreload 1
import twoelectrons as te
%aimport twoelectrons
# provide cell timings
%load_ext autotime
# %autoreload 2
from IPython.display import display, Math
# %config InlineBackend.figure_format='retina'
%config Completer.use_jedi = False
%matplotlib widget

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime


## $d^2$ (Nov 11)

The general goal of this section is to be able to programatically reproduce Ch. 2 of TSK for other groups other than cubic.

Then to apply this analysis to as many specific systems for which CF params are known.

Ideally I should also know at this stage how is that two valence holes can be made equivalent to two valence electrons so that the d^2 electron configurations are made equivalent to d^8 configurations (such as that of Ni^2+).

The general goal can be split into the following tasks:

- Calculation of slater determinant symbols together with visualization.

- Calculation of term symbols.

- Printout of reduction tables (i.e. Table 2.1 for other groups).

- Printout of CG coefficients for the refined Vee coefficients.

- Generation of total spin wavefunctions for two electrons.

- Number of predicted states according to Pauli exclusion principle.

- Calculation of allowed terms.

- Calculation of terms and associated wavefunctions (i.e. analogues of Table 2.2, 2.3, 2.4, ...).

- Calculation of term energies, in terms of slater integrals or racah parameters.

- Visualization of energy levels for different two-electron configurations (diagrams such as that in Fig 2.3).

- Visualization of some sort of the spatial dependence of these eigenfunctions.

- Two electron integrals in terms of Racah parameters.

- Two-Electron wavefunctions.

### Calculation of Term Energies

A number of electrons orbit around a charged nucleus. Many of those

Two valence electrons in an ion have kinetic contributions $\hat{K}_1$ and $\hat{K}_2$ to the Hamiltonian. They are also under the influence of each other's Coulomb potential $\frac{\alpha}{r_{1,2}}$. They move under the influence of the shielded potential of the nucleus $\hat{H}_{R}$ (which is  assumed to have radial symmetry), and of also accordingly to the effect of the crystal lattice in which they move, which is approximated by a crystal field potential $\hat{V}_{CF}$ which would act equally on both electrons, and which only depends on the coordinates of the corresponding single electron. In total the Hamiltonian for this system with two-electrons would then be 

$\hat{H} = \hat{K}_1 + \hat{K}_2 + \hat{H}_{R,1} + \hat{H}_{R,2} + \hat{V}_{CF,1} + \hat{V}_{CF,2} + \frac{\alpha}{r_{1,2}}$.

From this Hamiltonian one may separate the Coulomb repulsion between the two charges so that an unperturbed Hamiltoninan may be taken to be

$\hat{H}_0 = \hat{K}_1 + \hat{K}_2 + \hat{H}_{R,1} + \hat{H}_{R,2} + \hat{V}_{CF,1} + \hat{V}_{CF,2}$

For the unperturbed Hamiltonian, one further notices that it splits up in two equal single-electron Hamiltonian each acting on its corresponding single electron. 

$\hat{H}_s = \left(\hat{K} + \hat{H}_{R}\right) + \hat{V}_{CF}$

For this single electron Hamiltonian  one further notices that there's a spherically symmetric part, and that the crystal field potential lowers this symmetry to that of the point-group symmetry that corresponds to it.

The part with spherical symmetry immediately has solutions to the angular part of the wavefunction, and the radial parts would come from the solutions to ordinary differential equations.

For solving the single electron Hamiltonian one uses the single-electron basis that would come from the solution to the spherically symmetric part. With this basis one may attempt to find an exact solution, in which case one would solve the eigenvalue equation from diagonalizing in blocks this Hamiltonian (which splits in blocks according to the orbital quantum number l). In general, the spherically symmetric part would only raise or lower (by an amount that only depends on l, but which may be imponderable for finding them would require solution to the radial equations) the energies that are obtained from the eigenvalues of the crystal field potential $V_{CF}$. 

If one were to consider transitions between different manifolds (labeled by $l$) one would need to know the energies of the sphericaly symmetric hamiltonian, however if one only cares about transitions within the same manifold (intra-f, intra-d, ...), then the transition energies don't require this and correspond immediately to the eigenvalues of the crystal field potential. As such, from here onwards, we only focus on the ground term.

Furthermore, the eigenfunctions that would come from this neatly group according to the irreducible representations of the group, as such one pictures the degenerate states that belong to a given orbital angular momentum $l$ splitting in terms that are labeled with the labels of the irreducible representations.

For building up the solution to the two-electron ground term, one would then take products of the wave-functions that result from diagonalizing the single-electron hamiltonian over the l-manifold of the ground state. Interestingly, these products may also be catalogued according to single irreducible representations of the group. At this point one also needs to include the spin of the electrons, which become relevant in satisfying Pauli's exclusion principle

Once the terms for the ground terms of the two-electrons have been found as superpositions of products of wave-functions from the terms of the single-electron case, one then goes on to consider the effect of the Coulomb repulsion, which may lift some of the degeneracy in the ground state terms.

<span style="color:lime">
1. Determine the symbolic expressions for the matrix elements of the Coulomb repulsion over the states of a given two-electron irrep-configuration (which includes states from several single-electron terms)
</span>

In [19]:
ids = te.four_symbol_identities(te.CPGs.get_group_by_label('T_{h}'))

Took 20 s


In [20]:
ids = te.four_symbol_identities(te.CPGs.get_group_by_label('T_{h}'))

Took 18 s


In [507]:
def simplify_qet(qet):
    new_dict = {k:sp.simplify(v) for k,v in qet.dict.items()}
    return Qet(new_dict)
def term_energies(group_label, verbose=True):
    # version: 1637619241
    stages = 19 # this is how many distinct parts this algorithm has
    if verbose:
        pbar = tqdm(range(stages), bar_format='({n:d}/{total:d}) [{elapsed}] {desc}')
    terms = twoe_terms[group_label]
    group = CPGs.get_group_by_label(group_label)
    component_labels = {k:list(v.values()) for k,v in new_labels[group_label].items()}
    # for a given set of terms
    # split all the enclosed states
    # in groups of electron configurations
    configs = {}
    config_supplement = {}
    for term_key, term in terms.items():
        for state_key, state in zip(term.state_keys, term.states):
            (Γ1, Γ2, Γ3, γ3, S, mSz) = state_key
            α = sp.Symbol(sp.latex(Γ1).lower())*sp.Symbol(sp.latex(Γ2).lower())
            if α not in configs.keys():
                configs[α] = {}
                config_supplement[α] = {}
            if (S,Γ3) not in configs[α].keys():
                configs[α][(S,Γ3)] = []
                config_supplement[α][(S,Γ3)] = []
            configs[α][(S,Γ3)].append(state)
            config_supplement[α][(S,Γ3)].append(state_key)

    def overlinesqueegee(s):
        '''
        Going back from the overline shorthand for spin down,
        acting on a single set of quantum numbers.
        '''
        if 'overline' in str(s):
            spin = -sp.S(1)/2
            comp = sp.Symbol(str(s).replace('\\overline{','')[:-1])
        else:
            spin = sp.S(1)/2
            comp = s
        return (comp, spin)
    def spin_restoration(qet):
        '''
        Going back from the overline shorthand for spin down,
        acting on a qet.
        '''
        new_dict = {}
        for k,v in qet.dict.items():
            k = (*overlinesqueegee(k[0]),*overlinesqueegee(k[1]))
            new_dict[k] = v
        return Qet(new_dict)

    # within each configuration
    config_matrices = {}
    for config in configs.keys():
        # display(config)
        # within each term
        term_matrices = {}
        for term_key, states in configs[config].items():
            # find the brackets between all possible pairs of states
            ham_matrix = []
            for state0 in states:
                ham_row = []
                state0 = spin_restoration(state0)
                for state1 in states:
                    # recover the spin for the determinantal states
                    pstate = state1
                    state1 = spin_restoration(state1)
                    # determine the det braket between these two states (an operator in between is assumed and omitted)
                    detbraket = state0.dual()*state1 # changehere
                    # simplify this braket of detstates into brakets between regular states
                    regbraket = Qet({})
                    for k,v in detbraket.dict.items():
                        γ1, m1, γ2, m2, γ1p, m1p, γ2p, m2p = k
                        # k0 = (γ1, m1, γ2, m2, γ1p, m1p, γ2p, m2p)
                        # k1 = (γ1, m1, γ2, m2, γ2p, m2p, γ1p, m1p)
                        k0 = (γ1, γ2, γ1p, γ2p)
                        k1 = (γ1, γ2, γ2p, γ1p)
                        # enforce orthogonality wrt. spin
                        if (m1 == m1p) and (m2 == m2p):
                            qplus = Qet({k0:v}) 
                        else:
                            qplus = Qet({})
                        if (m1 == m2p) and (m2 == m1p):
                            qminus = Qet({k1:-v}) 
                        else:
                            qminus = Qet({})
                        regbraket += (qplus+qminus)
                    ham_row.append(regbraket)
                ham_matrix.append(ham_row)
            term_matrices[term_key] = ham_matrix
        config_matrices[config] = term_matrices

    def composite_symbol(x):
        return sp.Symbol('(%s)'%(','.join(list(map(sp.latex,x)))))
    def fourtuplerecovery(ft):
        '''the inverse of composite_symbol'''
        return tuple(map(sp.Symbol,sp.latex(ft)[1:-1].split(',')))
    
    if verbose:
        msg = group_label + " Creating all 4-symbol identities ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1

    # this   integral_identities  dictionary  will  have  as  keys
    # 4-tuples  of  irreps  and  its  values  will  be lists whose
    # elements  are  2-tuples whose first elements are 4-tuples of
    # irrep  components  and  whose values are qets whose keys are
    # 4-tuples  of  irrep components and whose values are numeric.
    # These 4-tuples represent a braket with the Coulomb repulsion
    # operator in between.

    integral_identities = {}
    ir_mats = group.irrep_matrices
    for ir1, ir2, ir3, ir4 in product(*([group.irrep_labels]*4)):
        # To simplify calculations this part
        # may only be done over quadruples in a standard
        # order.
        # Whatever is missed here is then brough back in
        # by the reality relations.
        altorder1 = (ir3, ir2, ir1, ir4)
        altorder2 = (ir1, ir4, ir3, ir2)
        altorder3 = (ir3, ir4, ir1, ir2)
        if (altorder1 in integral_identities) or\
           (altorder2 in integral_identities) or\
           (altorder3 in integral_identities):
            continue
        integral_identity_sector = []
        components = [component_labels[ir] for ir in [ir1, ir2, ir3, ir4]]
        comp_to_idx = [{c: idx for idx, c in enumerate(component_labels[ir])} for ir in [ir1, ir2, ir3, ir4]]
        for R in group.generators:
            R_id = {}
            for γ1, γ2, γ3, γ4 in product(*components):
                for γ1p, γ2p, γ3p, γ4p in product(*components):
                        val = sp.conjugate(ir_mats[ir1][R][comp_to_idx[0][γ1],comp_to_idx[0][γ1p]]) *\
                              sp.conjugate(ir_mats[ir2][R][comp_to_idx[1][γ2],comp_to_idx[1][γ2p]]) *\
                              ir_mats[ir3][R][comp_to_idx[2][γ3],comp_to_idx[2][γ3p]] *\
                              ir_mats[ir4][R][comp_to_idx[3][γ4],comp_to_idx[3][γ4p]]
                        if val== 0:
                            continue
                        key = (γ1, γ2, γ3, γ4)
                        if key not in R_id.keys():
                            R_id[key] = []
                        R_id[key].append( Qet({(γ1p,γ2p,γ3p,γ4p): val}) )
            R_id_total = [(key, sum(R_id[key], Qet({}))) for key in R_id.keys()]
            R_id_total = [q for q in R_id_total if len(q[1].dict)>0]
            integral_identity_sector.extend(R_id_total)
        integral_identities[(ir1,ir2,ir3,ir4)] = integral_identity_sector

    # For solving the linear system it is convenient
    # to have everything on one side of the equation.
    if verbose:
        msg = group_label + " Creating set of identities ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    identities = {}
    for ircombo in integral_identities:
        these_ids = []
        for v in integral_identities[ircombo]:
            lhs, rhs = v
            diff = Qet({lhs:1}) - rhs
            if len(diff.dict) > 0:
                these_ids.append(diff)
        identities[ircombo] = these_ids

    # If an equation has only one key, then
    # that immediately means that that braket is zero.
    if verbose:
        msg = group_label + " Finding trivial zeros ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    # first determine which ones have to be zero
    better_identities = {irc:[] for irc in identities}
    all_zeros = {}
    for ircombo in identities:
        zeros = []
        for identity in identities[ircombo]:
            if len(identity.dict) == 1:
                zeros.append((list(identity.dict.keys())[0]))
            else:
                better_identities[ircombo].append(identity)
        all_zeros[ircombo] = zeros

    # use them to simplify things.
    if verbose:
        msg = group_label + " Using them to simplify things ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    great_identities = {irc:[] for irc in identities}
    for ircombo in better_identities:
        for identity in better_identities[ircombo]:
            new_qet = Qet({})
            for k,v in identity.dict.items():
                if k in all_zeros[ircombo]:
                    continue
                else:
                    new_qet+= Qet({k: v})
            if len(new_qet.dict) == 0:
                continue
            great_identities[ircombo].append(new_qet)

    # Inside of a four-symbol braket one may do three exchanges
    # that must result in the same value. That if the wave
    # functions are assumed to be real-valued.
    
    if verbose:
        msg = group_label + " Creating reality identities ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    real_var_simplifiers = {irc:[] for irc in great_identities}
    kprimes = set()
    # this has to run over all the quadruples of irs
    for ir1, ir2, ir3, ir4 in product(*([group.irrep_labels]*4)):
        real_var_simplifier = {}
        ircombo = (ir1, ir2, ir3, ir4)
        components = [component_labels[ir] for ir in ircombo]
        for γ1, γ2, γ3, γ4 in product(*components):
            k = (γ1, γ2, γ3, γ4)
            kalt1 = (γ3, γ2, γ1, γ4)
            kalt2 = (γ1, γ4, γ3, γ2)
            kalt3 = (γ3, γ4, γ1, γ2)
            if kalt1 in kprimes:
                real_var_simplifier[k] = kalt1
            elif kalt2 in kprimes:
                real_var_simplifier[k] = kalt2
            elif kalt3 in kprimes:
                real_var_simplifier[k] = kalt3
            else:
                real_var_simplifier[k] = k
                kprimes.add(k)
        real_var_simplifiers[ircombo] = real_var_simplifier

    real_var_full_simplifier = {}
    for ircombo in real_var_simplifiers:
        real_var_full_simplifier.update(real_var_simplifiers[ircombo])

    # For each 4-tuple of irreps
    # create a system of symbolic solutions
    # and let sympy solve that.
    # For each 4-tuple of irreps
    # the end result is a dictionary
    # whose keys represent the dependent
    # brakets and whose values are the
    # relation that those dependent values
    # have with the independent brakets.
    # As such, when these dictionaries are
    # used on an expression, everything should
    # then be given in terms of indepedent brakets.

    if verbose:
        msg = group_label + " Solving for independent 4-symbol brakets ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    all_sols = {irc:[] for irc in great_identities}
    for ircombo in great_identities:
        zeros = all_zeros[ircombo]
        problem_vars = list(set(sum([list(identity.dict.keys()) for identity in great_identities[ircombo]],[])))
        big_ma = [awe.vec_in_basis(problem_vars) for awe in great_identities[ircombo]] 
        big_mat = sp.Matrix(big_ma)
        big_mat = sp.re(big_mat) + sp.I*sp.im(big_mat)
        rref_mat, pivots = big_mat.rref()
        num_rows = rref_mat.rows
        num_cols = rref_mat.cols
        rref_ma_non_zero = [rref_mat[row,:] for row in range(num_rows) if (sum(np.array(rref_mat[row,:])[0] == 0) != num_cols)]
        rref_mat_non_zero = sp.Matrix(rref_ma_non_zero)
        varvec = sp.Matrix(list(map(composite_symbol,problem_vars)))
        eqns = rref_mat_non_zero*varvec
        ssol = sp.solve(list(eqns), dict=True)
        all_sols[ircombo] = ssol
        assert len(ssol) in [0,1]
        if len(ssol) == 0:
            sol_dict = {}
        else:
            sol_dict = ssol[0]
        zero_addendum = {k:{} for k in all_zeros[ircombo]}
        sol_dict = {fourtuplerecovery(k):{fourtuplerecovery(s):v.coeff(s) for s in v.free_symbols} for k,v in sol_dict.items()}
        sol_dict.update(zero_addendum)
        all_sols[ircombo] = sol_dict

    # This final dictionary is unnecessary but
    # simplifies calling the replacements onto
    # a symbolic expression.
    if verbose:
        msg = group_label + " Creating a dictionary with all the 4-symbol replacements ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    super_solution = {}
    for ircombo in all_sols:
        super_solution.update(all_sols[ircombo])

    def simplifier(qet):
        simp_ket = Qet({})
        for k,v in qet.dict.items():
            simp_ket += Qet({real_var_full_simplifier[k]:v})
        true_qet = Qet({})
        for k,v in simp_ket.dict.items():
            if k in super_solution:
                true_qet += v*Qet(super_solution[k])
            else:
                true_qet += Qet({k:v})
        return true_qet

    def simplify_config_matrix(ir1ir2, S, ir3):
        config_matrix = config_matrices[ir1ir2][(S,ir3)]
        num_rows = len(config_matrix)
        simple_matrix = [[simplifier(config_matrix[row][col]) for col in range(num_rows)] for row in range(num_rows)]
        return simple_matrix
    
    if verbose:
        msg = group_label + " Simplifying configuration matrices ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    simple_config_matrices = {k:{} for k in config_matrices}
    for ir1ir2 in config_matrices.keys():
        for term_key in config_matrices[ir1ir2]:
            S, Γ3 = term_key
            simple_config_matrices[ir1ir2][term_key] = simplify_config_matrix(ir1ir2, S, Γ3)

    twotuplerecovery = fourtuplerecovery
    if verbose:
        msg = group_label + " Creating all 2-symbol identities ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1

    # this   integral_identities  dictionary  will  have  as  keys
    # 2-tuples  of  irreps  and  its  values  will  be lists whose
    # elements  are  2-tuples whose first elements are 2-tuples of
    # irrep  components  and  whose values are qets whose keys are
    # 2-tuples  of  irrep components and whose values are numeric.
    # These 2-tuples represent a braket with the an invariant operator
    # operator in between.

    integral_identities_2 = {}
    ir_mats = group.irrep_matrices
    for ir1, ir2 in product(*([group.irrep_labels]*2)):
        # To simplify calculations this part
        # can only be done over quadruples in a standard
        # order.
        # Whatever is missed here is then brough back in
        # by the reality relations.
        altorder1 = (ir2, ir1, ir1, ir4)
        if (altorder1 in integral_identities_2):
            continue
        integral_identity_sector = []
        components = [component_labels[ir] for ir in [ir1, ir2]]
        comp_to_idx = [{c: idx for idx, c in enumerate(component_labels[ir])} for ir in [ir1, ir2]]
        for R in group.generators:
            R_id = {}
            for γ1, γ2 in product(*components):
                for γ1p, γ2p in product(*components):
                        val = sp.conjugate(ir_mats[ir1][R][comp_to_idx[0][γ1],comp_to_idx[0][γ1p]]) *\
                              ir_mats[ir2][R][comp_to_idx[1][γ2],comp_to_idx[1][γ2p]]
                        if val== 0:
                            continue
                        key = (γ1, γ2)
                        if key not in R_id.keys():
                            R_id[key] = []
                        R_id[key].append( Qet({(γ1p,γ2p): val}) )
            R_id_total = [(key, sum(R_id[key], Qet({}))) for key in R_id.keys()]
            R_id_total = [q for q in R_id_total if len(q[1].dict)>0]
            integral_identity_sector.extend(R_id_total)
        integral_identities_2[(ir1,ir2)] = integral_identity_sector

    # For solving the linear system it is convenient
    # to have everything on one side of the equation.
    if verbose:
        msg = group_label + " Creating set of 2 symbol identities ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    identities_2 = {}
    
    for ircombo in integral_identities_2:
        these_ids = []
        for v in integral_identities_2[ircombo]:
            lhs, rhs = v
            diff = Qet({lhs:1}) - rhs
            if len(diff.dict) > 0:
                these_ids.append(diff)
        identities_2[ircombo] = these_ids

    # If an equation has only one term, then
    # that immediately means that that term is zero.
    if verbose:
        msg = group_label + " Finding trivial zeros ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    # first determine which ones have to be zero
    better_identities_2 = {irc:[] for irc in identities_2}
    all_zeros_2 = {}
    for ircombo in identities_2:
        zeros = []
        for identity in identities_2[ircombo]:
            if len(identity.dict) == 1:
                zeros.append((list(identity.dict.keys())[0]))
            else:
                better_identities_2[ircombo].append(identity)
        all_zeros_2[ircombo] = zeros

    # use them to simplify things.
    if verbose:
        msg = group_label + " Using them to simplify things ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    great_identities_2 = {irc:[] for irc in identities_2}
    for ircombo in better_identities_2:
        for identity in better_identities_2[ircombo]:
            new_qet = Qet({})
            for k,v in identity.dict.items():
                if k in all_zeros_2[ircombo]:
                    continue
                else:
                    new_qet+= Qet({k: v})
            if len(new_qet.dict) == 0:
                continue
            great_identities_2[ircombo].append(new_qet)

    # Inside of a two-symbol braket one may do three exchanges
    # that must result in the same value. That if the wave
    # functions are assumed to be real-valued.
    if verbose:
        msg = group_label + " Creating reality identities ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    real_var_simplifiers_2 = {irc:[] for irc in great_identities_2}
    # this has to run over all the quadruples of irs
    kprimes = set()
    for ir1, ir2 in product(*([group.irrep_labels]*2)):
        real_var_simplifier = {}
        ircombo = (ir1, ir2)
        components = [component_labels[ir] for ir in ircombo]
        for γ1, γ2 in product(*components):
            k = (γ1, γ2)
            kalt = (γ2, γ1)
            # If for a given key I find that its
            # switched version has already been seen
            # Then that key has to be mapped to be
            # mapped to the key already present.
            if kalt in kprimes:
                real_var_simplifier[k] = kalt
            else:
                real_var_simplifier[k] = k
                kprimes.add(k)
        real_var_simplifiers_2[ircombo] = real_var_simplifier

    # For each 2-tuple of irreps
    # create a system of symbolic solutions
    # and let sympy solve that.
    # For each 2-tuple of irreps
    # the end result is a dictionary
    # whose keys represent the dependent
    # brakets and whose values are the
    # relation that those dependent values
    # have with the independent brakets.
    # As such, when these dictionaries are
    # used on an expression, everything should
    # then be given in terms of indepedent brakets.
    if verbose:
        msg = group_label + " Solving for independent 2-symbol brakets ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    all_sols_2 = {irc:[] for irc in great_identities_2}
    
    for ircombo in great_identities_2:
        zeros = all_zeros_2[ircombo]
        problem_vars = list(set(sum([list(identity.dict.keys()) for identity in great_identities_2[ircombo]],[])))
        big_ma = [awe.vec_in_basis(problem_vars) for awe in great_identities_2[ircombo]] 
        big_mat = sp.Matrix(big_ma)
        big_mat = sp.re(big_mat)+sp.I*sp.im(big_mat)
        rref_mat, pivots = big_mat.rref()
        num_rows = rref_mat.rows
        num_cols = rref_mat.cols
        rref_ma_non_zero = [rref_mat[row,:] for row in range(num_rows) if (sum(np.array(rref_mat[row,:])[0] == 0) != num_cols)]
        rref_mat_non_zero = sp.Matrix(rref_ma_non_zero)
        varvec = sp.Matrix(list(map(composite_symbol,problem_vars)))
        eqns = rref_mat_non_zero*varvec
        ssol = sp.solve(list(eqns), dict=True)
        all_sols[ircombo] = ssol
        assert len(ssol) in [0,1]
        if len(ssol) == 0:
            sol_dict = {}
        else:
            sol_dict = ssol[0]
        zero_addendum = {k:{} for k in all_zeros_2[ircombo]}
        sol_dict = {twotuplerecovery(k):{twotuplerecovery(s):v.coeff(s) for s in v.free_symbols} for k,v in sol_dict.items()}
        sol_dict.update(zero_addendum)
        all_sols_2[ircombo] = sol_dict

    # This final dictionary is unnecessary but
    # simplifies calling the replacements onto
    # a symbolic expression.
    if verbose:
        msg = group_label + " Creating a dictionary with all the 2-symbol replacements ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    super_solution_2 = {}
    for ircombo in all_sols_2:
        super_solution_2.update(all_sols_2[ircombo])

    # for a given electron config
    more_ids = {e_config:[] for e_config in simple_config_matrices}
    for e_config in simple_config_matrices:
        for term in simple_config_matrices[e_config]:
            the_matrix = simple_config_matrices[e_config][term]
            num_rows = len(the_matrix)
            num_cols = len(the_matrix)
            the_state_keys = config_supplement[e_config][term]
            the_key_matrix = {}
            for row_idx in range(num_rows):
                Γ3_row = the_state_keys[row_idx][2]
                γ3_row = the_state_keys[row_idx][3]
                for col_idx in range(num_cols):
                    Γ3_col = the_state_keys[col_idx][2]
                    γ3_col = the_state_keys[col_idx][3]
                    matrix_val = the_matrix[row_idx][col_idx]
                    the_key_matrix[(γ3_row,γ3_col)] = matrix_val
            # now go over the keys of super_solution_2
            # and if one of those keys matches with a key in the_key_matrix
            # do something about it
            for k in super_solution_2:
                if k in the_key_matrix:
                    v = super_solution_2[k]
                    # this matrix element
                    matrix_element = the_key_matrix[k]
                    # must be identified with the sum
                    # as given in v
                    matrix_equiv = sum([kv*the_key_matrix[km] for km, kv in v.items()],Qet({}))
                    identity = (matrix_element - matrix_equiv) #=0
                    if len(identity.dict) != 0:
                        more_ids[e_config].append(identity)
                        problem_vars = list(set(sum([list(identity.dict.keys()) for identity in more_ids[e_config]],[])))
    if verbose:
        msg = group_label + " Simplifying numeric values of qets ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    more_ids = {e_config:list(filter(lambda x: len(x.dict) > 0,list(map(simplify_qet,more_ids[e_config])))) for e_config in more_ids}
    
    if verbose:
        msg = group_label + " Solving for dependent vars in terms of independent ones ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    all_sols_2_4 = {irc:[] for irc in more_ids}
    
    for e_config in more_ids:
        problem_vars = list(set(sum([list(identity.dict.keys()) for identity in more_ids[e_config]],[])))
        big_ma = [awe.vec_in_basis(problem_vars) for awe in more_ids[e_config]] 
        big_mat = sp.Matrix(big_ma)
        big_mat = sp.re(big_mat)+sp.I*sp.im(big_mat)
        rref_mat, pivots = big_mat.rref()
        num_rows = rref_mat.rows
        num_cols = rref_mat.cols
        rref_ma_non_zero = [rref_mat[row,:] for row in range(num_rows) if (sum(np.array(rref_mat[row,:])[0] == 0) != num_cols)]
        rref_mat_non_zero = sp.Matrix(rref_ma_non_zero)
        varvec = sp.Matrix(list(map(composite_symbol, problem_vars)))
        eqns = rref_mat_non_zero*varvec
        ssol = sp.solve(list(eqns), dict=True)
        all_sols_2_4[e_config] = ssol
        assert len(ssol) in [0,1]
        if len(ssol) == 0:
            sol_dict = {}
        else:
            sol_dict = ssol[0]
        sol_dict = {twotuplerecovery(k):{twotuplerecovery(s):v.coeff(s) for s in v.free_symbols} for k,v in sol_dict.items()}
        all_sols_2_4[e_config] = sol_dict

    # flatten into a single dictionary of replacements
    fab_solution_2_4 = {}
    for e_config in all_sols_2_4:
        fab_solution_2_4.update(all_sols_2_4[e_config])
    if verbose:
        msg = group_label + " There are %d less independent variables ..."
        pbar.set_description("There are %d less independent variables ..." % len(fab_solution_2_4))
        pbar.refresh()
        pbar.n += 1
    
    def simplifier_f(qet):
        true_qet = Qet({})
        for k,v in qet.dict.items():
            if k in fab_solution_2_4:
                true_qet += v*Qet(fab_solution_2_4[k])
            else:
                true_qet += Qet({k:v})
        return true_qet
    
    def simplify_config_matrix_f(ir1ir2, S, ir3):
        config_matrix = simple_config_matrices[ir1ir2][(S,ir3)]
        num_rows = len(config_matrix)
        simple_matrix = [[simplifier_f(config_matrix[row][col]) for col in range(num_rows)] for row in range(num_rows)]
        #simple_matrix = sp.Matrix(simple_matrix)
        return simple_matrix
    
    if verbose:
        msg = group_label + " Making final simplifications ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    final_config_matrices = {k:{} for k in config_matrices}
    for ir1ir2 in config_matrices.keys():
        for term_key in config_matrices[ir1ir2]:
            S, Γ3 = term_key
            final_config_matrices[ir1ir2][term_key] = simplify_config_matrix_f(ir1ir2, S, Γ3)
    
    if verbose:
        msg = group_label + " Finished"
        pbar.set_description(msg)
        pbar.refresh()
    
    return final_config_matrices

In [508]:
all_term_energies = {}
for group_label in CPGs.all_group_labels:
    if group_label in all_term_energies:
        continue
    final_config_matrices = term_energies(group_label)
#     print("Printing ...")
#     for counter, ir1ir2 in enumerate(final_config_matrices):
#         print('-'*10)
#         display(ir1ir2)
#         for term_key in final_config_matrices[ir1ir2]:
#             display(term_key)
#             display(sp.Matrix([[el.as_braket() for el in row] for row in final_config_matrices[ir1ir2][term_key]]))
#     print("Saving ...")
    all_term_energies[group_label] = final_config_matrices

(0/19) [00:00] 

(0/19) [00:00] 

(0/19) [00:00] 

(0/19) [00:00] 

(0/19) [00:00] 

...

(0/19) [00:00] 

(0/19) [00:00] 

(0/19) [00:00] 

(0/19) [00:00] 

(0/19) [00:00] 

In [509]:
def simplify_qet_values(qet):
    sqetdict = {k:sp.simplify(v) for k,v in qet.dict.items()}
    return Qet(sqetdict)

In [510]:
term_energies_for_printout = {}
for group_label, term_energies in all_term_energies.items():
    for e_config, terms  in term_energies.items():
        for term_key, term_matrix in terms.items():
            term_energy = term_matrix[0][0]
            term_energy = simplify_qet_values(term_energy)
            term_energies_for_printout[(group_label, e_config, term_key)] = term_energy

In [511]:
pickle.dump(term_energies_for_printout, open('./data/term_energies_for_print_1637619241.pkl','wb'))

In [409]:
!beep

After this jump to "2e terms and their wavefunctions"

#### Fixed version comparison to O

In [375]:
def simplify_qet(qet):
    new_dict = {k:sp.simplify(v) for k,v in qet.dict.items()}
    return Qet(new_dict)
def term_energies(group_label, verbose=True):
    # version: 1637596104
    stages = 19 # this is how many distinct parts this algorithm has
    if verbose:
        pbar = tqdm(range(stages), bar_format='({n:d}/{total:d}) [{elapsed}] {desc}')
    terms = twoe_terms[group_label]
    group = CPGs.get_group_by_label(group_label)
    component_labels = {k:list(v.values()) for k,v in new_labels[group_label].items()}
    # for a given set of terms
    # split all the enclosed states
    # in groups of electron configurations
    configs = {}
    config_supplement = {}
    for term_key, term in terms.items():
        for state_key, state in zip(term.state_keys, term.states):
            (Γ1, Γ2, Γ3, γ3, S, mSz) = state_key
            α = sp.Symbol(sp.latex(Γ1).lower())*sp.Symbol(sp.latex(Γ2).lower())
            if α not in configs.keys():
                configs[α] = {}
                config_supplement[α] = {}
            if (S,Γ3) not in configs[α].keys():
                configs[α][(S,Γ3)] = []
                config_supplement[α][(S,Γ3)] = []
            configs[α][(S,Γ3)].append(state)
            config_supplement[α][(S,Γ3)].append(state_key)

    def overlinesqueegee(s):
        '''
        Going back from the overline shorthand for spin down,
        acting on a single set of quantum numbers.
        '''
        if 'overline' in str(s):
            spin = -sp.S(1)/2
            comp = sp.Symbol(str(s).replace('\\overline{','')[:-1])
        else:
            spin = sp.S(1)/2
            comp = s
        return (comp, spin)
    def spin_restoration(qet):
        '''
        Going back from the overline shorthand for spin down,
        acting on a qet.
        '''
        new_dict = {}
        for k,v in qet.dict.items():
            k = (*overlinesqueegee(k[0]),*overlinesqueegee(k[1]))
            new_dict[k] = v
        return Qet(new_dict)

    # within each configuration
    config_matrices = {}
    for config in configs.keys():
        # display(config)
        # within each term
        term_matrices = {}
        for term_key, states in configs[config].items():
            # find the brackets between all possible pairs of states
            ham_matrix = []
            for state0 in states:
                ham_row = []
                state0 = spin_restoration(state0)
                for state1 in states:
                    # recover the spin for the determinantal states
                    pstate = state1
                    state1 = spin_restoration(state1)
                    # determine the det braket between these two states (an operator in between is assumed and omitted)
                    detbraket = state0.dual()*state1 # changehere
                    # simplify this braket of detstates into brakets between regular states
                    regbraket = Qet({})
                    for k,v in detbraket.dict.items():
                        γ1, m1, γ2, m2, γ1p, m1p, γ2p, m2p = k
                        # k0 = (γ1, m1, γ2, m2, γ1p, m1p, γ2p, m2p)
                        # k1 = (γ1, m1, γ2, m2, γ2p, m2p, γ1p, m1p)
                        k0 = (γ1, γ2, γ1p, γ2p)
                        k1 = (γ1, γ2, γ2p, γ1p)
                        # enforce orthogonality wrt. spin
                        if (m1 == m1p) and (m2 == m2p):
                            qplus = Qet({k0:v}) 
                        else:
                            qplus = Qet({})
                        if (m1 == m2p) and (m2 == m1p):
                            qminus = Qet({k1:-v}) 
                        else:
                            qminus = Qet({})
                        regbraket += (qplus+qminus)
                    ham_row.append(regbraket)
                ham_matrix.append(ham_row)
            term_matrices[term_key] = ham_matrix
        config_matrices[config] = term_matrices

    def composite_symbol(x):
        return sp.Symbol('(%s)'%(','.join(list(map(sp.latex,x)))))
    def fourtuplerecovery(ft):
        '''the inverse of composite_symbol'''
        return tuple(map(sp.Symbol,sp.latex(ft)[1:-1].split(',')))
    
    if verbose:
        msg = "Creating all 4-symbol identities ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1

    # this   integral_identities  dictionary  will  have  as  keys
    # 4-tuples  of  irreps  and  its  values  will  be lists whose
    # elements  are  2-tuples whose first elements are 4-tuples of
    # irrep  components  and  whose values are qets whose keys are
    # 4-tuples  of  irrep components and whose values are numeric.
    # These 4-tuples represent a braket with the Coulomb repulsion
    # operator in between.

    integral_identities = {}
    ir_mats = group.irrep_matrices
    for ir1, ir2, ir3, ir4 in product(*([group.irrep_labels]*4)):
        # To simplify calculations this part
        # may only be done over quadruples in a standard
        # order.
        # Whatever is missed here is then brough back in
        # by the reality relations.
        altorder1 = (ir3, ir2, ir1, ir4)
        altorder2 = (ir1, ir4, ir3, ir2)
        altorder3 = (ir3, ir4, ir1, ir2)
        if (altorder1 in integral_identities) or\
           (altorder2 in integral_identities) or\
           (altorder3 in integral_identities):
            continue
        integral_identity_sector = []
        components = [component_labels[ir] for ir in [ir1, ir2, ir3, ir4]]
        comp_to_idx = [{c: idx for idx, c in enumerate(component_labels[ir])} for ir in [ir1, ir2, ir3, ir4]]
        for R in group.generators + [sp.Symbol('{Ee}')]:
            R_id = {}
            for γ1, γ2, γ3, γ4 in product(*components):
                for γ1p, γ2p, γ3p, γ4p in product(*components):
                        val = sp.conjugate(ir_mats[ir1][R][comp_to_idx[0][γ1],comp_to_idx[0][γ1p]]) *\
                              sp.conjugate(ir_mats[ir2][R][comp_to_idx[1][γ2],comp_to_idx[1][γ2p]]) *\
                              ir_mats[ir3][R][comp_to_idx[2][γ3],comp_to_idx[2][γ3p]] *\
                              ir_mats[ir4][R][comp_to_idx[3][γ4],comp_to_idx[3][γ4p]]
                        if val== 0:
                            continue
                        key = (γ1, γ2, γ3, γ4)
                        if key not in R_id.keys():
                            R_id[key] = []
                        R_id[key].append( Qet({(γ1p,γ2p,γ3p,γ4p): val}) )
            R_id_total = [(key, sum(R_id[key], Qet({}))) for key in R_id.keys()]
            R_id_total = [q for q in R_id_total if len(q[1].dict)>0]
            integral_identity_sector.extend(R_id_total)
        integral_identities[(ir1,ir2,ir3,ir4)] = integral_identity_sector

    # For solving the linear system it is convenient
    # to have everything on one side of the equation.
    if verbose:
        msg = "Creating set of identities ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    identities = {}
    for ircombo in integral_identities:
        these_ids = []
        for v in integral_identities[ircombo]:
            lhs, rhs = v
            diff = Qet({lhs:1}) - rhs
            if len(diff.dict) > 0:
                these_ids.append(diff)
        identities[ircombo] = these_ids

    # If an equation has only one key, then
    # that immediately means that that braket is zero.
    if verbose:
        msg = "Finding trivial zeros ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    # first determine which ones have to be zero
    better_identities = {irc:[] for irc in identities}
    all_zeros = {}
    for ircombo in identities:
        zeros = []
        for identity in identities[ircombo]:
            if len(identity.dict) == 1:
                zeros.append((list(identity.dict.keys())[0]))
            else:
                better_identities[ircombo].append(identity)
        all_zeros[ircombo] = zeros

    # use them to simplify things.
    if verbose:
        msg = "Using them to simplify things ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    great_identities = {irc:[] for irc in identities}
    for ircombo in better_identities:
        for identity in better_identities[ircombo]:
            new_qet = Qet({})
            for k,v in identity.dict.items():
                if k in all_zeros[ircombo]:
                    continue
                else:
                    new_qet+= Qet({k: v})
            if len(new_qet.dict) == 0:
                continue
            great_identities[ircombo].append(new_qet)

    # Inside of a four-symbol braket one may do three exchanges
    # that must result in the same value. That if the wave
    # functions are assumed to be real-valued.
    
    if verbose:
        msg = "Creating reality identities ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    real_var_simplifiers = {irc:[] for irc in great_identities}
    kprimes = set()
    # this has to run over all the quadruples of irs
    for ir1, ir2, ir3, ir4 in product(*([group.irrep_labels]*4)):
        real_var_simplifier = {}
        ircombo = (ir1, ir2, ir3, ir4)
        components = [component_labels[ir] for ir in ircombo]
        for γ1, γ2, γ3, γ4 in product(*components):
            k = (γ1, γ2, γ3, γ4)
            kalt1 = (γ3, γ2, γ1, γ4)
            kalt2 = (γ1, γ4, γ3, γ2)
            kalt3 = (γ3, γ4, γ1, γ2)
            if kalt1 in kprimes:
                real_var_simplifier[k] = kalt1
            elif kalt2 in kprimes:
                real_var_simplifier[k] = kalt2
            elif kalt3 in kprimes:
                real_var_simplifier[k] = kalt3
            else:
                real_var_simplifier[k] = k
                kprimes.add(k)
        real_var_simplifiers[ircombo] = real_var_simplifier

    real_var_full_simplifier = {}
    for ircombo in real_var_simplifiers:
        real_var_full_simplifier.update(real_var_simplifiers[ircombo])

    # For each 4-tuple of irreps
    # create a system of symbolic solutions
    # and let sympy solve that.
    # For each 4-tuple of irreps
    # the end result is a dictionary
    # whose keys represent the dependent
    # brakets and whose values are the
    # relation that those dependent values
    # have with the independent brakets.
    # As such, when these dictionaries are
    # used on an expression, everything should
    # then be given in terms of indepedent brakets.

    if verbose:
        msg = "Solving for independent 4-symbol brakets ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    all_sols = {irc:[] for irc in great_identities}
    for ircombo in great_identities:
        zeros = all_zeros[ircombo]
        problem_vars = list(set(sum([list(identity.dict.keys()) for identity in great_identities[ircombo]],[])))
        big_ma = [awe.vec_in_basis(problem_vars) for awe in great_identities[ircombo]] 
        big_mat = sp.Matrix(big_ma)
        big_mat = sp.re(big_mat)+sp.I*sp.im(big_mat)
        rref_mat, pivots = big_mat.rref()
        num_rows = rref_mat.rows
        num_cols = rref_mat.cols
        rref_ma_non_zero = [rref_mat[row,:] for row in range(num_rows) if (sum(np.array(rref_mat[row,:])[0] == 0) != num_cols)]
        rref_mat_non_zero = sp.Matrix(rref_ma_non_zero)
        varvec = sp.Matrix(list(map(composite_symbol,problem_vars)))
        eqns = rref_mat_non_zero*varvec
        ssol = sp.solve(list(eqns), dict=True)
        all_sols[ircombo] = ssol
        assert len(ssol) in [0,1]
        if len(ssol) == 0:
            sol_dict = {}
        else:
            sol_dict = ssol[0]
        zero_addendum = {k:{} for k in all_zeros[ircombo]}
        sol_dict = {fourtuplerecovery(k):{fourtuplerecovery(s):v.coeff(s) for s in v.free_symbols} for k,v in sol_dict.items()}
        sol_dict.update(zero_addendum)
        all_sols[ircombo] = sol_dict

    # This final dictionary is unnecessary but
    # simplifies calling the replacements onto
    # a symbolic expression.
    if verbose:
        msg = "Creating a dictionary with all the 4-symbol replacements ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    super_solution = {}
    for ircombo in all_sols:
        super_solution.update(all_sols[ircombo])

    def simplifier(qet):
        simp_ket = Qet({})
        for k,v in qet.dict.items():
            simp_ket += Qet({real_var_full_simplifier[k]:v})
        true_qet = Qet({})
        for k,v in simp_ket.dict.items():
            if k in super_solution:
                true_qet += v*Qet(super_solution[k])
            else:
                true_qet += Qet({k:v})
        return true_qet

    def simplify_config_matrix(ir1ir2, S, ir3):
        config_matrix = config_matrices[ir1ir2][(S,ir3)]
        num_rows = len(config_matrix)
        simple_matrix = [[simplifier(config_matrix[row][col]) for col in range(num_rows)] for row in range(num_rows)]
        return simple_matrix
    
    if verbose:
        msg = "Simplifying configuration matrices ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    simple_config_matrices = {k:{} for k in config_matrices}
    for ir1ir2 in config_matrices.keys():
        for term_key in config_matrices[ir1ir2]:
            S, Γ3 = term_key
            simple_config_matrices[ir1ir2][term_key] = simplify_config_matrix(ir1ir2, S, Γ3)

    twotuplerecovery = fourtuplerecovery
    if verbose:
        msg = "Creating all 2-symbol identities ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1

    # this   integral_identities  dictionary  will  have  as  keys
    # 2-tuples  of  irreps  and  its  values  will  be lists whose
    # elements  are  2-tuples whose first elements are 2-tuples of
    # irrep  components  and  whose values are qets whose keys are
    # 2-tuples  of  irrep components and whose values are numeric.
    # These 2-tuples represent a braket with the an invariant operator
    # operator in between.

    integral_identities_2 = {}
    ir_mats = group.irrep_matrices
    for ir1, ir2 in product(*([group.irrep_labels]*2)):
        # To simplify calculations this part
        # can only be done over quadruples in a standard
        # order.
        # Whatever is missed here is then brough back in
        # by the reality relations.
        altorder1 = (ir2, ir1, ir1, ir4)
        if (altorder1 in integral_identities_2):
            continue
        integral_identity_sector = []
        components = [component_labels[ir] for ir in [ir1, ir2]]
        comp_to_idx = [{c: idx for idx, c in enumerate(component_labels[ir])} for ir in [ir1, ir2]]
        for R in group.generators + [sp.Symbol('{Ee}')]:
            R_id = {}
            for γ1, γ2 in product(*components):
                for γ1p, γ2p in product(*components):
                        val = sp.conjugate(ir_mats[ir1][R][comp_to_idx[0][γ1],comp_to_idx[0][γ1p]]) *\
                              ir_mats[ir2][R][comp_to_idx[1][γ2],comp_to_idx[1][γ2p]]
                        if val== 0:
                            continue
                        key = (γ1, γ2)
                        if key not in R_id.keys():
                            R_id[key] = []
                        R_id[key].append( Qet({(γ1p,γ2p): val}) )
            R_id_total = [(key, sum(R_id[key], Qet({}))) for key in R_id.keys()]
            R_id_total = [q for q in R_id_total if len(q[1].dict)>0]
            integral_identity_sector.extend(R_id_total)
        integral_identities_2[(ir1,ir2)] = integral_identity_sector

    # For solving the linear system it is convenient
    # to have everything on one side of the equation.
    if verbose:
        msg = "Creating set of 2 symbol identities ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    identities_2 = {}
    
    for ircombo in integral_identities_2:
        these_ids = []
        for v in integral_identities_2[ircombo]:
            lhs, rhs = v
            diff = Qet({lhs:1}) - rhs
            if len(diff.dict) > 0:
                these_ids.append(diff)
        identities_2[ircombo] = these_ids

    # If an equation has only one term, then
    # that immediately means that that term is zero.
    if verbose:
        msg = "Finding trivial zeros ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    # first determine which ones have to be zero
    better_identities_2 = {irc:[] for irc in identities_2}
    all_zeros_2 = {}
    for ircombo in identities_2:
        zeros = []
        for identity in identities_2[ircombo]:
            if len(identity.dict) == 1:
                zeros.append((list(identity.dict.keys())[0]))
            else:
                better_identities_2[ircombo].append(identity)
        all_zeros_2[ircombo] = zeros

    # use them to simplify things.
    if verbose:
        msg = "Using them to simplify things ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    great_identities_2 = {irc:[] for irc in identities_2}
    for ircombo in better_identities_2:
        for identity in better_identities_2[ircombo]:
            new_qet = Qet({})
            for k,v in identity.dict.items():
                if k in all_zeros_2[ircombo]:
                    continue
                else:
                    new_qet+= Qet({k: v})
            if len(new_qet.dict) == 0:
                continue
            great_identities_2[ircombo].append(new_qet)

    # Inside of a two-symbol braket one may do three exchanges
    # that must result in the same value. That if the wave
    # functions are assumed to be real-valued.
    if verbose:
        msg = "Creating reality identities ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    real_var_simplifiers_2 = {irc:[] for irc in great_identities_2}
    # this has to run over all the quadruples of irs
    kprimes = set()
    for ir1, ir2 in product(*([group.irrep_labels]*2)):
        real_var_simplifier = {}
        ircombo = (ir1, ir2)
        components = [component_labels[ir] for ir in ircombo]
        for γ1, γ2 in product(*components):
            k = (γ1, γ2)
            kalt = (γ2, γ1)
            # If for a given key I find that its
            # switched version has already been seen
            # Then that key has to be mapped to be
            # mapped to the key already present.
            if kalt in kprimes:
                real_var_simplifier[k] = kalt
            else:
                real_var_simplifier[k] = k
                kprimes.add(k)
        real_var_simplifiers_2[ircombo] = real_var_simplifier

    # For each 2-tuple of irreps
    # create a system of symbolic solutions
    # and let sympy solve that.
    # For each 2-tuple of irreps
    # the end result is a dictionary
    # whose keys represent the dependent
    # brakets and whose values are the
    # relation that those dependent values
    # have with the independent brakets.
    # As such, when these dictionaries are
    # used on an expression, everything should
    # then be given in terms of indepedent brakets.
    if verbose:
        msg = "Solving for independent 2-symbol brakets ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    all_sols_2 = {irc:[] for irc in great_identities_2}
    
    for ircombo in great_identities_2:
        zeros = all_zeros_2[ircombo]
        problem_vars = list(set(sum([list(identity.dict.keys()) for identity in great_identities_2[ircombo]],[])))
        big_ma = [awe.vec_in_basis(problem_vars) for awe in great_identities_2[ircombo]] 
        big_mat = sp.Matrix(big_ma)
        big_mat = sp.re(big_mat)+sp.I*sp.im(big_mat)
        rref_mat, pivots = big_mat.rref()
        num_rows = rref_mat.rows
        num_cols = rref_mat.cols
        rref_ma_non_zero = [rref_mat[row,:] for row in range(num_rows) if (sum(np.array(rref_mat[row,:])[0] == 0) != num_cols)]
        rref_mat_non_zero = sp.Matrix(rref_ma_non_zero)
        varvec = sp.Matrix(list(map(composite_symbol,problem_vars)))
        eqns = rref_mat_non_zero*varvec
        ssol = sp.solve(list(eqns), dict=True)
        all_sols[ircombo] = ssol
        assert len(ssol) in [0,1]
        if len(ssol) == 0:
            sol_dict = {}
        else:
            sol_dict = ssol[0]
        zero_addendum = {k:{} for k in all_zeros_2[ircombo]}
        sol_dict = {twotuplerecovery(k):{twotuplerecovery(s):v.coeff(s) for s in v.free_symbols} for k,v in sol_dict.items()}
        sol_dict.update(zero_addendum)
        all_sols_2[ircombo] = sol_dict

    # This final dictionary is unnecessary but
    # simplifies calling the replacements onto
    # a symbolic expression.
    if verbose:
        msg = "Creating a dictionary with all the 2-symbol replacements ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    super_solution_2 = {}
    for ircombo in all_sols_2:
        super_solution_2.update(all_sols_2[ircombo])

    # for a given electron config
    more_ids = {e_config:[] for e_config in simple_config_matrices}
    for e_config in simple_config_matrices:
        for term in simple_config_matrices[e_config]:
            the_matrix = simple_config_matrices[e_config][term]
            num_rows = len(the_matrix)
            num_cols = len(the_matrix)
            the_state_keys = config_supplement[e_config][term]
            the_key_matrix = {}
            for row_idx in range(num_rows):
                Γ3_row = the_state_keys[row_idx][2]
                γ3_row = the_state_keys[row_idx][3]
                for col_idx in range(num_cols):
                    Γ3_col = the_state_keys[col_idx][2]
                    γ3_col = the_state_keys[col_idx][3]
                    matrix_val = the_matrix[row_idx][col_idx]
                    the_key_matrix[(γ3_row,γ3_col)] = matrix_val
            # now go over the keys of super_solution_2
            # and if one of those keys matches with a key in the_key_matrix
            # do something about it
            for k in super_solution_2:
                if k in the_key_matrix:
                    v = super_solution_2[k]
                    # this matrix element
                    matrix_element = the_key_matrix[k]
                    # must be identified with the sum
                    # as given in v
                    matrix_equiv = sum([kv*the_key_matrix[km] for km, kv in v.items()],Qet({}))
                    identity = (matrix_element - matrix_equiv) #=0
                    if len(identity.dict) != 0:
                        more_ids[e_config].append(identity)
                        problem_vars = list(set(sum([list(identity.dict.keys()) for identity in more_ids[e_config]],[])))
    if verbose:
        msg = "Simplifying numeric values of qets ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    more_ids = {e_config:list(filter(lambda x: len(x.dict) > 0,list(map(simplify_qet,more_ids[e_config])))) for e_config in more_ids}
    
    if verbose:
        msg = "Solving for dependent vars in terms of independent ones ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    all_sols_2_4 = {irc:[] for irc in more_ids}
    
    for e_config in more_ids:
        problem_vars = list(set(sum([list(identity.dict.keys()) for identity in more_ids[e_config]],[])))
        big_ma = [awe.vec_in_basis(problem_vars) for awe in more_ids[e_config]] 
        big_mat = sp.Matrix(big_ma)
        big_mat = sp.re(big_mat)+sp.I*sp.im(big_mat)
        rref_mat, pivots = big_mat.rref()
        num_rows = rref_mat.rows
        num_cols = rref_mat.cols
        rref_ma_non_zero = [rref_mat[row,:] for row in range(num_rows) if (sum(np.array(rref_mat[row,:])[0] == 0) != num_cols)]
        rref_mat_non_zero = sp.Matrix(rref_ma_non_zero)
        varvec = sp.Matrix(list(map(composite_symbol, problem_vars)))
        eqns = rref_mat_non_zero*varvec
        ssol = sp.solve(list(eqns), dict=True)
        all_sols_2_4[e_config] = ssol
        assert len(ssol) in [0,1]
        if len(ssol) == 0:
            sol_dict = {}
        else:
            sol_dict = ssol[0]
        sol_dict = {twotuplerecovery(k):{twotuplerecovery(s):v.coeff(s) for s in v.free_symbols} for k,v in sol_dict.items()}
        all_sols_2_4[e_config] = sol_dict

    # flatten into a single dictionary of replacements
    fab_solution_2_4 = {}
    for e_config in all_sols_2_4:
        fab_solution_2_4.update(all_sols_2_4[e_config])
    if verbose:
        msg = "There are %d less independent variables ..."
        pbar.set_description("There are %d less independent variables ..." % len(fab_solution_2_4))
        pbar.refresh()
        pbar.n += 1
    
    def simplifier_f(qet):
        true_qet = Qet({})
        for k,v in qet.dict.items():
            if k in fab_solution_2_4:
                true_qet += v*Qet(fab_solution_2_4[k])
            else:
                true_qet += Qet({k:v})
        return true_qet
    
    def simplify_config_matrix_f(ir1ir2, S, ir3):
        config_matrix = simple_config_matrices[ir1ir2][(S,ir3)]
        num_rows = len(config_matrix)
        simple_matrix = [[simplifier_f(config_matrix[row][col]) for col in range(num_rows)] for row in range(num_rows)]
        #simple_matrix = sp.Matrix(simple_matrix)
        return simple_matrix
    
    if verbose:
        msg = "Making final simplifications ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    final_config_matrices = {k:{} for k in config_matrices}
    for ir1ir2 in config_matrices.keys():
        for term_key in config_matrices[ir1ir2]:
            S, Γ3 = term_key
            final_config_matrices[ir1ir2][term_key] = simplify_config_matrix_f(ir1ir2, S, Γ3)
    
    if verbose:
        msg = "Finished"
        pbar.set_description(msg)
        pbar.refresh()
    
    return final_config_matrices

In [376]:
O = term_energies('O')

(0/19) [00:00] 

In [386]:
term_key = (0,sp.Symbol('T_2'))

In [416]:
k = sp.Symbol('t_{2}')**2
for term_key in [(0,sp.Symbol('A_1')),
                (1,sp.Symbol('T_1')),
                (0,sp.Symbol('E')),
                (0,sp.Symbol('T_2'))]:
    display(sp.Matrix([[el.as_braket() for el in row] for row in O[k][term_key]])[0,0])

2*<{\xi}{\xi}|{\phi}{\phi}> + <{\xi}{\xi}|{\xi}{\xi}>

<{\xi}{\phi}|{\xi}{\phi}> - <{\xi}{\xi}|{\phi}{\phi}>

-<{\xi}{\xi}|{\phi}{\phi}> + <{\xi}{\xi}|{\xi}{\xi}>

<{\xi}{\phi}|{\xi}{\phi}> + <{\xi}{\xi}|{\phi}{\phi}>

In [418]:
k = sp.Symbol('t_{2}')**2
for term_key in O[k]:#[(1,sp.Symbol('T_{u}'))]#[0][0].as_braket()
    term_symb = sp.Symbol('{}^{%d}%s' % (term_key[0]*2+1,sp.latex(term_key[1])))
    display(term_symb)
    display(simple_config_matrices[k][term_key][0][0].as_braket())
    display(O[k][term_key][0][0].as_braket())

{}^{1}A_{1}

2*<{\xi}{\xi}|{\phi}{\phi}> + <{\xi}{\xi}|{\xi}{\xi}>

2*<{\xi}{\xi}|{\phi}{\phi}> + <{\xi}{\xi}|{\xi}{\xi}>

{}^{1}E

-<{\xi}{\xi}|{\phi}{\phi}> + <{\xi}{\xi}|{\xi}{\xi}>

...

<{\xi}{\phi}|{\xi}{\phi}> - <{\xi}{\xi}|{\phi}{\phi}>

<{\xi}{\phi}|{\xi}{\phi}> - <{\xi}{\xi}|{\phi}{\phi}>

{}^{1}T_{2}

<{\xi}{\phi}|{\xi}{\phi}> + <{\xi}{\xi}|{\phi}{\phi}>

<{\xi}{\phi}|{\xi}{\phi}> + <{\xi}{\xi}|{\phi}{\phi}>

In [431]:
k = sp.Symbol('e')**2
free_ints = set()
energies = {}
for term_key in O[k]:#[(1,sp.Symbol('T_{u}'))]#[0][0].as_braket()
    term_symb = sp.Symbol('{}^{%d}%s' % (term_key[0]*2+1,sp.latex(term_key[1])))
    display(term_symb)
    # display(simple_config_matrices[k][term_key][0][0].as_braket())
    print("simple")
    display(sp.Matrix([[l.as_braket() for l in row] for row in simple_config_matrices[k][term_key]]))
    print("final")
    display(sp.Matrix([[l.as_braket() for l in row] for row in O[k][term_key]]))
#     display(O[k][term_key][0][0].as_braket())
    energies[term_key] = O[k][term_key][0][0]
    free_ints.update(list(O[k][term_key][0][0].dict.keys()))
print("----")
for free_int in free_ints:
    display(Qet({free_int:1}).as_braket())

{}^{1}A_{1}

simple


Matrix([[<{\zeta}{\zeta}|{\gamma}{\gamma}> + <{\zeta}{\zeta}|{\zeta}{\zeta}>]])

final


Matrix([[<{\zeta}{\zeta}|{\gamma}{\gamma}> + <{\zeta}{\zeta}|{\zeta}{\zeta}>]])

...

final


Matrix([
[-<{\zeta}{\zeta}|{\gamma}{\gamma}> + <{\zeta}{\zeta}|{\zeta}{\zeta}>,                                                                    0],
[                                                                   0, -<{\zeta}{\zeta}|{\gamma}{\gamma}> + <{\zeta}{\zeta}|{\zeta}{\zeta}>]])

----


<{\zeta}{\zeta}|{\gamma}{\gamma}>

<{\zeta}{\zeta}|{\zeta}{\zeta}>

In [445]:
k = sp.Symbol('t_{2}')*sp.Symbol('e')
free_ints = set()
energies = {}
for term_key in O[k]:#[(1,sp.Symbol('T_{u}'))]#[0][0].as_braket()
    term_symb = sp.Symbol('{}^{%d}%s' % (term_key[0]*2+1,sp.latex(term_key[1])))
    display(term_symb)
    # display(simple_config_matrices[k][term_key][0][0].as_braket())
    print("simple")
    display(sp.Matrix([[l.as_braket() for l in row] for row in simple_config_matrices[k][term_key]]))
    print("final")
    display(sp.Matrix([[l.as_braket() for l in row] for row in O[k][term_key]]))
#     display(O[k][term_key][0][0].as_braket())
    energies[term_key] = O[k][term_key][0][0]
    free_ints.update(list(O[k][term_key][0][0].dict.keys()))
print("----")
for free_int in free_ints:
    display(Qet({free_int:1}).as_braket())

{}^{3}T_{1}

simple


Matrix([
[<{\zeta}{\xi}|{\zeta}{\xi}> - <{\zeta}{\zeta}|{\xi}{\xi}>,                                                         0,                                                         0,                                                         0,                                                         0,                                                         0,                                                         0,                                                         0,                                                         0],
[                                                        0, <{\zeta}{\xi}|{\zeta}{\xi}> - <{\zeta}{\zeta}|{\xi}{\xi}>,                                                         0,                                                         0,                                                         0,                                                         0,                                                         0,                                             

final


Matrix([
[<{\zeta}{\xi}|{\zeta}{\xi}> - <{\zeta}{\zeta}|{\xi}{\xi}>,                                                         0,                                                         0,                                                         0,                                                         0,                                                         0,                                                         0,                                                         0,                                                         0],
[                                                        0, <{\zeta}{\xi}|{\zeta}{\xi}> - <{\zeta}{\zeta}|{\xi}{\xi}>,                                                         0,                                                         0,                                                         0,                                                         0,                                                         0,                                             

...

----


<{\zeta}{\zeta}|{\phi}{\phi}>

<{\zeta}{\xi}|{\zeta}{\xi}>

<{\zeta}{\phi}|{\zeta}{\phi}>

<{\zeta}{\zeta}|{\xi}{\xi}>

In [440]:
free_integrals = list(set(sum([list(q.dict.keys()) for q in list(energies.values())],[])))

In [444]:
energies[(0,sp.Symbol('A_1'))]

Qet({({\zeta}, {\zeta}, {\zeta}, {\zeta}): 1, ({\zeta}, {\zeta}, {\gamma}, {\gamma}): 1})

#### More debugging

In [ ]:
def simplify_qet(qet):
    new_dict = {k:sp.simplify(v) for k,v in qet.dict.items()}
    return Qet(new_dict)

In [367]:
group_label = 'O'
verbose = True
stages = 19 # this is how many distinct parts this algorithm has
if verbose:
    pbar = tqdm(range(stages), bar_format='({n:d}/{total:d}) [{elapsed}] {desc}')
terms = twoe_terms[group_label]
group = CPGs.get_group_by_label(group_label)
component_labels = {k:list(v.values()) for k,v in new_labels[group_label].items()}
# for a given set of terms
# split all the enclosed states
# in groups of electron configurations
configs = {}
config_supplement = {}
for term_key, term in terms.items():
    for state_key, state in zip(term.state_keys, term.states):
        (Γ1, Γ2, Γ3, γ3, S, mSz) = state_key
        α = sp.Symbol(sp.latex(Γ1).lower())*sp.Symbol(sp.latex(Γ2).lower())
        if α not in configs.keys():
            configs[α] = {}
            config_supplement[α] = {}
        if (S,Γ3) not in configs[α].keys():
            configs[α][(S,Γ3)] = []
            config_supplement[α][(S,Γ3)] = []
        configs[α][(S,Γ3)].append(state)
        config_supplement[α][(S,Γ3)].append(state_key)

def overlinesqueegee(s):
    '''
    Going back from the overline shorthand for spin down,
    acting on a single set of quantum numbers.
    '''
    if 'overline' in str(s):
        spin = -sp.S(1)/2
        comp = sp.Symbol(str(s).replace('\\overline{','')[:-1])
    else:
        spin = sp.S(1)/2
        comp = s
    return (comp, spin)
def spin_restoration(qet):
    '''
    Going back from the overline shorthand for spin down,
    acting on a qet.
    '''
    new_dict = {}
    for k,v in qet.dict.items():
        k = (*overlinesqueegee(k[0]),*overlinesqueegee(k[1]))
        new_dict[k] = v
    return Qet(new_dict)

# within each configuration
config_matrices = {}
for config in configs.keys():
    # display(config)
    # within each term
    term_matrices = {}
    for term_key, states in configs[config].items():
        # find the brackets between all possible pairs of states
        ham_matrix = []
        for state0 in states:
            ham_row = []
            state0 = spin_restoration(state0)
            for state1 in states:
                # recover the spin for the determinantal states
                pstate = state1
                state1 = spin_restoration(state1)
                # determine the det braket between these two states (an operator in between is assumed and omitted)
                detbraket = state0.dual()*state1 # changehere
                # simplify this braket of detstates into brakets between regular states
                regbraket = Qet({})
                for k,v in detbraket.dict.items():
                    γ1, m1, γ2, m2, γ1p, m1p, γ2p, m2p = k
                    # k0 = (γ1, m1, γ2, m2, γ1p, m1p, γ2p, m2p)
                    # k1 = (γ1, m1, γ2, m2, γ2p, m2p, γ1p, m1p)
                    k0 = (γ1, γ2, γ1p, γ2p)
                    k1 = (γ1, γ2, γ2p, γ1p)
                    # enforce orthogonality wrt. spin
                    if (m1 == m1p) and (m2 == m2p):
                        qplus = Qet({k0:v}) 
                    else:
                        qplus = Qet({})
                    if (m1 == m2p) and (m2 == m1p):
                        qminus = Qet({k1:-v}) 
                    else:
                        qminus = Qet({})
                    regbraket += (qplus+qminus)
                ham_row.append(regbraket)
            ham_matrix.append(ham_row)
        term_matrices[term_key] = ham_matrix
    config_matrices[config] = term_matrices

def composite_symbol(x):
    return sp.Symbol('(%s)'%(','.join(list(map(sp.latex,x)))))
def fourtuplerecovery(ft):
    '''the inverse of composite_symbol'''
    return tuple(map(sp.Symbol,sp.latex(ft)[1:-1].split(',')))

if verbose:
    msg = "Creating all 4-symbol identities ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

# this   integral_identities  dictionary  will  have  as  keys
# 4-tuples  of  irreps  and  its  values  will  be lists whose
# elements  are  2-tuples whose first elements are 4-tuples of
# irrep  components  and  whose values are qets whose keys are
# 4-tuples  of  irrep components and whose values are numeric.
# These 4-tuples represent a braket with the Coulomb repulsion
# operator in between.

integral_identities = {}
ir_mats = group.irrep_matrices
for ir1, ir2, ir3, ir4 in product(*([group.irrep_labels]*4)):
    # To simplify calculations this part
    # may only be done over quadruples in a standard
    # order.
    # Whatever is missed here is then brough back in
    # by the reality relations.
    altorder1 = (ir3, ir2, ir1, ir4)
    altorder2 = (ir1, ir4, ir3, ir2)
    altorder3 = (ir3, ir4, ir1, ir2)
    if (altorder1 in integral_identities) or\
       (altorder2 in integral_identities) or\
       (altorder3 in integral_identities):
        continue
    integral_identity_sector = []
    components = [component_labels[ir] for ir in [ir1, ir2, ir3, ir4]]
    comp_to_idx = [{c: idx for idx, c in enumerate(component_labels[ir])} for ir in [ir1, ir2, ir3, ir4]]
    for R in group.generators + [sp.Symbol('{Ee}')]:
        R_id = {}
        for γ1, γ2, γ3, γ4 in product(*components):
            for γ1p, γ2p, γ3p, γ4p in product(*components):
#                         val = sp.conjugate(ir_mats[ir1][R][comp_to_idx[0][γ1],comp_to_idx[0][γ1p]]) *\
#                               sp.conjugate(ir_mats[ir2][R][comp_to_idx[1][γ2],comp_to_idx[1][γ2p]]) *\
#                               ir_mats[ir3][R][comp_to_idx[2][γ3],comp_to_idx[2][γ3p]] *\
#                               ir_mats[ir4][R][comp_to_idx[3][γ4],comp_to_idx[3][γ4p]]
                    val = sp.conjugate(ir_mats[ir1][R][comp_to_idx[0][γ1],comp_to_idx[0][γ1p]]) *\
                      sp.conjugate(ir_mats[ir2][R][comp_to_idx[1][γ2],comp_to_idx[1][γ2p]]) *\
                      ir_mats[ir3][R][comp_to_idx[2][γ3],comp_to_idx[2][γ3p]] *\
                      ir_mats[ir4][R][comp_to_idx[3][γ4],comp_to_idx[3][γ4p]]
                    if val== 0:
                        continue
                    key = (γ1, γ2, γ3, γ4)
                    if key not in R_id.keys():
                        R_id[key] = []
                    R_id[key].append( Qet({(γ1p,γ2p,γ3p,γ4p): val}) )
        R_id_total = [(key, sum(R_id[key], Qet({}))) for key in R_id.keys()]
        R_id_total = [q for q in R_id_total if len(q[1].dict)>0]
        integral_identity_sector.extend(R_id_total)
    integral_identities[(ir1,ir2,ir3,ir4)] = integral_identity_sector

# For solving the linear system it is convenient
# to have everything on one side of the equation.
if verbose:
    msg = "Creating set of identities ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

identities = {}
for ircombo in integral_identities:
    these_ids = []
    for v in integral_identities[ircombo]:
        lhs, rhs = v
        diff = Qet({lhs:1}) - rhs
        if len(diff.dict) > 0:
            these_ids.append(diff)
    identities[ircombo] = these_ids

# If an equation has only one key, then
# that immediately means that that braket is zero.
if verbose:
    msg = "Finding trivial zeros ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

# first determine which ones have to be zero
better_identities = {irc:[] for irc in identities}
all_zeros = {}
for ircombo in identities:
    zeros = []
    for identity in identities[ircombo]:
        if len(identity.dict) == 1:
            zeros.append((list(identity.dict.keys())[0]))
        else:
            better_identities[ircombo].append(identity)
    all_zeros[ircombo] = zeros

# use them to simplify things.
if verbose:
    msg = "Using them to simplify things ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

great_identities = {irc:[] for irc in identities}
for ircombo in better_identities:
    for identity in better_identities[ircombo]:
        new_qet = Qet({})
        for k,v in identity.dict.items():
            if k in all_zeros[ircombo]:
                continue
            else:
                new_qet+= Qet({k: v})
        if len(new_qet.dict) == 0:
            continue
        great_identities[ircombo].append(new_qet)

# Inside of a four-symbol braket one may do three exchanges
# that must result in the same value. That if the wave
# functions are assumed to be real-valued.

if verbose:
    msg = "Creating reality identities ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

real_var_simplifiers = {irc:[] for irc in great_identities}
kprimes = set()
# this has to run over all the quadruples of irs
for ir1, ir2, ir3, ir4 in product(*([group.irrep_labels]*4)):
    real_var_simplifier = {}
    ircombo = (ir1, ir2, ir3, ir4)
    components = [component_labels[ir] for ir in ircombo]
    for γ1, γ2, γ3, γ4 in product(*components):
        k = (γ1, γ2, γ3, γ4)
        kalt1 = (γ3, γ2, γ1, γ4)
        kalt2 = (γ1, γ4, γ3, γ2)
        kalt3 = (γ3, γ4, γ1, γ2)
        if kalt1 in kprimes:
            real_var_simplifier[k] = kalt1
        elif kalt2 in kprimes:
            real_var_simplifier[k] = kalt2
        elif kalt3 in kprimes:
            real_var_simplifier[k] = kalt3
        else:
            real_var_simplifier[k] = k
            kprimes.add(k)
    real_var_simplifiers[ircombo] = real_var_simplifier

real_var_full_simplifier = {}
for ircombo in real_var_simplifiers:
    real_var_full_simplifier.update(real_var_simplifiers[ircombo])

# For each 4-tuple of irreps
# create a system of symbolic solutions
# and let sympy solve that.
# For each 4-tuple of irreps
# the end result is a dictionary
# whose keys represent the dependent
# brakets and whose values are the
# relation that those dependent values
# have with the independent brakets.
# As such, when these dictionaries are
# used on an expression, everything should
# then be given in terms of indepedent brakets.

if verbose:
    msg = "Solving for independent 4-symbol brakets ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

all_sols = {irc:[] for irc in great_identities}
for ircombo in great_identities:
    zeros = all_zeros[ircombo]
    problem_vars = list(set(sum([list(identity.dict.keys()) for identity in great_identities[ircombo]],[])))
    big_ma = [awe.vec_in_basis(problem_vars) for awe in great_identities[ircombo]] 
    big_mat = sp.Matrix(big_ma)
    big_mat = sp.re(big_mat)+sp.I*sp.im(big_mat)
    rref_mat, pivots = big_mat.rref()
    num_rows = rref_mat.rows
    num_cols = rref_mat.cols
    rref_ma_non_zero = [rref_mat[row,:] for row in range(num_rows) if (sum(np.array(rref_mat[row,:])[0] == 0) != num_cols)]
    rref_mat_non_zero = sp.Matrix(rref_ma_non_zero)
    varvec = sp.Matrix(list(map(composite_symbol,problem_vars)))
    eqns = rref_mat_non_zero*varvec
    ssol = sp.solve(list(eqns), dict=True)
    all_sols[ircombo] = ssol
    assert len(ssol) in [0,1]
    if len(ssol) == 0:
        sol_dict = {}
    else:
        sol_dict = ssol[0]
    zero_addendum = {k:{} for k in all_zeros[ircombo]}
    sol_dict = {fourtuplerecovery(k):{fourtuplerecovery(s):v.coeff(s) for s in v.free_symbols} for k,v in sol_dict.items()}
    sol_dict.update(zero_addendum)
    all_sols[ircombo] = sol_dict

# This final dictionary is unnecessary but
# simplifies calling the replacements onto
# a symbolic expression.
if verbose:
    msg = "Creating a dictionary with all the 4-symbol replacements ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

super_solution = {}
for ircombo in all_sols:
    super_solution.update(all_sols[ircombo])

def simplifier(qet):
    simp_ket = Qet({})
    for k,v in qet.dict.items():
        simp_ket += Qet({real_var_full_simplifier[k]:v})
    true_qet = Qet({})
    for k,v in simp_ket.dict.items():
        if k in super_solution:
            true_qet += v*Qet(super_solution[k])
        else:
            true_qet += Qet({k:v})
    return true_qet

def simplify_config_matrix(ir1ir2, S, ir3):
    config_matrix = config_matrices[ir1ir2][(S,ir3)]
    num_rows = len(config_matrix)
    simple_matrix = [[simplifier(config_matrix[row][col]) for col in range(num_rows)] for row in range(num_rows)]
    return simple_matrix

if verbose:
    msg = "Simplifying configuration matrices ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

simple_config_matrices = {k:{} for k in config_matrices}
for ir1ir2 in config_matrices.keys():
    for term_key in config_matrices[ir1ir2]:
        S, Γ3 = term_key
        simple_config_matrices[ir1ir2][term_key] = simplify_config_matrix(ir1ir2, S, Γ3)

twotuplerecovery = fourtuplerecovery
if verbose:
    msg = "Creating all 2-symbol identities ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

# this   integral_identities  dictionary  will  have  as  keys
# 2-tuples  of  irreps  and  its  values  will  be lists whose
# elements  are  2-tuples whose first elements are 2-tuples of
# irrep  components  and  whose values are qets whose keys are
# 2-tuples  of  irrep components and whose values are numeric.
# These 2-tuples represent a braket with the an invariant operator
# operator in between.

In [372]:
verbose = False
integral_identities_2 = {}
ir_mats = group.irrep_matrices
for ir1, ir2 in product(*([group.irrep_labels]*2)):
    # To simplify calculations this part
    # can only be done over quadruples in a standard
    # order.
    # Whatever is missed here is then brough back in
    # by the reality relations.
    altorder1 = (ir2, ir1, ir1, ir4)
    if (altorder1 in integral_identities_2):
        continue
    integral_identity_sector = []
    components = [component_labels[ir] for ir in [ir1, ir2]]
    comp_to_idx = [{c: idx for idx, c in enumerate(component_labels[ir])} for ir in [ir1, ir2]]
    for R in group.generators + [sp.Symbol('{Ee}')]:
        R_id = {}
        for γ1, γ2 in product(*components):
            for γ1p, γ2p in product(*components):
                    val = sp.conjugate(ir_mats[ir1][R][comp_to_idx[0][γ1],comp_to_idx[0][γ1p]]) *\
                          ir_mats[ir2][R][comp_to_idx[1][γ2],comp_to_idx[1][γ2p]]
#                     val = sp.conjugate(ir_mats[ir1][R][comp_to_idx[0][γ1p],comp_to_idx[0][γ1]]) *\
#                           ir_mats[ir2][R][comp_to_idx[1][γ2],comp_to_idx[1][γ2p]]
                    if val== 0:
                        continue
                    key = (γ1, γ2)
                    if key not in R_id.keys():
                        R_id[key] = []
                    R_id[key].append( Qet({(γ1p,γ2p): val}) )
        R_id_total = [(key, sum(R_id[key], Qet({}))) for key in R_id.keys()]
        R_id_total = [q for q in R_id_total if len(q[1].dict)>0]
        integral_identity_sector.extend(R_id_total)
    integral_identities_2[(ir1,ir2)] = integral_identity_sector

# For solving the linear system it is convenient
# to have everything on one side of the equation.
if verbose:
    msg = "Creating set of 2 symbol identities ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1
identities_2 = {}

for ircombo in integral_identities_2:
    these_ids = []
    for v in integral_identities_2[ircombo]:
        lhs, rhs = v
        diff = Qet({lhs:1}) - rhs
        if len(diff.dict) > 0:
            these_ids.append(diff)
    identities_2[ircombo] = these_ids

# If an equation has only one term, then
# that immediately means that that term is zero.
if verbose:
    msg = "Finding trivial zeros ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

# first determine which ones have to be zero
better_identities_2 = {irc:[] for irc in identities_2}
all_zeros_2 = {}
for ircombo in identities_2:
    zeros = []
    for identity in identities_2[ircombo]:
        if len(identity.dict) == 1:
            zeros.append((list(identity.dict.keys())[0]))
        else:
            better_identities_2[ircombo].append(identity)
    all_zeros_2[ircombo] = zeros

# use them to simplify things.
if verbose:
    msg = "Using them to simplify things ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

great_identities_2 = {irc:[] for irc in identities_2}
for ircombo in better_identities_2:
    for identity in better_identities_2[ircombo]:
        new_qet = Qet({})
        for k,v in identity.dict.items():
            if k in all_zeros_2[ircombo]:
                continue
            else:
                new_qet+= Qet({k: v})
        if len(new_qet.dict) == 0:
            continue
        great_identities_2[ircombo].append(new_qet)

# Inside of a two-symbol braket one may do three exchanges
# that must result in the same value. That if the wave
# functions are assumed to be real-valued.
if verbose:
    msg = "Creating reality identities ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1
real_var_simplifiers_2 = {irc:[] for irc in great_identities_2}
# this has to run over all the quadruples of irs
kprimes = set()
for ir1, ir2 in product(*([group.irrep_labels]*2)):
    real_var_simplifier = {}
    ircombo = (ir1, ir2)
    components = [component_labels[ir] for ir in ircombo]
    for γ1, γ2 in product(*components):
        k = (γ1, γ2)
        kalt = (γ2, γ1)
        # If for a given key I find that its
        # switched version has already been seen
        # Then that key has to be mapped to be
        # mapped to the key already present.
        if kalt in kprimes:
            real_var_simplifier[k] = kalt
        else:
            real_var_simplifier[k] = k
            kprimes.add(k)
    real_var_simplifiers_2[ircombo] = real_var_simplifier

# For each 2-tuple of irreps
# create a system of symbolic solutions
# and let sympy solve that.
# For each 2-tuple of irreps
# the end result is a dictionary
# whose keys represent the dependent
# brakets and whose values are the
# relation that those dependent values
# have with the independent brakets.
# As such, when these dictionaries are
# used on an expression, everything should
# then be given in terms of indepedent brakets.
if verbose:
    msg = "Solving for independent 2-symbol brakets ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1
all_sols_2 = {irc:[] for irc in great_identities_2}

for ircombo in great_identities_2:
    zeros = all_zeros_2[ircombo]
    problem_vars = list(set(sum([list(identity.dict.keys()) for identity in great_identities_2[ircombo]],[])))
    big_ma = [awe.vec_in_basis(problem_vars) for awe in great_identities_2[ircombo]] 
    big_mat = sp.Matrix(big_ma)
    big_mat = sp.re(big_mat)+sp.I*sp.im(big_mat)
    rref_mat, pivots = big_mat.rref()
    num_rows = rref_mat.rows
    num_cols = rref_mat.cols
    rref_ma_non_zero = [rref_mat[row,:] for row in range(num_rows) if (sum(np.array(rref_mat[row,:])[0] == 0) != num_cols)]
    rref_mat_non_zero = sp.Matrix(rref_ma_non_zero)
    varvec = sp.Matrix(list(map(composite_symbol,problem_vars)))
    eqns = rref_mat_non_zero*varvec
    ssol = sp.solve(list(eqns), dict=True)
    all_sols[ircombo] = ssol
    assert len(ssol) in [0,1]
    if len(ssol) == 0:
        sol_dict = {}
    else:
        sol_dict = ssol[0]
    zero_addendum = {k:{} for k in all_zeros_2[ircombo]}
    sol_dict = {twotuplerecovery(k):{twotuplerecovery(s):v.coeff(s) for s in v.free_symbols} for k,v in sol_dict.items()}
    sol_dict.update(zero_addendum)
    all_sols_2[ircombo] = sol_dict

# This final dictionary is unnecessary but
# simplifies calling the replacements onto
# a symbolic expression.
if verbose:
    msg = "Creating a dictionary with all the 2-symbol replacements ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

super_solution_2 = {}
for ircombo in all_sols_2:
    super_solution_2.update(all_sols_2[ircombo])

# for a given electron config
more_ids = {e_config:[] for e_config in simple_config_matrices}
for e_config in simple_config_matrices:
    for term in simple_config_matrices[e_config]:
        the_matrix = simple_config_matrices[e_config][term]
        num_rows = len(the_matrix)
        num_cols = len(the_matrix)
        the_state_keys = config_supplement[e_config][term]
        the_key_matrix = {}
        for row_idx in range(num_rows):
            Γ3_row = the_state_keys[row_idx][2]
            γ3_row = the_state_keys[row_idx][3]
            for col_idx in range(num_cols):
                Γ3_col = the_state_keys[col_idx][2]
                γ3_col = the_state_keys[col_idx][3]
                matrix_val = the_matrix[row_idx][col_idx]
                the_key_matrix[(γ3_row,γ3_col)] = matrix_val
        # now go over the keys of super_solution_2
        # and if one of those keys matches with a key in the_key_matrix
        # do something about it
        for k in super_solution_2:
            if k in the_key_matrix:
                v = super_solution_2[k]
                # this matrix element
                matrix_element = the_key_matrix[k]
                # must be identified with the sum
                # as given in v
                matrix_equiv = sum([kv*the_key_matrix[km] for km, kv in v.items()],Qet({}))
                identity = (matrix_element - matrix_equiv) #=0
                if len(identity.dict) != 0:
                    more_ids[e_config].append(identity)
                    problem_vars = list(set(sum([list(identity.dict.keys()) for identity in more_ids[e_config]],[])))
if verbose:
    msg = "Simplifying numeric values of qets ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

more_ids = {e_config:list(filter(lambda x: len(x.dict) > 0,list(map(simplify_qet,more_ids[e_config])))) for e_config in more_ids}

if verbose:
    msg = "Solving for dependent vars in terms of independent ones ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

all_sols_2_4 = {irc:[] for irc in more_ids}

for e_config in more_ids:
    problem_vars = list(set(sum([list(identity.dict.keys()) for identity in more_ids[e_config]],[])))
    big_ma = [awe.vec_in_basis(problem_vars) for awe in more_ids[e_config]] 
    big_mat = sp.Matrix(big_ma)
    big_mat = sp.re(big_mat)+sp.I*sp.im(big_mat)
    rref_mat, pivots = big_mat.rref()
    num_rows = rref_mat.rows
    num_cols = rref_mat.cols
    rref_ma_non_zero = [rref_mat[row,:] for row in range(num_rows) if (sum(np.array(rref_mat[row,:])[0] == 0) != num_cols)]
    rref_mat_non_zero = sp.Matrix(rref_ma_non_zero)
    varvec = sp.Matrix(list(map(composite_symbol, problem_vars)))
    eqns = rref_mat_non_zero*varvec
    ssol = sp.solve(list(eqns), dict=True)
    all_sols_2_4[e_config] = ssol
    assert len(ssol) in [0,1]
    if len(ssol) == 0:
        sol_dict = {}
    else:
        sol_dict = ssol[0]
    sol_dict = {twotuplerecovery(k):{twotuplerecovery(s):v.coeff(s) for s in v.free_symbols} for k,v in sol_dict.items()}
    all_sols_2_4[e_config] = sol_dict

# flatten into a single dictionary of replacements
fab_solution_2_4 = {}
for e_config in all_sols_2_4:
    fab_solution_2_4.update(all_sols_2_4[e_config])
if verbose:
    msg = "There are %d less independent variables ..."
    pbar.set_description("There are %d less independent variables ..." % len(fab_solution_2_4))
    pbar.refresh()
    pbar.n += 1

def simplifier_f(qet):
    true_qet = Qet({})
    for k,v in qet.dict.items():
        if k in fab_solution_2_4:
            true_qet += v*Qet(fab_solution_2_4[k])
        else:
            true_qet += Qet({k:v})
    return true_qet

def simplify_config_matrix_f(ir1ir2, S, ir3):
    config_matrix = simple_config_matrices[ir1ir2][(S,ir3)]
    num_rows = len(config_matrix)
    simple_matrix = [[simplifier_f(config_matrix[row][col]) for col in range(num_rows)] for row in range(num_rows)]
    #simple_matrix = sp.Matrix(simple_matrix)
    return simple_matrix

if verbose:
    msg = "Making final simplifications ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

final_config_matrices = {k:{} for k in config_matrices}
for ir1ir2 in config_matrices.keys():
    for term_key in config_matrices[ir1ir2]:
        S, Γ3 = term_key
        final_config_matrices[ir1ir2][term_key] = simplify_config_matrix_f(ir1ir2, S, Γ3)

if verbose:
    msg = "Finished"
    pbar.set_description(msg)
    pbar.refresh()

In [373]:
O = final_config_matrices

In [374]:
O[sp.Symbol('t_{2}')**2][(1,sp.Symbol('T_1'))]

[[Qet({({\xi}, {\phi}, {\xi}, {\phi}): 1, ({\xi}, {\xi}, {\phi}, {\phi}): -1}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({})],
 [Qet({}),
  Qet({({\xi}, {\phi}, {\xi}, {\phi}): 1, ({\xi}, {\xi}, {\phi}, {\phi}): -1}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({})],
 [Qet({}),
  Qet({}),
  Qet({({\xi}, {\phi}, {\xi}, {\phi}): 1, ({\xi}, {\xi}, {\phi}, {\phi}): -1}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({})],
 [Qet({}),
  Qet({}),
  Qet({}),
  Qet({({\xi}, {\phi}, {\xi}, {\phi}): 1, ({\xi}, {\xi}, {\phi}, {\phi}): -1}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({})],
 [Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({({\xi}, {\phi}, {\xi}, {\phi}): 1, ({\xi}, {\xi}, {\phi}, {\phi}): -1}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({})],
 [Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({({\xi}, {\phi}, {\xi}, {\phi}): 1, ({\xi}, {\xi}, {\phi}, {\phi}): -1}),
  Qet({}),
  Qet({})

#### DEBUG

In [217]:
for R in group.generators:
    print(R)
    display(group.operations_matrices[R])
    print(group.operations_matrices[R].det())

C_{4x}


Matrix([
[1,  0, 0],
[0,  0, 1],
[0, -1, 0]])

1
C_{2a}


Matrix([
[0, 1,  0],
[1, 0,  0],
[0, 0, -1]])

1


In [212]:
for R in group.generators:
    print(R)
    for irrep in group.irrep_labels:
        print(irrep)
        display(group.irrep_matrices[irrep][R])

C_{4x}
A_1


Matrix([[1]])

A_2


Matrix([[-1]])

E


...

Matrix([
[1,  0],
[0, -1]])

T_1


Matrix([
[ 0,  0, -1],
[ 0, -1,  0],
[-1,  0,  0]])

T_2


Matrix([
[1, 0, 0],
[0, 0, 1],
[0, 1, 0]])

In [344]:
def simplify_qet(qet):
    new_dict = {k:sp.simplify(v) for k,v in qet.dict.items()}
    return Qet(new_dict)

verbose = True
group_label = 'O'
stages = 19 # this is how many distinct parts this algorithm has
if verbose:
    pbar = tqdm(range(stages), bar_format='({n:d}/{total:d}) [{elapsed}] {desc}')
terms = twoe_terms[group_label]
group = CPGs.get_group_by_label(group_label)
component_labels = {k:list(v.values()) for k,v in new_labels[group_label].items()}

# for a given set of terms
# split all the enclosed states
# in groups of electron configurations

configs = {}
config_supplement = {}
for term_key, term in terms.items():
    for state_key, state in zip(term.state_keys, term.states):
        (Γ1, Γ2, Γ3, γ3, S, mSz) = state_key
        α = sp.Symbol(sp.latex(Γ1).lower())*sp.Symbol(sp.latex(Γ2).lower())
        if α not in configs.keys():
            configs[α] = {}
            config_supplement[α] = {}
        if (S,Γ3) not in configs[α].keys():
            configs[α][(S,Γ3)] = []
            config_supplement[α][(S,Γ3)] = []
        configs[α][(S,Γ3)].append(state)
        config_supplement[α][(S,Γ3)].append(state_key)

def overlinesqueegee(s):
    '''
    Going back from the overline shorthand for spin down,
    acting on a single set of quantum numbers.
    '''
    if 'overline' in str(s):
        spin = -sp.S(1)/2
        comp = sp.Symbol(str(s).replace('\\overline{','')[:-1])
    else:
        spin = sp.S(1)/2
        comp = s
    return (comp, spin)
def spin_restoration(qet):
    '''
    Going back from the overline shorthand for spin down,
    acting on a qet.
    '''
    new_dict = {}
    for k,v in qet.dict.items():
        k = (*overlinesqueegee(k[0]),*overlinesqueegee(k[1]))
        new_dict[k] = v
    return Qet(new_dict)

# within each configuration
config_matrices = {}
for config in configs.keys():
    # display(config)
    # within each term
    term_matrices = {}
    for term_key, states in configs[config].items():
        # find the brackets between all possible pairs of states
        ham_matrix = []
        for state0 in states:
            ham_row = []
            state0 = spin_restoration(state0)
            for state1 in states:
                # recover the spin for the determinantal states
                pstate = state1
                state1 = spin_restoration(state1)
                # determine the det braket between these two states (an operator in between is assumed and omitted)
                detbraket = state0.dual()*state1 # changehere
                # simplify this braket of detstates into brakets between regular states
                regbraket = Qet({})
                for k,v in detbraket.dict.items():
                    γ1, m1, γ2, m2, γ1p, m1p, γ2p, m2p = k
                    # k0 = (γ1, m1, γ2, m2, γ1p, m1p, γ2p, m2p)
                    # k1 = (γ1, m1, γ2, m2, γ2p, m2p, γ1p, m1p)
                    k0 = (γ1, γ2, γ1p, γ2p)
                    k1 = (γ1, γ2, γ2p, γ1p)
                    # enforce orthogonality wrt. spin
                    if (m1 == m1p) and (m2 == m2p):
                        qplus = Qet({k0:v}) 
                    else:
                        qplus = Qet({})
                    if (m1 == m2p) and (m2 == m1p):
                        qminus = Qet({k1:-v}) 
                    else:
                        qminus = Qet({})
                    regbraket += (qplus+qminus)
                ham_row.append(regbraket)
            ham_matrix.append(ham_row)
        term_matrices[term_key] = ham_matrix
    config_matrices[config] = term_matrices

def composite_symbol(x):
    return sp.Symbol('(%s)'%(','.join(list(map(sp.latex,x)))))
def fourtuplerecovery(ft):
    '''the inverse of composite_symbol'''
    return tuple(map(sp.Symbol,sp.latex(ft)[1:-1].split(',')))

if verbose:
    msg = "Creating all 4-symbol identities ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

# this   integral_identities  dictionary  will  have  as  keys
# 4-tuples  of  irreps  and  its  values  will  be lists whose
# elements  are  2-tuples whose first elements are 4-tuples of
# irrep  components  and  whose values are qets whose keys are
# 4-tuples  of  irrep components and whose values are numeric.
# These 4-tuples represent a braket with the Coulomb repulsion
# operator in between.

integral_identities = {}
ir_mats = group.irrep_matrices
for ir1, ir2, ir3, ir4 in product(*([group.irrep_labels]*4)):
# for ir1, ir2, ir3, ir4 in [(sp.Symbol('T_2'),sp.Symbol('T_2'),sp.Symbol('T_2'),sp.Symbol('T_2'))]:
    # To simplify calculations this part
    # may only be done over quadruples in a standard
    # order.
    # Whatever is missed here is then brough back in
    # by the reality relations.
    altorder1 = (ir3, ir2, ir1, ir4)
    altorder2 = (ir1, ir4, ir3, ir2)
    altorder3 = (ir3, ir4, ir1, ir2)
    if (altorder1 in integral_identities) or\
       (altorder2 in integral_identities) or\
       (altorder3 in integral_identities):
        continue
    integral_identity_sector = []
    components = [component_labels[ir] for ir in [ir1, ir2, ir3, ir4]]
    comp_to_idx = [{c: idx for idx, c in enumerate(component_labels[ir])} for ir in [ir1, ir2, ir3, ir4]]
    for R in group.generators:
        R_id = {}
        for γ1, γ2, γ3, γ4 in product(*components):
            for γ1p, γ2p, γ3p, γ4p in product(*components):
                    val = sp.conjugate(ir_mats[ir1][R][comp_to_idx[0][γ1],comp_to_idx[0][γ1p]]) *\
                          sp.conjugate(ir_mats[ir2][R][comp_to_idx[1][γ2],comp_to_idx[1][γ2p]]) *\
                          ir_mats[ir3][R][comp_to_idx[2][γ3],comp_to_idx[2][γ3p]] *\
                          ir_mats[ir4][R][comp_to_idx[3][γ4],comp_to_idx[3][γ4p]]
                    if val== 0:
                        continue
                    key = (γ1, γ2, γ3, γ4)
                    if key == (sp.Symbol(r'{\phi}'),sp.Symbol(r'{\chi}'),sp.Symbol(r'{\chi}'),sp.Symbol(r'{\phi}')):
                        display(sp.Matrix([γ1p, γ2p, γ3p, γ4p]).T)
                        display(sp.Matrix([γ1, γ2, γ3, γ4]).T)
#                         print(sp.conjugate(ir_mats[ir1][R][comp_to_idx[0][γ1],comp_to_idx[0][γ1p]]),
#                           sp.conjugate(ir_mats[ir2][R][comp_to_idx[1][γ2],comp_to_idx[1][γ2p]]),
#                           ir_mats[ir3][R][comp_to_idx[2][γ3],comp_to_idx[2][γ3p]],
#                           ir_mats[ir4][R][comp_to_idx[3][γ4],comp_to_idx[3][γ4p]])
# #                         1/0
                    if key not in R_id.keys():
                        R_id[key] = []
                    R_id[key].append( Qet({(γ1p,γ2p,γ3p,γ4p): val}) )
#         R_id_total = []
#         for key in R_id.keys():
#             vee = (key, sum(R_id[key], Qet({}))) 
#             if key == (sp.Symbol(r'{\phi}'),sp.Symbol(r'{\chi}'),sp.Symbol(r'{\chi}'),sp.Symbol(r'{\phi}')):
#                         1/0
        R_id_total = [(key, sum(R_id[key], Qet({}))) for key in R_id.keys()]
        R_id_total = [q for q in R_id_total if len(q[1].dict)>0]
        integral_identity_sector.extend(R_id_total)
    integral_identities[(ir1,ir2,ir3,ir4)] = integral_identity_sector

(0/19) [00:00] 

Matrix([[{\phi}, {\xi}, {\xi}, {\phi}]])

Matrix([[{\phi}, {\chi}, {\chi}, {\phi}]])

Matrix([[{\chi}, {\phi}, {\phi}, {\chi}]])

Matrix([[{\phi}, {\chi}, {\chi}, {\phi}]])

In [345]:
# print(sp.conjugate(ir_mats[ir1][R][comp_to_idx[0][γ1p],comp_to_idx[0][γ1]]),
#       sp.conjugate(ir_mats[ir2][R][comp_to_idx[1][γ2p],comp_to_idx[1][γ2]]),
#                    ir_mats[ir3][R][comp_to_idx[2][γ3],comp_to_idx[2][γ3p]],
#                           ir_mats[ir4][R][comp_to_idx[3][γ4],comp_to_idx[3][γ4p]])

In [346]:
# val = (sp.conjugate(ir_mats[ir1][R][comp_to_idx[0][γ1p],comp_to_idx[0][γ1]]),
#       sp.conjugate(ir_mats[ir2][R][comp_to_idx[1][γ2p],comp_to_idx[1][γ2]]),
#       ir_mats[ir3][R][comp_to_idx[2][γ3],comp_to_idx[2][γ3p]],
#       ir_mats[ir4][R][comp_to_idx[3][γ4],comp_to_idx[3][γ4p]])
# val

In [ ]:
for q in great_identities[(sp.Symbol('T_2'),sp.Symbol('T_2'),sp.Symbol('T_2'),sp.Symbol('T_2'))]:
    display(q.as_braket())

In [347]:
for q in integral_identities[(sp.Symbol('T_2'),sp.Symbol('T_2'),sp.Symbol('T_2'),sp.Symbol('T_2'))]:
    lhs = sp.latex(Qet({q[0]:1}).as_braket())
    rhs = sp.latex(q[1].as_braket())
    if ' {\\phi}{\\chi}' in lhs and 'chi' in lhs:
        display(Math('%s = %s' % (lhs, rhs)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [348]:
# For solving the linear system it is convenient
# to have everything on one side of the equation.

if verbose:
    msg = "Creating set of identities ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

identities = {}
for ircombo in integral_identities:
    these_ids = []
    for v in integral_identities[ircombo]:
        lhs, rhs = v
        diff = Qet({lhs:1}) - rhs
        if len(diff.dict) > 0:
            these_ids.append(diff)
    identities[ircombo] = these_ids

# If an equation has only one key, then
# that immediately means that that braket is zero.

if verbose:
    msg = "Finding trivial zeros ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

# first determine which ones have to be zero

better_identities = {irc:[] for irc in identities}
all_zeros = {}
for ircombo in identities:
    zeros = []
    for identity in identities[ircombo]:
        if len(identity.dict) == 1:
            zeros.append((list(identity.dict.keys())[0]))
        else:
            better_identities[ircombo].append(identity)
    all_zeros[ircombo] = zeros

# use them to simplify things.
if verbose:
    msg = "Using them to simplify things ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

great_identities = {irc:[] for irc in identities}
for ircombo in better_identities:
    for identity in better_identities[ircombo]:
        new_qet = Qet({})
        for k,v in identity.dict.items():
            if k in all_zeros[ircombo]:
                continue
            else:
                new_qet += Qet({k: v})
        if len(new_qet.dict) == 0:
            continue
        great_identities[ircombo].append(new_qet)

# Inside of a four-symbol braket one may do three exchanges
# that must result in the same value. That if the wave
# functions are assumed to be real-valued.

if verbose:
    msg = "Creating reality identities ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

real_var_simplifiers = {irc:[] for irc in great_identities}

# this has to run over all the quadruples of irs
kprimes = set()
for ir1, ir2, ir3, ir4 in product(*([group.irrep_labels]*4)):
    real_var_simplifier = {}
    ircombo = (ir1, ir2, ir3, ir4)
    components = [component_labels[ir] for ir in ircombo]
    for γ1, γ2, γ3, γ4 in product(*components):
        k = (γ1, γ2, γ3, γ4)
        kalt1 = (γ3, γ2, γ1, γ4)
        kalt2 = (γ1, γ4, γ3, γ2)
        kalt3 = (γ3, γ4, γ1, γ2)
        if kalt1 in kprimes:
            real_var_simplifier[k] = kalt1
        elif kalt2 in kprimes:
            real_var_simplifier[k] = kalt2
        elif kalt3 in kprimes:
            real_var_simplifier[k] = kalt3
        else:
            real_var_simplifier[k] = k
            kprimes.add(k)
    real_var_simplifiers[ircombo] = real_var_simplifier

real_var_full_simplifier = {}
for ircombo in real_var_simplifiers:
    real_var_full_simplifier.update(real_var_simplifiers[ircombo])

# For each 4-tuple of irreps
# create a system of symbolic solutions
# and let sympy solve that.
# For each 4-tuple of irreps
# the end result is a dictionary
# whose keys represent the dependent
# brakets and whose values are the
# relation that those dependent values
# have with the independent brakets.
# As such, when these dictionaries are
# used on an expression, everything should
# then be given in terms of indepedent brakets.

if verbose:
    msg = "Solving for independent 4-symbol brakets ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

all_sols = {irc:[] for irc in great_identities}
for ircombo in great_identities:
    zeros = all_zeros[ircombo]
    problem_vars = list(set(sum([list(identity.dict.keys()) for identity in great_identities[ircombo]],[])))
    big_ma = [awe.vec_in_basis(problem_vars) for awe in great_identities[ircombo]] 
    big_mat = sp.Matrix(big_ma)
    big_mat = sp.re(big_mat)+sp.I*sp.im(big_mat)
    rref_mat, pivots = big_mat.rref()
    num_rows = rref_mat.rows
    num_cols = rref_mat.cols
    rref_ma_non_zero = [rref_mat[row,:] for row in range(num_rows) if (sum(np.array(rref_mat[row,:])[0] == 0) != num_cols)]
    rref_mat_non_zero = sp.Matrix(rref_ma_non_zero)
    varvec = sp.Matrix(list(map(composite_symbol,problem_vars)))
    eqns = rref_mat_non_zero*varvec
    ssol = sp.solve(list(eqns), dict=True)
    all_sols[ircombo] = ssol
    assert len(ssol) in [0,1]
    if len(ssol) == 0:
        sol_dict = {}
    else:
        sol_dict = ssol[0]
    zero_addendum = {k:{} for k in all_zeros[ircombo]}
    sol_dict = {fourtuplerecovery(k):{fourtuplerecovery(s):v.coeff(s) for s in v.free_symbols} for k,v in sol_dict.items()}
    sol_dict.update(zero_addendum)
    all_sols[ircombo] = sol_dict

# This final dictionary is unnecessary but
# simplifies calling the replacements onto
# a symbolic expression.

if verbose:
    msg = "Creating a dictionary with all the 4-symbol replacements ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

super_solution = {}
for ircombo in all_sols:
    super_solution.update(all_sols[ircombo])

def simplifier(qet): #yyy
    simp_ket = Qet({})
    for k,v in qet.dict.items():
        simp_ket += Qet({real_var_full_simplifier[k]:v})
    true_qet = Qet({})
    for k,v in simp_ket.dict.items():
        if k in super_solution:
            true_qet += v*Qet(super_solution[k])
        else:
            true_qet += Qet({k:v})
    return true_qet

def simplify_config_matrix(ir1ir2, S, ir3):
    config_matrix = config_matrices[ir1ir2][(S,ir3)]
    num_rows = len(config_matrix)
    simple_matrix = [[simplifier(config_matrix[row][col]) for col in range(num_rows)] for row in range(num_rows)]
    return simple_matrix

if verbose:
    msg = "Simplifying configuration matrices ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

simple_config_matrices = {k:{} for k in config_matrices}
for ir1ir2 in config_matrices.keys():
    for term_key in config_matrices[ir1ir2]:
        S, Γ3 = term_key
        simple_config_matrices[ir1ir2][term_key] = simplify_config_matrix(ir1ir2, S, Γ3)

In [350]:
twotuplerecovery = fourtuplerecovery
if verbose:
    msg = "Creating all 2-symbol identities ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

# this   integral_identities  dictionary  will  have  as  keys
# 2-tuples  of  irreps  and  its  values  will  be lists whose
# elements  are  2-tuples whose first elements are 2-tuples of
# irrep  components  and  whose values are qets whose keys are
# 2-tuples  of  irrep components and whose values are numeric.
# These 2-tuples represent a braket with the an invariant operator
# operator in between.

integral_identities_2 = {}
ir_mats = group.irrep_matrices
for ir1, ir2 in product(*([group.irrep_labels]*2)):
    # To simplify calculations this part
    # can only be done over quadruples in a standard
    # order.
    # Whatever is missed here is then brough back in
    # by the reality relations.
    altorder1 = (ir2, ir1, ir1, ir4)
    if (altorder1 in integral_identities_2):
        continue
    integral_identity_sector = []
    components = [component_labels[ir] for ir in [ir1, ir2]]
    comp_to_idx = [{c: idx for idx, c in enumerate(component_labels[ir])} for ir in [ir1, ir2]]
    for R in group.generators:
        R_id = {}
        for γ1, γ2 in product(*components):
            for γ1p, γ2p in product(*components):
                    val = sp.conjugate(ir_mats[ir1][R][comp_to_idx[0][γ1p],comp_to_idx[0][γ1]]) *\
                          ir_mats[ir2][R][comp_to_idx[1][γ2],comp_to_idx[1][γ2p]]
                    if val== 0:
                        continue
                    key = (γ1, γ2)
                    if key not in R_id.keys():
                        R_id[key] = []
                    R_id[key].append( Qet({(γ1p,γ2p): val}) )
        R_id_total = [(key, sum(R_id[key], Qet({}))) for key in R_id.keys()]
        R_id_total = [q for q in R_id_total if len(q[1].dict)>0]
        integral_identity_sector.extend(R_id_total)
    integral_identities_2[(ir1,ir2)] = integral_identity_sector

# For solving the linear system it is convenient
# to have everything on one side of the equation.

if verbose:
    msg = "Creating set of 2 symbol identities ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1
identities_2 = {}

for ircombo in integral_identities_2:
    these_ids = []
    for v in integral_identities_2[ircombo]:
        lhs, rhs = v
        diff = Qet({lhs:1}) - rhs
        if len(diff.dict) > 0:
            these_ids.append(diff)
    identities_2[ircombo] = these_ids

# If an equation has only one term, then
# that immediately means that that term is zero.

if verbose:
    msg = "Finding trivial zeros ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

# first determine which ones have to be zero
better_identities_2 = {irc:[] for irc in identities_2}
all_zeros_2 = {}
for ircombo in identities_2:
    zeros = []
    for identity in identities_2[ircombo]:
        if len(identity.dict) == 1:
            zeros.append((list(identity.dict.keys())[0]))
        else:
            better_identities_2[ircombo].append(identity)
    all_zeros_2[ircombo] = zeros

# use them to simplify things.

if verbose:
    msg = "Using them to simplify things ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

great_identities_2 = {irc:[] for irc in identities_2}
for ircombo in better_identities_2:
    for identity in better_identities_2[ircombo]:
        new_qet = Qet({})
        for k,v in identity.dict.items():
            if k in all_zeros_2[ircombo]:
                continue
            else:
                new_qet+= Qet({k: v})
        if len(new_qet.dict) == 0:
            continue
        great_identities_2[ircombo].append(new_qet)

# Inside of a two-symbol braket one may do three exchanges
# that must result in the same value. That if the wave
# functions are assumed to be real-valued.

if verbose:
    msg = "Creating reality identities ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1
real_var_simplifiers_2 = {irc:[] for irc in great_identities_2}
# this has to run over all the quadruples of irs
kprimes = set()
for ir1, ir2 in product(*([group.irrep_labels]*2)):
    real_var_simplifier = {}
    ircombo = (ir1, ir2)
    components = [component_labels[ir] for ir in ircombo]
    for γ1, γ2 in product(*components):
        k = (γ1, γ2)
        kalt = (γ2, γ1)
        # If for a given key I find that its
        # switched version has already been seen
        # Then that key has to be mapped to be
        # mapped to the key already present.
        if kalt in kprimes:
            real_var_simplifier[k] = kalt
        else:
            real_var_simplifier[k] = k
            kprimes.add(k)
    real_var_simplifiers_2[ircombo] = real_var_simplifier

# For each 2-tuple of irreps
# create a system of symbolic solutions
# and let sympy solve that.
# For each 2-tuple of irreps
# the end result is a dictionary
# whose keys represent the dependent
# brakets and whose values are the
# relation that those dependent values
# have with the independent brakets.
# As such, when these dictionaries are
# used on an expression, everything should
# then be given in terms of indepedent brakets.

if verbose:
    msg = "Solving for independent 2-symbol brakets ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1
all_sols_2 = {irc:[] for irc in great_identities_2}

for ircombo in great_identities_2:
    zeros = all_zeros_2[ircombo]
    problem_vars = list(set(sum([list(identity.dict.keys()) for identity in great_identities_2[ircombo]],[])))
    big_ma = [awe.vec_in_basis(problem_vars) for awe in great_identities_2[ircombo]] 
    big_mat = sp.Matrix(big_ma)
    big_mat = sp.re(big_mat)+sp.I*sp.im(big_mat)
    rref_mat, pivots = big_mat.rref()
    num_rows = rref_mat.rows
    num_cols = rref_mat.cols
    rref_ma_non_zero = [rref_mat[row,:] for row in range(num_rows) if (sum(np.array(rref_mat[row,:])[0] == 0) != num_cols)]
    rref_mat_non_zero = sp.Matrix(rref_ma_non_zero)
    varvec = sp.Matrix(list(map(composite_symbol,problem_vars)))
    eqns = rref_mat_non_zero*varvec
    ssol = sp.solve(list(eqns), dict=True)
    all_sols[ircombo] = ssol
    assert len(ssol) in [0,1]
    if len(ssol) == 0:
        sol_dict = {}
    else:
        sol_dict = ssol[0]
    zero_addendum = {k:{} for k in all_zeros_2[ircombo]}
    sol_dict = {twotuplerecovery(k):{twotuplerecovery(s):v.coeff(s) for s in v.free_symbols} for k,v in sol_dict.items()}
    sol_dict.update(zero_addendum)
    all_sols_2[ircombo] = sol_dict

# This final dictionary is unnecessary but
# simplifies calling the replacements onto
# a symbolic expression.
if verbose:
    msg = "Creating a dictionary with all the 2-symbol replacements ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

super_solution_2 = {}
for ircombo in all_sols_2:
    super_solution_2.update(all_sols_2[ircombo])

In [352]:
# for a given electron config
more_ids = {e_config:[] for e_config in simple_config_matrices}
counter = 0
for e_config in simple_config_matrices:
    for term in simple_config_matrices[e_config]:
        the_matrix = simple_config_matrices[e_config][term]
        num_rows = len(the_matrix)
        num_cols = len(the_matrix)
        the_state_keys = config_supplement[e_config][term]
        the_key_matrix = {}
        for row_idx in range(num_rows):
            Γ3_row = the_state_keys[row_idx][2]
            γ3_row = the_state_keys[row_idx][3]
            for col_idx in range(num_cols):
                Γ3_col = the_state_keys[col_idx][2]
                γ3_col = the_state_keys[col_idx][3]
                matrix_val = the_matrix[row_idx][col_idx]
                the_key_matrix[(γ3_row,γ3_col)] = matrix_val
        # now go over the keys of super_solution_2
        # and if one of those keys matches with a key in the_key_matrix
        # do something about it
        for k in super_solution_2:
            if k in the_key_matrix:
                v = super_solution_2[k]
                # this matrix element
                matrix_element = the_key_matrix[k]
                # must be identified with the sum
                # as given in v
                matrix_equiv = sum([kv*the_key_matrix[km] for km, kv in v.items()],Qet({}))
                identity = (matrix_element - matrix_equiv) #=0
                counter += 1
#                 if counter == 28:
#                     print('-----')
#                     display(matrix_element.as_braket())
#                     display(matrix_equiv.as_braket())
#                     display(identity.as_braket())
#                     1/0
                if len(identity.dict) != 0:
                    more_ids[e_config].append(identity)


In [353]:
# for q in great_identities[(sp.Symbol('T_2'),sp.Symbol('T_2'),sp.Symbol('T_2'),sp.Symbol('T_2'))]:
#     display(q.as_braket())

In [354]:
the_state_keys

[(A_2, T_1, T_2, {\chi}, 0, 0),
 (A_2, T_1, T_2, {\xi}, 0, 0),
 (A_2, T_1, T_2, {\phi}, 0, 0)]

In [355]:
for k, v in super_solution.items(): #yyy
    if len(v) != 0:
        if sp.Symbol(r'{\phi}') in k and\
            sp.Symbol(r'{\chi}') in k and\
            sp.Symbol(r'{\alpha}') not in k and\
            sp.Symbol(r'{\eta}') not in k and\
            sp.Symbol(r'{\beta}') not in k and\
            sp.Symbol(r'{\gamma}') not in k and\
            sp.Symbol(r'{\zeta}') not in k and\
            sp.Symbol(r'{\mu}') not in k and\
            sp.Symbol(r'{\nu}') not in k:
            lhs = sp.latex(Qet({k:1}).as_braket())
            rhs = sp.latex(Qet(v).as_braket())
            out = '$%s = %s$' % (lhs, rhs)
            display(Math(out))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [356]:
sp.Matrix([[l.as_braket() for l in row] for row in the_matrix])

Matrix([
[<{\beta}{\beta}|{\nu}{\nu}> + <{\beta}{\nu}|{\beta}{\nu}>,                                                         0,                                                         0],
[                                                        0, <{\beta}{\beta}|{\nu}{\nu}> + <{\beta}{\nu}|{\beta}{\nu}>,                                                         0],
[                                                        0,                                                         0, <{\beta}{\beta}|{\nu}{\nu}> + <{\beta}{\nu}|{\beta}{\nu}>]])

In [357]:
for k,v in super_solution_2.items():
    if (len(v) > 0):
        lhs = sp.latex(Qet(v).as_braket())
        rhs = sp.latex(Qet({k:1}).as_braket())
        display(Math('%s=%s' % (lhs, rhs)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [358]:
def simplifier(qet): #yyy
    simp_ket = Qet({})
    for k,v in qet.dict.items():
        simp_ket += Qet({real_var_full_simplifier[k]:v})
    display(simp_ket.as_braket())
    true_qet = Qet({})
    for k,v in simp_ket.dict.items():
        if k in super_solution:
            print('-')
            display(Qet({k:1}).as_braket())
            display(Qet(super_solution[k]).as_braket())
            true_qet += v*Qet(super_solution[k])
        else:
            print('+')
            true_qet += Qet({k:v})
    return true_qet

In [359]:
simplifier(config_matrices[sp.Symbol('t_{2}')**2][(1,sp.Symbol('T_1'))][2][2]).as_braket()

<{\phi}{\chi}|{\phi}{\chi}> - <{\phi}{\phi}|{\chi}{\chi}>

-


<{\phi}{\chi}|{\phi}{\chi}>

<{\xi}{\phi}|{\xi}{\phi}>

-


<{\phi}{\phi}|{\chi}{\chi}>

<{\xi}{\xi}|{\phi}{\phi}>

<{\xi}{\phi}|{\xi}{\phi}> - <{\xi}{\xi}|{\phi}{\phi}>

In [360]:
display(sp.Matrix([[k.as_braket() for k in row] for row in config_matrices[sp.Symbol('t_{2}')**2][(1,sp.Symbol('T_1'))]])[2,2])
display(sp.Matrix([[k.as_braket() for k in row] for row in config_matrices[sp.Symbol('t_{2}')**2][(1,sp.Symbol('T_1'))]]))

-<{\phi}{\chi}|{\chi}{\phi}> + <{\phi}{\chi}|{\phi}{\chi}>

Matrix([
[  -<{\xi}{\phi}|{\phi}{\xi}> + <{\xi}{\phi}|{\xi}{\phi}>,   <{\xi}{\phi}|{\chi}{\xi}> - <{\xi}{\phi}|{\xi}{\chi}>,   -<{\xi}{\phi}|{\chi}{\phi}> + <{\xi}{\phi}|{\phi}{\chi}>,                                                                                                                         0,                                                                                                                          0,                                                                                                                             0,                                                        0,                                                       0,                                                          0],
[   <{\xi}{\chi}|{\phi}{\xi}> - <{\xi}{\chi}|{\xi}{\phi}>,  -<{\xi}{\chi}|{\chi}{\xi}> + <{\xi}{\chi}|{\xi}{\chi}>,    <{\xi}{\chi}|{\chi}{\phi}> - <{\xi}{\chi}|{\phi}{\chi}>,                                                                                          

In [ ]:
!beep

In [155]:
for q in sum(more_ids.values(),[]):
    display(q.as_braket())

-<{\zeta}{\gamma}|{\zeta}{\gamma}> - 2*<{\zeta}{\zeta}|{\gamma}{\gamma}> + <{\zeta}{\zeta}|{\zeta}{\zeta}>

-2*<{\nu}{\nu}|{\mu}{\mu}>

2*<{\nu}{\nu}|{\mu}{\mu}>

2*<{\nu}{\nu}|{\mu}{\mu}>

-2*<{\xi}{\xi}|{\phi}{\phi}>

...

2*<{\alpha}{\alpha}|{\xi}{\xi}>

-2*<{\beta}{\beta}|{\nu}{\nu}>

-2*<{\beta}{\beta}|{\nu}{\nu}>

2*<{\beta}{\beta}|{\nu}{\nu}>

2*<{\beta}{\beta}|{\nu}{\nu}>

In [361]:
if verbose:
    msg = "Simplifying numeric values of qets ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

more_ids = {e_config:list(filter(lambda x: len(x.dict) > 0,list(map(simplify_qet,more_ids[e_config])))) for e_config in more_ids}

if verbose:
    msg = "Solving for dependent vars in terms of independent ones ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

all_sols_2_4 = {irc:[] for irc in more_ids}

for e_config in more_ids:
    problem_vars = list(set(sum([list(identity.dict.keys()) for identity in more_ids[e_config]],[])))
    big_ma = [awe.vec_in_basis(problem_vars) for awe in more_ids[e_config]] 
    big_mat = sp.Matrix(big_ma)
    big_mat = sp.re(big_mat)+sp.I*sp.im(big_mat)
    rref_mat, pivots = big_mat.rref()
    num_rows = rref_mat.rows
    num_cols = rref_mat.cols
    rref_ma_non_zero = [rref_mat[row,:] for row in range(num_rows) if (sum(np.array(rref_mat[row,:])[0] == 0) != num_cols)]
    rref_mat_non_zero = sp.Matrix(rref_ma_non_zero)
    varvec = sp.Matrix(list(map(composite_symbol, problem_vars)))
    eqns = rref_mat_non_zero*varvec
    ssol = sp.solve(list(eqns), dict=True)
    all_sols_2_4[e_config] = ssol
    assert len(ssol) in [0,1]
    if len(ssol) == 0:
        sol_dict = {}
    else:
        sol_dict = ssol[0]
    sol_dict = {twotuplerecovery(k):{twotuplerecovery(s):v.coeff(s) for s in v.free_symbols} for k,v in sol_dict.items()}
    all_sols_2_4[e_config] = sol_dict

# flatten into a single dictionary of replacements
fab_solution_2_4 = {}
for e_config in all_sols_2_4:
    fab_solution_2_4.update(all_sols_2_4[e_config])
if verbose:
    msg = "There are %d less independent variables ..."
    pbar.set_description("There are %d less independent variables ..." % len(fab_solution_2_4))
    pbar.refresh()
    pbar.n += 1

def simplifier_f(qet):
    true_qet = Qet({})
    for k,v in qet.dict.items():
        if k in fab_solution_2_4:
            true_qet += v*Qet(fab_solution_2_4[k])
        else:
            true_qet += Qet({k:v})
    return true_qet

def simplify_config_matrix_f(ir1ir2, S, ir3):
    config_matrix = simple_config_matrices[ir1ir2][(S,ir3)]
    num_rows = len(config_matrix)
    simple_matrix = [[simplifier_f(config_matrix[row][col]) for col in range(num_rows)] for row in range(num_rows)]
    #simple_matrix = sp.Matrix(simple_matrix)
    return simple_matrix

if verbose:
    msg = "Making final simplifications ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

final_config_matrices = {k:{} for k in config_matrices}
for ir1ir2 in config_matrices.keys():
    for term_key in config_matrices[ir1ir2]:
        S, Γ3 = term_key
        final_config_matrices[ir1ir2][term_key] = simplify_config_matrix_f(ir1ir2, S, Γ3)

if verbose:
    msg = "Finished"
    pbar.set_description(msg)
    pbar.refresh()
    
O = final_config_matrices

In [366]:
O[sp.Symbol('t_{2}')**2][(1,sp.Symbol('T_1'))]

[[Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({})],
 [Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({})],
 [Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({})],
 [Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({})],
 [Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({})],
 [Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({})],
 [Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({})],
 [Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({})],
 [Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({})]]

#### Detailed Comparison to O (invalid (was using old version))

In [141]:
def simplify_qet(qet):
    new_dict = {k:sp.simplify(v) for k,v in qet.dict.items()}
    return Qet(new_dict)

# def term_energies(group_label, verbose=True):

group_label = 'O'
verbose = True
stages = 19 # this is how many distinct parts this algorithm has
if verbose:
    pbar = tqdm(range(stages), bar_format='({n:d}/{total:d}) [{elapsed}] {desc}')
terms = twoe_terms[group_label]
group = CPGs.get_group_by_label(group_label)
component_labels = {k:list(v.values()) for k,v in new_labels[group_label].items()}
# for a given set of terms
# split all the enclosed states
# in groups of electron configurations
configs = {}
config_supplement = {}
for term_key, term in terms.items():
    for state_key, state in zip(term.state_keys, term.states):
        (Γ1, Γ2, Γ3, γ3, S, mSz) = state_key
        α = sp.Symbol(sp.latex(Γ1).lower())*sp.Symbol(sp.latex(Γ2).lower())
        if α not in configs.keys():
            configs[α] = {}
            config_supplement[α] = {}
        if (S,Γ3) not in configs[α].keys():
            configs[α][(S,Γ3)] = []
            config_supplement[α][(S,Γ3)] = []
        configs[α][(S,Γ3)].append(state)
        config_supplement[α][(S,Γ3)].append(state_key)

def overlinesqueegee(s):
    '''
    Going back from the overline shorthand for spin down,
    acting on a single set of quantum numbers.
    '''
    if 'overline' in str(s):
        spin = -sp.S(1)/2
        comp = sp.Symbol(str(s).replace('\\overline{','')[:-1])
    else:
        spin = sp.S(1)/2
        comp = s
    return (comp, spin)

def spin_restoration(qet):
    '''
    Going back from the overline shorthand for spin down,
    acting on a qet.
    '''
    new_dict = {}
    for k,v in qet.dict.items():
        k = (*overlinesqueegee(k[0]),*overlinesqueegee(k[1]))
        new_dict[k] = v
    return Qet(new_dict)

# within each configuration
config_matrices = {}
for config in configs.keys():
    # within each term
    term_matrices = {}
    for term_key, states in configs[config].items():
        # find the brackets between all possible pairs of states
        ham_matrix = []
        for state0 in states:
            ham_row = []
            state0 = spin_restoration(state0)
            for state1 in states:
                # recover the spin for the determinantal states
                pstate = state1
                state1 = spin_restoration(state1)
                # determine the det braket between these two states (an operator in between is assumed and omitted)
                detbraket = state0*state1
                # simplify this braket of detstates into brakets between regular states
                regbraket = Qet({})
                for k,v in detbraket.dict.items():
                    γ1, m1, γ2, m2, γ1p, m1p, γ2p, m2p = k
                    # k0 = (γ1, m1, γ2, m2, γ1p, m1p, γ2p, m2p)
                    # k1 = (γ1, m1, γ2, m2, γ2p, m2p, γ1p, m1p)
                    k0 = (γ1, γ2, γ1p, γ2p)
                    k1 = (γ1, γ2, γ2p, γ1p)
                    # enforce orthogonality wrt. spin
                    if (m1 == m1p) and (m2 == m2p):
                        qplus = Qet({k0:v}) 
                    else:
                        qplus = Qet({})
                    if (m1 == m2p) and (m2 == m1p):
                        qminus = Qet({k1:-v}) 
                    else:
                        qminus = Qet({})
                    regbraket += (qplus+qminus)
                ham_row.append(regbraket)
            ham_matrix.append(ham_row)
        term_matrices[term_key] = ham_matrix
    config_matrices[config] = term_matrices

def composite_symbol(x):
    return sp.Symbol('(%s)'%(','.join(list(map(sp.latex,x)))))
def fourtuplerecovery(ft):
    '''the inverse of composite_symbol'''
    return tuple(map(sp.Symbol,sp.latex(ft)[1:-1].split(',')))

if verbose:
    msg = "Creating all 4-symbol identities ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

# this   integral_identities  dictionary  will  have  as  keys
# 4-tuples  of  irreps  and  its  values  will  be lists whose
# elements  are  2-tuples whose first elements are 4-tuples of
# irrep  components  and  whose values are qets whose keys are
# 4-tuples  of  irrep components and whose values are numeric.
# These 4-tuples represent a braket with the Coulomb repulsion
# operator in between.

integral_identities = {}
ir_mats = group.irrep_matrices
for ir1, ir2, ir3, ir4 in product(*([group.irrep_labels]*4)):
    # To simplify calculations this part
    # may only be done over quadruples in a standard
    # order.
    # Whatever is missed here is then brough back in
    # by the reality relations.
    altorder1 = (ir3, ir2, ir1, ir4)
    altorder2 = (ir1, ir4, ir3, ir2)
    altorder3 = (ir3, ir4, ir1, ir2)
    if (altorder1 in integral_identities) or\
       (altorder2 in integral_identities) or\
       (altorder3 in integral_identities):
        continue
    integral_identity_sector = []
    components = [component_labels[ir] for ir in [ir1, ir2, ir3, ir4]]
    comp_to_idx = [{c: idx for idx, c in enumerate(component_labels[ir])} for ir in [ir1, ir2, ir3, ir4]]
    for R in group.generators + [sp.Symbol('{Ee}')]:
        R_id = {}
        for γ1, γ2, γ3, γ4 in product(*components):
            for γ1p, γ2p, γ3p, γ4p in product(*components):
                    val = sp.conjugate(ir_mats[ir1][R][comp_to_idx[0][γ1p],comp_to_idx[0][γ1]]) *\
                          sp.conjugate(ir_mats[ir2][R][comp_to_idx[1][γ2p],comp_to_idx[1][γ2]]) *\
                          ir_mats[ir3][R][comp_to_idx[2][γ3],comp_to_idx[2][γ3p]] *\
                          ir_mats[ir4][R][comp_to_idx[3][γ4],comp_to_idx[3][γ4p]]
                    if val== 0:
                        continue
                    key = (γ1, γ2, γ3, γ4)
                    if key not in R_id.keys():
                        R_id[key] = []
                    R_id[key].append( Qet({(γ1p,γ2p,γ3p,γ4p): val}) )
        R_id_total = [(key, sum(R_id[key], Qet({}))) for key in R_id.keys()]
        R_id_total = [q for q in R_id_total if len(q[1].dict)>0]
        integral_identity_sector.extend(R_id_total)
    integral_identities[(ir1,ir2,ir3,ir4)] = integral_identity_sector

# For solving the linear system it is convenient
# to have everything on one side of the equation.
if verbose:
    msg = "Creating set of identities ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

identities = {}
for ircombo in integral_identities:
    these_ids = []
    for v in integral_identities[ircombo]:
        lhs, rhs = v
        diff = Qet({lhs:1}) - rhs
        if len(diff.dict) > 0:
            these_ids.append(diff)
    identities[ircombo] = these_ids

# If an equation has only one key, then
# that immediately means that that braket is zero.
if verbose:
    msg = "Finding trivial zeros ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

# first determine which ones have to be zero
better_identities = {irc:[] for irc in identities}
all_zeros = {}
for ircombo in identities:
    zeros = []
    for identity in identities[ircombo]:
        if len(identity.dict) == 1:
            zeros.append((list(identity.dict.keys())[0]))
        else:
            better_identities[ircombo].append(identity)
    all_zeros[ircombo] = zeros

# use them to simplify things.
if verbose:
    msg = "Using them to simplify things ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

great_identities = {irc:[] for irc in identities}
for ircombo in better_identities:
    for identity in better_identities[ircombo]:
        new_qet = Qet({})
        for k,v in identity.dict.items():
            if k in all_zeros[ircombo]:
                continue
            else:
                new_qet+= Qet({k: v})
        if len(new_qet.dict) == 0:
            continue
        great_identities[ircombo].append(new_qet)

# Inside of a four-symbol braket one may do three exchanges
# that must result in the same value. That if the wave
# functions are assumed to be real-valued.

if verbose:
    msg = "Creating reality identities ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

real_var_simplifiers = {irc:[] for irc in great_identities}
kprimes = set()
# this has to run over all the quadruples of irs
for ir1, ir2, ir3, ir4 in product(*([group.irrep_labels]*4)):
    real_var_simplifier = {}
    ircombo = (ir1, ir2, ir3, ir4)
    components = [component_labels[ir] for ir in ircombo]
    for γ1, γ2, γ3, γ4 in product(*components):
        k = (γ1, γ2, γ3, γ4)
        kalt1 = (γ3, γ2, γ1, γ4)
        kalt2 = (γ1, γ4, γ3, γ2)
        kalt3 = (γ3, γ4, γ1, γ2)
        if kalt1 in kprimes:
            real_var_simplifier[k] = kalt1
        elif kalt2 in kprimes:
            real_var_simplifier[k] = kalt2
        elif kalt3 in kprimes:
            real_var_simplifier[k] = kalt3
        else:
            real_var_simplifier[k] = k
            kprimes.add(k)
    real_var_simplifiers[ircombo] = real_var_simplifier

real_var_full_simplifier = {}
for ircombo in real_var_simplifiers:
    real_var_full_simplifier.update(real_var_simplifiers[ircombo])

# For each 4-tuple of irreps
# create a system of symbolic solutions
# and let sympy solve that.
# For each 4-tuple of irreps
# the end result is a dictionary
# whose keys represent the dependent
# brakets and whose values are the
# relation that those dependent values
# have with the independent brakets.
# As such, when these dictionaries are
# used on an expression, everything should
# then be given in terms of indepedent brakets.

if verbose:
    msg = "Solving for independent 4-symbol brakets ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

all_sols = {irc:[] for irc in great_identities}
for ircombo in great_identities:
    zeros = all_zeros[ircombo]
    problem_vars = list(set(sum([list(identity.dict.keys()) for identity in great_identities[ircombo]],[])))
    big_ma = [awe.vec_in_basis(problem_vars) for awe in great_identities[ircombo]] 
    big_mat = sp.Matrix(big_ma)
    big_mat = sp.re(big_mat)+sp.I*sp.im(big_mat)
    rref_mat, pivots = big_mat.rref()
    num_rows = rref_mat.rows
    num_cols = rref_mat.cols
    rref_ma_non_zero = [rref_mat[row,:] for row in range(num_rows) if (sum(np.array(rref_mat[row,:])[0] == 0) != num_cols)]
    rref_mat_non_zero = sp.Matrix(rref_ma_non_zero)
    varvec = sp.Matrix(list(map(composite_symbol,problem_vars)))
    eqns = rref_mat_non_zero*varvec
    ssol = sp.solve(list(eqns), dict=True)
    all_sols[ircombo] = ssol
    assert len(ssol) in [0,1]
    if len(ssol) == 0:
        sol_dict = {}
    else:
        sol_dict = ssol[0]
    zero_addendum = {k:{} for k in all_zeros[ircombo]}
    sol_dict = {fourtuplerecovery(k):{fourtuplerecovery(s):v.coeff(s) for s in v.free_symbols} for k,v in sol_dict.items()}
    sol_dict.update(zero_addendum)
    all_sols[ircombo] = sol_dict

# This final dictionary is unnecessary but
# simplifies calling the replacements onto
# a symbolic expression.
if verbose:
    msg = "Creating a dictionary with all the 4-symbol replacements ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

super_solution = {}
for ircombo in all_sols:
    super_solution.update(all_sols[ircombo])

def simplifier(qet):
    simp_ket = Qet({})
    for k,v in qet.dict.items():
        simp_ket += Qet({real_var_full_simplifier[k]:v})
    true_qet = Qet({})
    for k,v in simp_ket.dict.items():
        if k in super_solution:
            true_qet += v*Qet(super_solution[k])
        else:
            true_qet += Qet({k:v})
    return true_qet

def simplify_config_matrix(ir1ir2, S, ir3):
    config_matrix = config_matrices[ir1ir2][(S,ir3)]
    num_rows = len(config_matrix)
    simple_matrix = [[simplifier(config_matrix[row][col]) for col in range(num_rows)] for row in range(num_rows)]
    return simple_matrix

if verbose:
    msg = "Simplifying configuration matrices ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

simple_config_matrices = {k:{} for k in config_matrices}
for ir1ir2 in config_matrices.keys():
    for term_key in config_matrices[ir1ir2]:
        S, Γ3 = term_key
        simple_config_matrices[ir1ir2][term_key] = simplify_config_matrix(ir1ir2, S, Γ3)

twotuplerecovery = fourtuplerecovery
if verbose:
    msg = "Creating all 2-symbol identities ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

# this   integral_identities  dictionary  will  have  as  keys
# 2-tuples  of  irreps  and  its  values  will  be lists whose
# elements  are  2-tuples whose first elements are 2-tuples of
# irrep  components  and  whose values are qets whose keys are
# 2-tuples  of  irrep components and whose values are numeric.
# These 2-tuples represent a braket with the an invariant operator
# operator in between.

integral_identities_2 = {}
ir_mats = group.irrep_matrices
for ir1, ir2 in product(*([group.irrep_labels]*2)):
    # To simplify calculations this part
    # can only be done over quadruples in a standard
    # order.
    # Whatever is missed here is then brough back in
    # by the reality relations.
    altorder1 = (ir2, ir1, ir1, ir4)
    if (altorder1 in integral_identities_2):
        continue
    integral_identity_sector = []
    components = [component_labels[ir] for ir in [ir1, ir2]]
    comp_to_idx = [{c: idx for idx, c in enumerate(component_labels[ir])} for ir in [ir1, ir2]]
    for R in group.generators + [sp.Symbol('{Ee}')]:
        R_id = {}
        for γ1, γ2 in product(*components):
            for γ1p, γ2p in product(*components):
                    val = sp.conjugate(ir_mats[ir1][R][comp_to_idx[0][γ1p],comp_to_idx[0][γ1]]) *\
                          ir_mats[ir2][R][comp_to_idx[1][γ2],comp_to_idx[1][γ2p]]
                    if val== 0:
                        continue
                    key = (γ1, γ2)
                    if key not in R_id.keys():
                        R_id[key] = []
                    R_id[key].append( Qet({(γ1p,γ2p): val}) )
        R_id_total = [(key, sum(R_id[key], Qet({}))) for key in R_id.keys()]
        R_id_total = [q for q in R_id_total if len(q[1].dict)>0]
        integral_identity_sector.extend(R_id_total)
    integral_identities_2[(ir1,ir2)] = integral_identity_sector

# For solving the linear system it is convenient
# to have everything on one side of the equation.
if verbose:
    msg = "Creating set of 2 symbol identities ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1
identities_2 = {}

for ircombo in integral_identities_2:
    these_ids = []
    for v in integral_identities_2[ircombo]:
        lhs, rhs = v
        diff = Qet({lhs:1}) - rhs
        if len(diff.dict) > 0:
            these_ids.append(diff)
    identities_2[ircombo] = these_ids

# If an equation has only one term, then
# that immediately means that that term is zero.
if verbose:
    msg = "Finding trivial zeros ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

# first determine which ones have to be zero
better_identities_2 = {irc:[] for irc in identities_2}
all_zeros_2 = {}
for ircombo in identities_2:
    zeros = []
    for identity in identities_2[ircombo]:
        if len(identity.dict) == 1:
            zeros.append((list(identity.dict.keys())[0]))
        else:
            better_identities_2[ircombo].append(identity)
    all_zeros_2[ircombo] = zeros

# use them to simplify things.
if verbose:
    msg = "Using them to simplify things ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

great_identities_2 = {irc:[] for irc in identities_2}
for ircombo in better_identities_2:
    for identity in better_identities_2[ircombo]:
        new_qet = Qet({})
        for k,v in identity.dict.items():
            if k in all_zeros_2[ircombo]:
                continue
            else:
                new_qet+= Qet({k: v})
        if len(new_qet.dict) == 0:
            continue
        great_identities_2[ircombo].append(new_qet)

# Inside of a two-symbol braket one may do three exchanges
# that must result in the same value. That if the wave
# functions are assumed to be real-valued.
if verbose:
    msg = "Creating reality identities ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1
real_var_simplifiers_2 = {irc:[] for irc in great_identities_2}
# this has to run over all the quadruples of irs
kprimes = set()
for ir1, ir2 in product(*([group.irrep_labels]*2)):
    real_var_simplifier = {}
    ircombo = (ir1, ir2)
    components = [component_labels[ir] for ir in ircombo]
    for γ1, γ2 in product(*components):
        k = (γ1, γ2)
        kalt = (γ2, γ1)
        # If for a given key I find that its
        # switched version has already been seen
        # Then that key has to be mapped to be
        # mapped to the key already present.
        if kalt in kprimes:
            real_var_simplifier[k] = kalt
        else:
            real_var_simplifier[k] = k
            kprimes.add(k)
    real_var_simplifiers_2[ircombo] = real_var_simplifier

# For each 2-tuple of irreps
# create a system of symbolic solutions
# and let sympy solve that.
# For each 2-tuple of irreps
# the end result is a dictionary
# whose keys represent the dependent
# brakets and whose values are the
# relation that those dependent values
# have with the independent brakets.
# As such, when these dictionaries are
# used on an expression, everything should
# then be given in terms of indepedent brakets.
if verbose:
    msg = "Solving for independent 2-symbol brakets ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1
all_sols_2 = {irc:[] for irc in great_identities_2}

for ircombo in great_identities_2:
    zeros = all_zeros_2[ircombo]
    problem_vars = list(set(sum([list(identity.dict.keys()) for identity in great_identities_2[ircombo]],[])))
    big_ma = [awe.vec_in_basis(problem_vars) for awe in great_identities_2[ircombo]] 
    big_mat = sp.Matrix(big_ma)
    big_mat = sp.re(big_mat)+sp.I*sp.im(big_mat)
    rref_mat, pivots = big_mat.rref()
    num_rows = rref_mat.rows
    num_cols = rref_mat.cols
    rref_ma_non_zero = [rref_mat[row,:] for row in range(num_rows) if (sum(np.array(rref_mat[row,:])[0] == 0) != num_cols)]
    rref_mat_non_zero = sp.Matrix(rref_ma_non_zero)
    varvec = sp.Matrix(list(map(composite_symbol,problem_vars)))
    eqns = rref_mat_non_zero*varvec
    ssol = sp.solve(list(eqns), dict=True)
    all_sols[ircombo] = ssol
    assert len(ssol) in [0,1]
    if len(ssol) == 0:
        sol_dict = {}
    else:
        sol_dict = ssol[0]
    zero_addendum = {k:{} for k in all_zeros_2[ircombo]}
    sol_dict = {twotuplerecovery(k):{twotuplerecovery(s):v.coeff(s) for s in v.free_symbols} for k,v in sol_dict.items()}
    sol_dict.update(zero_addendum)
    all_sols_2[ircombo] = sol_dict

# This final dictionary is unnecessary but
# simplifies calling the replacements onto
# a symbolic expression.
if verbose:
    msg = "Creating a dictionary with all the 2-symbol replacements ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

super_solution_2 = {}
for ircombo in all_sols_2:
    super_solution_2.update(all_sols_2[ircombo])

(0/19) [00:00] 

In [177]:
lee = [[1,2],[3,4]]

In [176]:
for k,v in super_solution_2.items():
    if (len(v) > 0):
        lhs = sp.latex(Qet(v).as_braket())
        rhs = sp.latex(Qet({k:1}).as_braket())
        display(Math('%s=%s' % (lhs, rhs)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [156]:
# for a given electron config
more_ids = {e_config:[] for e_config in simple_config_matrices}
counter = 0
for e_config in simple_config_matrices:
    for term in simple_config_matrices[e_config]:
        the_matrix = simple_config_matrices[e_config][term]
        num_rows = len(the_matrix)
        num_cols = num_rows
        the_state_keys = config_supplement[e_config][term]
        the_key_matrix = {}
        for row_idx in range(num_rows):
            Γ3_row = the_state_keys[row_idx][2]
            γ3_row = the_state_keys[row_idx][3]
            for col_idx in range(num_cols):
                Γ3_col = the_state_keys[col_idx][2]
                γ3_col = the_state_keys[col_idx][3]
                matrix_val = the_matrix[row_idx][col_idx]
                the_key_matrix[(γ3_row,γ3_col)] = matrix_val
        # now go over the keys of super_solution_2
        # and if one of those keys matches with a key in the_key_matrix
        # do something about it
        for k in super_solution_2:
            if k in the_key_matrix:
                v = super_solution_2[k]
                # this matrix element
                matrix_element = the_key_matrix[k]
                # must be identified with the sum
                # as given in v
                matrix_equiv = sum([kv*the_key_matrix[km] for km, kv in v.items()], Qet({}))
                identity = (matrix_element - matrix_equiv) #=0
                if counter == 27:
                    print('-----')
                    display(matrix_element.as_braket())
                    display(matrix_equiv.as_braket())
                    display(identity.as_braket())
                    1/0
                counter += 1
#                 for v in identity.dict.keys():
#                     if v == (sp.Symbol(r'\phi'),sp.Symbol(r'\phi'),sp.Symbol(r'\phi'),sp.Symbol(r'\phi'))
                if len(identity.dict) != 0:
                    more_ids[e_config].append(identity)

-----


-<{\xi}{\phi}|{\xi}{\phi}> - <{\xi}{\xi}|{\phi}{\phi}>

-<{\xi}{\phi}|{\xi}{\phi}> + <{\xi}{\xi}|{\phi}{\phi}>

-2*<{\xi}{\xi}|{\phi}{\phi}>

ZeroDivisionError: division by zero

In [159]:
Γ3_col

T_1

In [160]:
Γ3_row

T_1

In [161]:
k

({\mu}, {\mu})

In [158]:
v

{({\nu}, {\nu}): -1}

In [154]:
sp.Matrix([[l.as_braket() for l in row] for row in the_matrix])

Matrix([
[<{\beta}{\beta}|{\nu}{\nu}> + <{\beta}{\nu}|{\beta}{\nu}>,                                                         0,                                                         0],
[                                                        0, <{\beta}{\beta}|{\nu}{\nu}> - <{\beta}{\nu}|{\beta}{\nu}>,                                                         0],
[                                                        0,                                                         0, <{\beta}{\beta}|{\nu}{\nu}> + <{\beta}{\nu}|{\beta}{\nu}>]])

In [155]:
for q in sum(more_ids.values(),[]):
    display(q.as_braket())

-<{\zeta}{\gamma}|{\zeta}{\gamma}> - 2*<{\zeta}{\zeta}|{\gamma}{\gamma}> + <{\zeta}{\zeta}|{\zeta}{\zeta}>

-2*<{\nu}{\nu}|{\mu}{\mu}>

2*<{\nu}{\nu}|{\mu}{\mu}>

2*<{\nu}{\nu}|{\mu}{\mu}>

-2*<{\xi}{\xi}|{\phi}{\phi}>

...

2*<{\alpha}{\alpha}|{\xi}{\xi}>

-2*<{\beta}{\beta}|{\nu}{\nu}>

-2*<{\beta}{\beta}|{\nu}{\nu}>

2*<{\beta}{\beta}|{\nu}{\nu}>

2*<{\beta}{\beta}|{\nu}{\nu}>

In [145]:
if verbose:
    msg = "Simplifying numeric values of qets ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

more_ids = {e_config:list(filter(lambda x: len(x.dict) > 0,list(map(simplify_qet,more_ids[e_config])))) for e_config in more_ids}

if verbose:
    msg = "Solving for dependent vars in terms of independent ones ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

all_sols_2_4 = {irc:[] for irc in more_ids}

for e_config in more_ids:
    problem_vars = list(set(sum([list(identity.dict.keys()) for identity in more_ids[e_config]],[])))
    big_ma = [awe.vec_in_basis(problem_vars) for awe in more_ids[e_config]] 
    big_mat = sp.Matrix(big_ma)
    big_mat = sp.re(big_mat)+sp.I*sp.im(big_mat)
    rref_mat, pivots = big_mat.rref()
    num_rows = rref_mat.rows
    num_cols = rref_mat.cols
    rref_ma_non_zero = [rref_mat[row,:] for row in range(num_rows) if (sum(np.array(rref_mat[row,:])[0] == 0) != num_cols)]
    rref_mat_non_zero = sp.Matrix(rref_ma_non_zero)
    varvec = sp.Matrix(list(map(composite_symbol, problem_vars)))
    eqns = rref_mat_non_zero*varvec
    ssol = sp.solve(list(eqns), dict=True)
    all_sols_2_4[e_config] = ssol
    assert len(ssol) in [0,1]
    if len(ssol) == 0:
        sol_dict = {}
    else:
        sol_dict = ssol[0]
    sol_dict = {twotuplerecovery(k):{twotuplerecovery(s):v.coeff(s) for s in v.free_symbols} for k,v in sol_dict.items()}
    all_sols_2_4[e_config] = sol_dict

# flatten into a single dictionary of replacements
fab_solution_2_4 = {}
for e_config in all_sols_2_4:
    fab_solution_2_4.update(all_sols_2_4[e_config])
if verbose:
    msg = "There are %d less independent variables ..."
    pbar.set_description("There are %d less independent variables ..." % len(fab_solution_2_4))
    pbar.refresh()
    pbar.n += 1

def simplifier_f(qet):
    true_qet = Qet({})
    for k,v in qet.dict.items():
        if k in fab_solution_2_4:
            true_qet += v*Qet(fab_solution_2_4[k])
        else:
            true_qet += Qet({k:v})
    return true_qet

def simplify_config_matrix_f(ir1ir2, S, ir3):
    config_matrix = simple_config_matrices[ir1ir2][(S,ir3)]
    num_rows = len(config_matrix)
    simple_matrix = [[simplifier_f(config_matrix[row][col]) for col in range(num_rows)] for row in range(num_rows)]
    #simple_matrix = sp.Matrix(simple_matrix)
    return simple_matrix

if verbose:
    msg = "Making final simplifications ..."
    pbar.set_description(msg)
    pbar.refresh()
    pbar.n += 1

final_config_matrices = {k:{} for k in config_matrices}
for ir1ir2 in config_matrices.keys():
    for term_key in config_matrices[ir1ir2]:
        S, Γ3 = term_key
        final_config_matrices[ir1ir2][term_key] = simplify_config_matrix_f(ir1ir2, S, Γ3)

if verbose:
    msg = "Finished"
    pbar.set_description(msg)
    pbar.refresh()

In [72]:
set(sum(all_zeros_2.values(),[])).intersection(set(super_solution_2.keys()))

{({\alpha}, {\beta}),
 ({\alpha}, {\mu}),
 ({\alpha}, {\phi}),
 ({\alpha}, {\zeta}),
 ({\beta}, {\alpha}),
 ({\beta}, {\gamma}),
 ({\beta}, {\nu}),
 ({\beta}, {\xi}),
 ({\gamma}, {\beta}),
 ({\gamma}, {\mu}),
 ({\gamma}, {\zeta}),
 ({\mu}, {\alpha}),
 ({\mu}, {\gamma}),
 ({\mu}, {\xi}),
 ({\nu}, {\beta}),
 ({\nu}, {\phi}),
 ({\phi}, {\alpha}),
 ({\phi}, {\nu}),
 ({\xi}, {\beta}),
 ({\xi}, {\mu}),
 ({\xi}, {\zeta}),
 ({\zeta}, {\alpha}),
 ({\zeta}, {\gamma}),
 ({\zeta}, {\xi})}

In [146]:
O = final_config_matrices

In [147]:
k = sp.Symbol('t_{2}')**2

In [148]:
sp.Matrix([[el.as_braket() for el in row] for row in simple_config_matrices[k][term_key]])

Matrix([
[<{\xi}{\phi}|{\xi}{\phi}> + <{\xi}{\xi}|{\phi}{\phi}>,                                                     0,                                                      0],
[                                                    0, <{\xi}{\phi}|{\xi}{\phi}> + <{\xi}{\xi}|{\phi}{\phi}>,                                                      0],
[                                                    0,                                                     0, -<{\xi}{\phi}|{\xi}{\phi}> + <{\xi}{\xi}|{\phi}{\phi}>]])

In [149]:
sp.Matrix([[el.as_braket() for el in row] for row in O[k][term_key]])

Matrix([
[<{\xi}{\phi}|{\xi}{\phi}>,                         0,                          0],
[                        0, <{\xi}{\phi}|{\xi}{\phi}>,                          0],
[                        0,                         0, -<{\xi}{\phi}|{\xi}{\phi}>]])

In [64]:
for term_key in O[k]:#[(1,sp.Symbol('T_{u}'))]#[0][0].as_braket()
    term_symb = sp.Symbol('{}^{%d}%s' % (term_key[0]*2+1,sp.latex(term_key[1])))
    display(term_symb)
    display(O[k][term_key][0][0].as_braket())
    display(simple_config_matrices[k][term_key][0][0].as_braket())


{}^{1}A_{1}

<{\xi}{\xi}|{\xi}{\xi}>

2*<{\xi}{\xi}|{\phi}{\phi}> + <{\xi}{\xi}|{\xi}{\xi}>

{}^{1}E

<{\xi}{\xi}|{\xi}{\xi}>

...

<{\xi}{\phi}|{\xi}{\phi}>

<{\xi}{\phi}|{\xi}{\phi}> - <{\xi}{\xi}|{\phi}{\phi}>

{}^{1}T_{2}

<{\xi}{\phi}|{\xi}{\phi}>

<{\xi}{\phi}|{\xi}{\phi}> + <{\xi}{\xi}|{\phi}{\phi}>

In [61]:
simple_config_matrices[k][term_key]

[[Qet({({\xi}, {\phi}, {\xi}, {\phi}): 1, ({\xi}, {\xi}, {\phi}, {\phi}): 1}),
  Qet({}),
  Qet({})],
 [Qet({}),
  Qet({({\xi}, {\phi}, {\xi}, {\phi}): 1, ({\xi}, {\xi}, {\phi}, {\phi}): 1}),
  Qet({})],
 [Qet({}),
  Qet({}),
  Qet({({\xi}, {\phi}, {\xi}, {\phi}): -1, ({\xi}, {\xi}, {\phi}, {\phi}): 1})]]

#### Debugging T_h

In [252]:
def simplify_qet(qet):
    new_dict = {k:sp.simplify(v) for k,v in qet.dict.items()}
    return Qet(new_dict)
def term_energies(group_label, verbose=True):
    stages = 19 # this is how many distinct parts this algorithm has
    if verbose:
        pbar = tqdm(range(stages), bar_format='({n:d}/{total:d}) [{elapsed}] {desc}')
    terms = twoe_terms[group_label]
    group = CPGs.get_group_by_label(group_label)
    component_labels = {k:list(v.values()) for k,v in new_labels[group_label].items()}
    # for a given set of terms
    # split all the enclosed states
    # in groups of electron configurations
    configs = {}
    config_supplement = {}
    for term_key, term in terms.items():
        for state_key, state in zip(term.state_keys, term.states):
            (Γ1, Γ2, Γ3, γ3, S, mSz) = state_key
            α = sp.Symbol(sp.latex(Γ1).lower())*sp.Symbol(sp.latex(Γ2).lower())
            if α not in configs.keys():
                configs[α] = {}
                config_supplement[α] = {}
            if (S,Γ3) not in configs[α].keys():
                configs[α][(S,Γ3)] = []
                config_supplement[α][(S,Γ3)] = []
            configs[α][(S,Γ3)].append(state)
            config_supplement[α][(S,Γ3)].append(state_key)

    def overlinesqueegee(s):
        '''
        Going back from the overline shorthand for spin down,
        acting on a single set of quantum numbers.
        '''
        if 'overline' in str(s):
            spin = -sp.S(1)/2
            comp = sp.Symbol(str(s).replace('\\overline{','')[:-1])
        else:
            spin = sp.S(1)/2
            comp = s
        return (comp, spin)
    def spin_restoration(qet):
        '''
        Going back from the overline shorthand for spin down,
        acting on a qet.
        '''
        new_dict = {}
        for k,v in qet.dict.items():
            k = (*overlinesqueegee(k[0]),*overlinesqueegee(k[1]))
            new_dict[k] = v
        return Qet(new_dict)

    # within each configuration
    config_matrices = {}
    for config in configs.keys():
        # display(config)
        # within each term
        term_matrices = {}
        for term_key, states in configs[config].items():
            # find the brackets between all possible pairs of states
            ham_matrix = []
            for state0 in states:
                ham_row = []
                state0 = spin_restoration(state0)
                for state1 in states:
                    # recover the spin for the determinantal states
                    pstate = state1
                    state1 = spin_restoration(state1)
                    # determine the det braket between these two states (an operator in between is assumed and omitted)
                    detbraket = state0.dual()*state1 # changehere
                    # simplify this braket of detstates into brakets between regular states
                    regbraket = Qet({})
                    for k,v in detbraket.dict.items():
                        γ1, m1, γ2, m2, γ1p, m1p, γ2p, m2p = k
                        # k0 = (γ1, m1, γ2, m2, γ1p, m1p, γ2p, m2p)
                        # k1 = (γ1, m1, γ2, m2, γ2p, m2p, γ1p, m1p)
                        k0 = (γ1, γ2, γ1p, γ2p)
                        k1 = (γ1, γ2, γ2p, γ1p)
                        # enforce orthogonality wrt. spin
                        if (m1 == m1p) and (m2 == m2p):
                            qplus = Qet({k0:v}) 
                        else:
                            qplus = Qet({})
                        if (m1 == m2p) and (m2 == m1p):
                            qminus = Qet({k1:-v}) 
                        else:
                            qminus = Qet({})
                        regbraket += (qplus+qminus)
                    ham_row.append(regbraket)
                ham_matrix.append(ham_row)
            term_matrices[term_key] = ham_matrix
        config_matrices[config] = term_matrices

    def composite_symbol(x):
        return sp.Symbol('(%s)'%(','.join(list(map(sp.latex,x)))))
    def fourtuplerecovery(ft):
        '''the inverse of composite_symbol'''
        return tuple(map(sp.Symbol,sp.latex(ft)[1:-1].split(',')))
    
    if verbose:
        msg = "Creating all 4-symbol identities ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1

    # this   integral_identities  dictionary  will  have  as  keys
    # 4-tuples  of  irreps  and  its  values  will  be lists whose
    # elements  are  2-tuples whose first elements are 4-tuples of
    # irrep  components  and  whose values are qets whose keys are
    # 4-tuples  of  irrep components and whose values are numeric.
    # These 4-tuples represent a braket with the Coulomb repulsion
    # operator in between.

    integral_identities = {}
    ir_mats = group.irrep_matrices
    for ir1, ir2, ir3, ir4 in product(*([group.irrep_labels]*4)):
        # To simplify calculations this part
        # may only be done over quadruples in a standard
        # order.
        # Whatever is missed here is then brough back in
        # by the reality relations.
        altorder1 = (ir3, ir2, ir1, ir4)
        altorder2 = (ir1, ir4, ir3, ir2)
        altorder3 = (ir3, ir4, ir1, ir2)
        if (altorder1 in integral_identities) or\
           (altorder2 in integral_identities) or\
           (altorder3 in integral_identities):
            continue
        integral_identity_sector = []
        components = [component_labels[ir] for ir in [ir1, ir2, ir3, ir4]]
        comp_to_idx = [{c: idx for idx, c in enumerate(component_labels[ir])} for ir in [ir1, ir2, ir3, ir4]]
        for R in group.generators + [sp.Symbol('{Ee}')]:
            R_id = {}
            for γ1, γ2, γ3, γ4 in product(*components):
                for γ1p, γ2p, γ3p, γ4p in product(*components):
#                         val = sp.conjugate(ir_mats[ir1][R][comp_to_idx[0][γ1],comp_to_idx[0][γ1p]]) *\
#                               sp.conjugate(ir_mats[ir2][R][comp_to_idx[1][γ2],comp_to_idx[1][γ2p]]) *\
#                               ir_mats[ir3][R][comp_to_idx[2][γ3],comp_to_idx[2][γ3p]] *\
#                               ir_mats[ir4][R][comp_to_idx[3][γ4],comp_to_idx[3][γ4p]]
                        val = sp.conjugate(ir_mats[ir1][R][comp_to_idx[0][γ1p],comp_to_idx[0][γ1]]) *\
                              sp.conjugate(ir_mats[ir2][R][comp_to_idx[1][γ2p],comp_to_idx[1][γ2]]) *\
                              ir_mats[ir3][R][comp_to_idx[2][γ3],comp_to_idx[2][γ3p]] *\
                              ir_mats[ir4][R][comp_to_idx[3][γ4],comp_to_idx[3][γ4p]]
                        if val== 0:
                            continue
                        key = (γ1, γ2, γ3, γ4)
                        if key not in R_id.keys():
                            R_id[key] = []
                        R_id[key].append( Qet({(γ1p,γ2p,γ3p,γ4p): val}) )
            R_id_total = [(key, sum(R_id[key], Qet({}))) for key in R_id.keys()]
            R_id_total = [q for q in R_id_total if len(q[1].dict)>0]
            integral_identity_sector.extend(R_id_total)
        integral_identities[(ir1,ir2,ir3,ir4)] = integral_identity_sector

    # For solving the linear system it is convenient
    # to have everything on one side of the equation.
    if verbose:
        msg = "Creating set of identities ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    identities = {}
    for ircombo in integral_identities:
        these_ids = []
        for v in integral_identities[ircombo]:
            lhs, rhs = v
            diff = Qet({lhs:1}) - rhs
            if len(diff.dict) > 0:
                these_ids.append(diff)
        identities[ircombo] = these_ids

    # If an equation has only one key, then
    # that immediately means that that braket is zero.
    if verbose:
        msg = "Finding trivial zeros ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    # first determine which ones have to be zero
    better_identities = {irc:[] for irc in identities}
    all_zeros = {}
    for ircombo in identities:
        zeros = []
        for identity in identities[ircombo]:
            if len(identity.dict) == 1:
                zeros.append((list(identity.dict.keys())[0]))
            else:
                better_identities[ircombo].append(identity)
        all_zeros[ircombo] = zeros

    # use them to simplify things.
    if verbose:
        msg = "Using them to simplify things ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    great_identities = {irc:[] for irc in identities}
    for ircombo in better_identities:
        for identity in better_identities[ircombo]:
            new_qet = Qet({})
            for k,v in identity.dict.items():
                if k in all_zeros[ircombo]:
                    continue
                else:
                    new_qet+= Qet({k: v})
            if len(new_qet.dict) == 0:
                continue
            great_identities[ircombo].append(new_qet)

    # Inside of a four-symbol braket one may do three exchanges
    # that must result in the same value. That if the wave
    # functions are assumed to be real-valued.
    
    if verbose:
        msg = "Creating reality identities ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    real_var_simplifiers = {irc:[] for irc in great_identities}
    kprimes = set()
    # this has to run over all the quadruples of irs
    for ir1, ir2, ir3, ir4 in product(*([group.irrep_labels]*4)):
        real_var_simplifier = {}
        ircombo = (ir1, ir2, ir3, ir4)
        components = [component_labels[ir] for ir in ircombo]
        for γ1, γ2, γ3, γ4 in product(*components):
            k = (γ1, γ2, γ3, γ4)
            kalt1 = (γ3, γ2, γ1, γ4)
            kalt2 = (γ1, γ4, γ3, γ2)
            kalt3 = (γ3, γ4, γ1, γ2)
            if kalt1 in kprimes:
                real_var_simplifier[k] = kalt1
            elif kalt2 in kprimes:
                real_var_simplifier[k] = kalt2
            elif kalt3 in kprimes:
                real_var_simplifier[k] = kalt3
            else:
                real_var_simplifier[k] = k
                kprimes.add(k)
        real_var_simplifiers[ircombo] = real_var_simplifier

    real_var_full_simplifier = {}
    for ircombo in real_var_simplifiers:
        real_var_full_simplifier.update(real_var_simplifiers[ircombo])

    # For each 4-tuple of irreps
    # create a system of symbolic solutions
    # and let sympy solve that.
    # For each 4-tuple of irreps
    # the end result is a dictionary
    # whose keys represent the dependent
    # brakets and whose values are the
    # relation that those dependent values
    # have with the independent brakets.
    # As such, when these dictionaries are
    # used on an expression, everything should
    # then be given in terms of indepedent brakets.

    if verbose:
        msg = "Solving for independent 4-symbol brakets ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    all_sols = {irc:[] for irc in great_identities}
    for ircombo in great_identities:
        zeros = all_zeros[ircombo]
        problem_vars = list(set(sum([list(identity.dict.keys()) for identity in great_identities[ircombo]],[])))
        big_ma = [awe.vec_in_basis(problem_vars) for awe in great_identities[ircombo]] 
        big_mat = sp.Matrix(big_ma)
        big_mat = sp.re(big_mat)+sp.I*sp.im(big_mat)
        rref_mat, pivots = big_mat.rref()
        num_rows = rref_mat.rows
        num_cols = rref_mat.cols
        rref_ma_non_zero = [rref_mat[row,:] for row in range(num_rows) if (sum(np.array(rref_mat[row,:])[0] == 0) != num_cols)]
        rref_mat_non_zero = sp.Matrix(rref_ma_non_zero)
        varvec = sp.Matrix(list(map(composite_symbol,problem_vars)))
        eqns = rref_mat_non_zero*varvec
        ssol = sp.solve(list(eqns), dict=True)
        all_sols[ircombo] = ssol
        assert len(ssol) in [0,1]
        if len(ssol) == 0:
            sol_dict = {}
        else:
            sol_dict = ssol[0]
        zero_addendum = {k:{} for k in all_zeros[ircombo]}
        sol_dict = {fourtuplerecovery(k):{fourtuplerecovery(s):v.coeff(s) for s in v.free_symbols} for k,v in sol_dict.items()}
        sol_dict.update(zero_addendum)
        all_sols[ircombo] = sol_dict

    # This final dictionary is unnecessary but
    # simplifies calling the replacements onto
    # a symbolic expression.
    if verbose:
        msg = "Creating a dictionary with all the 4-symbol replacements ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    super_solution = {}
    for ircombo in all_sols:
        super_solution.update(all_sols[ircombo])

    def simplifier(qet):
        simp_ket = Qet({})
        for k,v in qet.dict.items():
            simp_ket += Qet({real_var_full_simplifier[k]:v})
        true_qet = Qet({})
        for k,v in simp_ket.dict.items():
            if k in super_solution:
                true_qet += v*Qet(super_solution[k])
            else:
                true_qet += Qet({k:v})
        return true_qet

    def simplify_config_matrix(ir1ir2, S, ir3):
        config_matrix = config_matrices[ir1ir2][(S,ir3)]
        num_rows = len(config_matrix)
        simple_matrix = [[simplifier(config_matrix[row][col]) for col in range(num_rows)] for row in range(num_rows)]
        return simple_matrix
    
    if verbose:
        msg = "Simplifying configuration matrices ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    simple_config_matrices = {k:{} for k in config_matrices}
    for ir1ir2 in config_matrices.keys():
        for term_key in config_matrices[ir1ir2]:
            S, Γ3 = term_key
            simple_config_matrices[ir1ir2][term_key] = simplify_config_matrix(ir1ir2, S, Γ3)

    twotuplerecovery = fourtuplerecovery
    if verbose:
        msg = "Creating all 2-symbol identities ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1

    # this   integral_identities  dictionary  will  have  as  keys
    # 2-tuples  of  irreps  and  its  values  will  be lists whose
    # elements  are  2-tuples whose first elements are 2-tuples of
    # irrep  components  and  whose values are qets whose keys are
    # 2-tuples  of  irrep components and whose values are numeric.
    # These 2-tuples represent a braket with the an invariant operator
    # operator in between.

    integral_identities_2 = {}
    ir_mats = group.irrep_matrices
    for ir1, ir2 in product(*([group.irrep_labels]*2)):
        # To simplify calculations this part
        # can only be done over quadruples in a standard
        # order.
        # Whatever is missed here is then brough back in
        # by the reality relations.
        altorder1 = (ir2, ir1, ir1, ir4)
        if (altorder1 in integral_identities_2):
            continue
        integral_identity_sector = []
        components = [component_labels[ir] for ir in [ir1, ir2]]
        comp_to_idx = [{c: idx for idx, c in enumerate(component_labels[ir])} for ir in [ir1, ir2]]
        for R in group.generators + [sp.Symbol('{Ee}')]:
            R_id = {}
            for γ1, γ2 in product(*components):
                for γ1p, γ2p in product(*components):
#                         val = sp.conjugate(ir_mats[ir1][R][comp_to_idx[0][γ1],comp_to_idx[0][γ1p]]) *\
#                               ir_mats[ir2][R][comp_to_idx[1][γ2],comp_to_idx[1][γ2p]]
                        val = sp.conjugate(ir_mats[ir1][R][comp_to_idx[0][γ1p],comp_to_idx[0][γ1]]) *\
                              ir_mats[ir2][R][comp_to_idx[1][γ2],comp_to_idx[1][γ2p]]
                        if val== 0:
                            continue
                        key = (γ1, γ2)
                        if key not in R_id.keys():
                            R_id[key] = []
                        R_id[key].append( Qet({(γ1p,γ2p): val}) )
            R_id_total = [(key, sum(R_id[key], Qet({}))) for key in R_id.keys()]
            R_id_total = [q for q in R_id_total if len(q[1].dict)>0]
            integral_identity_sector.extend(R_id_total)
        integral_identities_2[(ir1,ir2)] = integral_identity_sector

    # For solving the linear system it is convenient
    # to have everything on one side of the equation.
    if verbose:
        msg = "Creating set of 2 symbol identities ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    identities_2 = {}
    
    for ircombo in integral_identities_2:
        these_ids = []
        for v in integral_identities_2[ircombo]:
            lhs, rhs = v
            diff = Qet({lhs:1}) - rhs
            if len(diff.dict) > 0:
                these_ids.append(diff)
        identities_2[ircombo] = these_ids

    # If an equation has only one term, then
    # that immediately means that that term is zero.
    if verbose:
        msg = "Finding trivial zeros ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    # first determine which ones have to be zero
    better_identities_2 = {irc:[] for irc in identities_2}
    all_zeros_2 = {}
    for ircombo in identities_2:
        zeros = []
        for identity in identities_2[ircombo]:
            if len(identity.dict) == 1:
                zeros.append((list(identity.dict.keys())[0]))
            else:
                better_identities_2[ircombo].append(identity)
        all_zeros_2[ircombo] = zeros

    # use them to simplify things.
    if verbose:
        msg = "Using them to simplify things ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    great_identities_2 = {irc:[] for irc in identities_2}
    for ircombo in better_identities_2:
        for identity in better_identities_2[ircombo]:
            new_qet = Qet({})
            for k,v in identity.dict.items():
                if k in all_zeros_2[ircombo]:
                    continue
                else:
                    new_qet+= Qet({k: v})
            if len(new_qet.dict) == 0:
                continue
            great_identities_2[ircombo].append(new_qet)

    # Inside of a two-symbol braket one may do three exchanges
    # that must result in the same value. That if the wave
    # functions are assumed to be real-valued.
    if verbose:
        msg = "Creating reality identities ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    real_var_simplifiers_2 = {irc:[] for irc in great_identities_2}
    # this has to run over all the quadruples of irs
    kprimes = set()
    for ir1, ir2 in product(*([group.irrep_labels]*2)):
        real_var_simplifier = {}
        ircombo = (ir1, ir2)
        components = [component_labels[ir] for ir in ircombo]
        for γ1, γ2 in product(*components):
            k = (γ1, γ2)
            kalt = (γ2, γ1)
            # If for a given key I find that its
            # switched version has already been seen
            # Then that key has to be mapped to be
            # mapped to the key already present.
            if kalt in kprimes:
                real_var_simplifier[k] = kalt
            else:
                real_var_simplifier[k] = k
                kprimes.add(k)
        real_var_simplifiers_2[ircombo] = real_var_simplifier

    # For each 2-tuple of irreps
    # create a system of symbolic solutions
    # and let sympy solve that.
    # For each 2-tuple of irreps
    # the end result is a dictionary
    # whose keys represent the dependent
    # brakets and whose values are the
    # relation that those dependent values
    # have with the independent brakets.
    # As such, when these dictionaries are
    # used on an expression, everything should
    # then be given in terms of indepedent brakets.
    if verbose:
        msg = "Solving for independent 2-symbol brakets ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    all_sols_2 = {irc:[] for irc in great_identities_2}
    
    for ircombo in great_identities_2:
        zeros = all_zeros_2[ircombo]
        problem_vars = list(set(sum([list(identity.dict.keys()) for identity in great_identities_2[ircombo]],[])))
        big_ma = [awe.vec_in_basis(problem_vars) for awe in great_identities_2[ircombo]] 
        big_mat = sp.Matrix(big_ma)
        big_mat = sp.re(big_mat)+sp.I*sp.im(big_mat)
        rref_mat, pivots = big_mat.rref()
        num_rows = rref_mat.rows
        num_cols = rref_mat.cols
        rref_ma_non_zero = [rref_mat[row,:] for row in range(num_rows) if (sum(np.array(rref_mat[row,:])[0] == 0) != num_cols)]
        rref_mat_non_zero = sp.Matrix(rref_ma_non_zero)
        varvec = sp.Matrix(list(map(composite_symbol,problem_vars)))
        eqns = rref_mat_non_zero*varvec
        ssol = sp.solve(list(eqns), dict=True)
        all_sols[ircombo] = ssol
        assert len(ssol) in [0,1]
        if len(ssol) == 0:
            sol_dict = {}
        else:
            sol_dict = ssol[0]
        zero_addendum = {k:{} for k in all_zeros_2[ircombo]}
        sol_dict = {twotuplerecovery(k):{twotuplerecovery(s):v.coeff(s) for s in v.free_symbols} for k,v in sol_dict.items()}
        sol_dict.update(zero_addendum)
        all_sols_2[ircombo] = sol_dict

    # This final dictionary is unnecessary but
    # simplifies calling the replacements onto
    # a symbolic expression.
    if verbose:
        msg = "Creating a dictionary with all the 2-symbol replacements ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    super_solution_2 = {}
    for ircombo in all_sols_2:
        super_solution_2.update(all_sols_2[ircombo])

    # for a given electron config
    more_ids = {e_config:[] for e_config in simple_config_matrices}
    for e_config in simple_config_matrices:
        for term in simple_config_matrices[e_config]:
            the_matrix = simple_config_matrices[e_config][term]
            num_rows = len(the_matrix)
            num_cols = len(the_matrix)
            the_state_keys = config_supplement[e_config][term]
            the_key_matrix = {}
            for row_idx in range(num_rows):
                Γ3_row = the_state_keys[row_idx][2]
                γ3_row = the_state_keys[row_idx][3]
                for col_idx in range(num_cols):
                    Γ3_col = the_state_keys[col_idx][2]
                    γ3_col = the_state_keys[col_idx][3]
                    matrix_val = the_matrix[row_idx][col_idx]
                    the_key_matrix[(γ3_row,γ3_col)] = matrix_val
            # now go over the keys of super_solution_2
            # and if one of those keys matches with a key in the_key_matrix
            # do something about it
            for k in super_solution_2:
                if k in the_key_matrix:
                    v = super_solution_2[k]
                    # this matrix element
                    matrix_element = the_key_matrix[k]
                    # must be identified with the sum
                    # as given in v
                    matrix_equiv = sum([kv*the_key_matrix[km] for km, kv in v.items()],Qet({}))
                    identity = (matrix_element - matrix_equiv) #=0
                    if len(identity.dict) != 0:
                        more_ids[e_config].append(identity)
                        problem_vars = list(set(sum([list(identity.dict.keys()) for identity in more_ids[e_config]],[])))
    if verbose:
        msg = "Simplifying numeric values of qets ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    more_ids = {e_config:list(filter(lambda x: len(x.dict) > 0,list(map(simplify_qet,more_ids[e_config])))) for e_config in more_ids}
    
    if verbose:
        msg = "Solving for dependent vars in terms of independent ones ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    all_sols_2_4 = {irc:[] for irc in more_ids}
    
    for e_config in more_ids:
        problem_vars = list(set(sum([list(identity.dict.keys()) for identity in more_ids[e_config]],[])))
        big_ma = [awe.vec_in_basis(problem_vars) for awe in more_ids[e_config]] 
        big_mat = sp.Matrix(big_ma)
        big_mat = sp.re(big_mat)+sp.I*sp.im(big_mat)
        rref_mat, pivots = big_mat.rref()
        num_rows = rref_mat.rows
        num_cols = rref_mat.cols
        rref_ma_non_zero = [rref_mat[row,:] for row in range(num_rows) if (sum(np.array(rref_mat[row,:])[0] == 0) != num_cols)]
        rref_mat_non_zero = sp.Matrix(rref_ma_non_zero)
        varvec = sp.Matrix(list(map(composite_symbol, problem_vars)))
        eqns = rref_mat_non_zero*varvec
        ssol = sp.solve(list(eqns), dict=True)
        all_sols_2_4[e_config] = ssol
        assert len(ssol) in [0,1]
        if len(ssol) == 0:
            sol_dict = {}
        else:
            sol_dict = ssol[0]
        sol_dict = {twotuplerecovery(k):{twotuplerecovery(s):v.coeff(s) for s in v.free_symbols} for k,v in sol_dict.items()}
        all_sols_2_4[e_config] = sol_dict

    # flatten into a single dictionary of replacements
    fab_solution_2_4 = {}
    for e_config in all_sols_2_4:
        fab_solution_2_4.update(all_sols_2_4[e_config])
    if verbose:
        msg = "There are %d less independent variables ..."
        pbar.set_description("There are %d less independent variables ..." % len(fab_solution_2_4))
        pbar.refresh()
        pbar.n += 1
    
    def simplifier_f(qet):
        true_qet = Qet({})
        for k,v in qet.dict.items():
            if k in fab_solution_2_4:
                true_qet += v*Qet(fab_solution_2_4[k])
            else:
                true_qet += Qet({k:v})
        return true_qet
    
    def simplify_config_matrix_f(ir1ir2, S, ir3):
        config_matrix = simple_config_matrices[ir1ir2][(S,ir3)]
        num_rows = len(config_matrix)
        simple_matrix = [[simplifier_f(config_matrix[row][col]) for col in range(num_rows)] for row in range(num_rows)]
        #simple_matrix = sp.Matrix(simple_matrix)
        return simple_matrix
    
    if verbose:
        msg = "Making final simplifications ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    final_config_matrices = {k:{} for k in config_matrices}
    for ir1ir2 in config_matrices.keys():
        for term_key in config_matrices[ir1ir2]:
            S, Γ3 = term_key
            final_config_matrices[ir1ir2][term_key] = simplify_config_matrix_f(ir1ir2, S, Γ3)
    
    if verbose:
        msg = "Finished"
        pbar.set_description(msg)
        pbar.refresh()
    
    return final_config_matrices

In [253]:
Th = term_energies('T_{h}')

(0/19) [00:00] 

In [254]:
for k in Th.keys():
    if 'e_{u}^1' in sp.latex(k):
        if 't_{g}' in sp.latex(k):
            print("x")
            print(k)
            break

x
e_{u}^1*t_{g}


In [255]:
Th[k][(1,sp.Symbol('T_{u}'))]#[0][0].as_braket()

[[Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({})],
 [Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({})],
 [Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({})],
 [Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({})],
 [Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({})],
 [Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({})],
 [Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({})],
 [Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({})],
 [Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({}),
  Qet({})]]

In [256]:
all_term_energies = {}
for group_label in CPGs.all_group_labels:
    if group_label in all_term_energies:
        continue
    final_config_matrices = term_energies(group_label)
#     print("Printing ...")
#     for counter, ir1ir2 in enumerate(final_config_matrices):
#         print('-'*10)
#         display(ir1ir2)
#         for term_key in final_config_matrices[ir1ir2]:
#             display(term_key)
#             display(sp.Matrix([[el.as_braket() for el in row] for row in final_config_matrices[ir1ir2][term_key]]))
#     print("Saving ...")
    all_term_energies[group_label] = final_config_matrices

(0/19) [00:00] 

(0/19) [00:00] 

(0/19) [00:00] 

(0/19) [00:00] 

(0/19) [00:00] 

...

(0/19) [00:00] 

(0/19) [00:00] 

(0/19) [00:00] 

(0/19) [00:00] 

(0/19) [00:00] 

In [257]:
def simplify_qet_values(qet):
    sqetdict = {k:sp.simplify(v) for k,v in qet.dict.items()}
    return Qet(sqetdict)

In [258]:
term_energies_for_printout = {}
for group_label, term_energies in all_term_energies.items():
    for e_config, terms  in term_energies.items():
        for term_key, term_matrix in terms.items():
            term_energy = term_matrix[0][0]
            term_energy = simplify_qet_values(term_energy)
            term_energies_for_printout[(group_label, e_config, term_key)] = term_energy
#             out = sp.simplify(term_energy.as_braket())
#             if len(sp.latex(out))> 200:
#                 if 'sqrt' in sp.latex(out) and 'i' in sp.latex(out):
#                     display(term_key)
#                     display(sp.simplify(term_energy.as_braket()))

In [259]:
pickle.dump(term_energies_for_printout, open('./data/term_energies_for_print_alt.pkl','wb'))

In [260]:
!beep

#### All-groups

In [3]:
from tqdm.notebook import tqdm

In [17]:
def simplify_qet(qet):
    new_dict = {k:sp.simplify(v) for k,v in qet.dict.items()}
    return Qet(new_dict)
def term_energies(group_label, verbose=True):
    stages = 19 # this is how many distinct parts this algorithm has
    if verbose:
        pbar = tqdm(range(stages), bar_format='({n:d}/{total:d}) [{elapsed}] {desc}')
    terms = twoe_terms[group_label]
    group = CPGs.get_group_by_label(group_label)
    component_labels = {k:list(v.values()) for k,v in new_labels[group_label].items()}
    # for a given set of terms
    # split all the enclosed states
    # in groups of electron configurations
    configs = {}
    config_supplement = {}
    for term_key, term in terms.items():
        for state_key, state in zip(term.state_keys, term.states):
            (Γ1, Γ2, Γ3, γ3, S, mSz) = state_key
            α = sp.Symbol(sp.latex(Γ1).lower())*sp.Symbol(sp.latex(Γ2).lower())
            if α not in configs.keys():
                configs[α] = {}
                config_supplement[α] = {}
            if (S,Γ3) not in configs[α].keys():
                configs[α][(S,Γ3)] = []
                config_supplement[α][(S,Γ3)] = []
            configs[α][(S,Γ3)].append(state)
            config_supplement[α][(S,Γ3)].append(state_key)

    def overlinesqueegee(s):
        '''
        Going back from the overline shorthand for spin down,
        acting on a single set of quantum numbers.
        '''
        if 'overline' in str(s):
            spin = -sp.S(1)/2
            comp = sp.Symbol(str(s).replace('\\overline{','')[:-1])
        else:
            spin = sp.S(1)/2
            comp = s
        return (comp, spin)
    def spin_restoration(qet):
        '''
        Going back from the overline shorthand for spin down,
        acting on a qet.
        '''
        new_dict = {}
        for k,v in qet.dict.items():
            k = (*overlinesqueegee(k[0]),*overlinesqueegee(k[1]))
            new_dict[k] = v
        return Qet(new_dict)

    # within each configuration
    config_matrices = {}
    for config in configs.keys():
        # display(config)
        # within each term
        term_matrices = {}
        for term_key, states in configs[config].items():
            # find the brackets between all possible pairs of states
            ham_matrix = []
            for state0 in states:
                ham_row = []
                state0 = spin_restoration(state0)
                for state1 in states:
                    # recover the spin for the determinantal states
                    pstate = state1
                    state1 = spin_restoration(state1)
                    # determine the det braket between these two states (an operator in between is assumed and omitted)
                    detbraket = state0*state1
                    # simplify this braket of detstates into brakets between regular states
                    regbraket = Qet({})
                    for k,v in detbraket.dict.items():
                        γ1, m1, γ2, m2, γ1p, m1p, γ2p, m2p = k
                        # k0 = (γ1, m1, γ2, m2, γ1p, m1p, γ2p, m2p)
                        # k1 = (γ1, m1, γ2, m2, γ2p, m2p, γ1p, m1p)
                        k0 = (γ1, γ2, γ1p, γ2p)
                        k1 = (γ1, γ2, γ2p, γ1p)
                        # enforce orthogonality wrt. spin
                        if (m1 == m1p) and (m2 == m2p):
                            qplus = Qet({k0:v}) 
                        else:
                            qplus = Qet({})
                        if (m1 == m2p) and (m2 == m1p):
                            qminus = Qet({k1:-v}) 
                        else:
                            qminus = Qet({})
                        regbraket += (qplus+qminus)
                    ham_row.append(regbraket)
                ham_matrix.append(ham_row)
            term_matrices[term_key] = ham_matrix
        config_matrices[config] = term_matrices

    def composite_symbol(x):
        return sp.Symbol('(%s)'%(','.join(list(map(sp.latex,x)))))
    def fourtuplerecovery(ft):
        '''the inverse of composite_symbol'''
        return tuple(map(sp.Symbol,sp.latex(ft)[1:-1].split(',')))
    
    if verbose:
        msg = "Creating all 4-symbol identities ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1

    # this   integral_identities  dictionary  will  have  as  keys
    # 4-tuples  of  irreps  and  its  values  will  be lists whose
    # elements  are  2-tuples whose first elements are 4-tuples of
    # irrep  components  and  whose values are qets whose keys are
    # 4-tuples  of  irrep components and whose values are numeric.
    # These 4-tuples represent a braket with the Coulomb repulsion
    # operator in between.

    integral_identities = {}
    ir_mats = group.irrep_matrices
    for ir1, ir2, ir3, ir4 in product(*([group.irrep_labels]*4)):
        # To simplify calculations this part
        # may only be done over quadruples in a standard
        # order.
        # Whatever is missed here is then brough back in
        # by the reality relations.
        altorder1 = (ir3, ir2, ir1, ir4)
        altorder2 = (ir1, ir4, ir3, ir2)
        altorder3 = (ir3, ir4, ir1, ir2)
        if (altorder1 in integral_identities) or\
           (altorder2 in integral_identities) or\
           (altorder3 in integral_identities):
            continue
        integral_identity_sector = []
        components = [component_labels[ir] for ir in [ir1, ir2, ir3, ir4]]
        comp_to_idx = [{c: idx for idx, c in enumerate(component_labels[ir])} for ir in [ir1, ir2, ir3, ir4]]
        for R in group.generators + [sp.Symbol('{Ee}')]:
            R_id = {}
            for γ1, γ2, γ3, γ4 in product(*components):
                for γ1p, γ2p, γ3p, γ4p in product(*components):
#                         val = sp.conjugate(ir_mats[ir1][R][comp_to_idx[0][γ1],comp_to_idx[0][γ1p]]) *\
#                               sp.conjugate(ir_mats[ir2][R][comp_to_idx[1][γ2],comp_to_idx[1][γ2p]]) *\
#                               ir_mats[ir3][R][comp_to_idx[2][γ3],comp_to_idx[2][γ3p]] *\
#                               ir_mats[ir4][R][comp_to_idx[3][γ4],comp_to_idx[3][γ4p]]
                        val = sp.conjugate(ir_mats[ir1][R][comp_to_idx[0][γ1p],comp_to_idx[0][γ1]]) *\
                              sp.conjugate(ir_mats[ir2][R][comp_to_idx[1][γ2p],comp_to_idx[1][γ2]]) *\
                              ir_mats[ir3][R][comp_to_idx[2][γ3],comp_to_idx[2][γ3p]] *\
                              ir_mats[ir4][R][comp_to_idx[3][γ4],comp_to_idx[3][γ4p]]
                        if val== 0:
                            continue
                        key = (γ1, γ2, γ3, γ4)
                        if key not in R_id.keys():
                            R_id[key] = []
                        R_id[key].append( Qet({(γ1p,γ2p,γ3p,γ4p): val}) )
            R_id_total = [(key, sum(R_id[key], Qet({}))) for key in R_id.keys()]
            R_id_total = [q for q in R_id_total if len(q[1].dict)>0]
            integral_identity_sector.extend(R_id_total)
        integral_identities[(ir1,ir2,ir3,ir4)] = integral_identity_sector

    # For solving the linear system it is convenient
    # to have everything on one side of the equation.
    if verbose:
        msg = "Creating set of identities ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    identities = {}
    for ircombo in integral_identities:
        these_ids = []
        for v in integral_identities[ircombo]:
            lhs, rhs = v
            diff = Qet({lhs:1}) - rhs
            if len(diff.dict) > 0:
                these_ids.append(diff)
        identities[ircombo] = these_ids

    # If an equation has only one key, then
    # that immediately means that that braket is zero.
    if verbose:
        msg = "Finding trivial zeros ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    # first determine which ones have to be zero
    better_identities = {irc:[] for irc in identities}
    all_zeros = {}
    for ircombo in identities:
        zeros = []
        for identity in identities[ircombo]:
            if len(identity.dict) == 1:
                zeros.append((list(identity.dict.keys())[0]))
            else:
                better_identities[ircombo].append(identity)
        all_zeros[ircombo] = zeros

    # use them to simplify things.
    if verbose:
        msg = "Using them to simplify things ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    great_identities = {irc:[] for irc in identities}
    for ircombo in better_identities:
        for identity in better_identities[ircombo]:
            new_qet = Qet({})
            for k,v in identity.dict.items():
                if k in all_zeros[ircombo]:
                    continue
                else:
                    new_qet+= Qet({k: v})
            if len(new_qet.dict) == 0:
                continue
            great_identities[ircombo].append(new_qet)

    # Inside of a four-symbol braket one may do three exchanges
    # that must result in the same value. That if the wave
    # functions are assumed to be real-valued.
    
    if verbose:
        msg = "Creating reality identities ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    real_var_simplifiers = {irc:[] for irc in great_identities}
    kprimes = set()
    # this has to run over all the quadruples of irs
    for ir1, ir2, ir3, ir4 in product(*([group.irrep_labels]*4)):
        real_var_simplifier = {}
        ircombo = (ir1, ir2, ir3, ir4)
        components = [component_labels[ir] for ir in ircombo]
        for γ1, γ2, γ3, γ4 in product(*components):
            k = (γ1, γ2, γ3, γ4)
            kalt1 = (γ3, γ2, γ1, γ4)
            kalt2 = (γ1, γ4, γ3, γ2)
            kalt3 = (γ3, γ4, γ1, γ2)
            if kalt1 in kprimes:
                real_var_simplifier[k] = kalt1
            elif kalt2 in kprimes:
                real_var_simplifier[k] = kalt2
            elif kalt3 in kprimes:
                real_var_simplifier[k] = kalt3
            else:
                real_var_simplifier[k] = k
                kprimes.add(k)
        real_var_simplifiers[ircombo] = real_var_simplifier

    real_var_full_simplifier = {}
    for ircombo in real_var_simplifiers:
        real_var_full_simplifier.update(real_var_simplifiers[ircombo])

    # For each 4-tuple of irreps
    # create a system of symbolic solutions
    # and let sympy solve that.
    # For each 4-tuple of irreps
    # the end result is a dictionary
    # whose keys represent the dependent
    # brakets and whose values are the
    # relation that those dependent values
    # have with the independent brakets.
    # As such, when these dictionaries are
    # used on an expression, everything should
    # then be given in terms of indepedent brakets.

    if verbose:
        msg = "Solving for independent 4-symbol brakets ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    all_sols = {irc:[] for irc in great_identities}
    for ircombo in great_identities:
        zeros = all_zeros[ircombo]
        problem_vars = list(set(sum([list(identity.dict.keys()) for identity in great_identities[ircombo]],[])))
        big_ma = [awe.vec_in_basis(problem_vars) for awe in great_identities[ircombo]] 
        big_mat = sp.Matrix(big_ma)
        big_mat = sp.re(big_mat)+sp.I*sp.im(big_mat)
        rref_mat, pivots = big_mat.rref()
        num_rows = rref_mat.rows
        num_cols = rref_mat.cols
        rref_ma_non_zero = [rref_mat[row,:] for row in range(num_rows) if (sum(np.array(rref_mat[row,:])[0] == 0) != num_cols)]
        rref_mat_non_zero = sp.Matrix(rref_ma_non_zero)
        varvec = sp.Matrix(list(map(composite_symbol,problem_vars)))
        eqns = rref_mat_non_zero*varvec
        ssol = sp.solve(list(eqns), dict=True)
        all_sols[ircombo] = ssol
        assert len(ssol) in [0,1]
        if len(ssol) == 0:
            sol_dict = {}
        else:
            sol_dict = ssol[0]
        zero_addendum = {k:{} for k in all_zeros[ircombo]}
        sol_dict = {fourtuplerecovery(k):{fourtuplerecovery(s):v.coeff(s) for s in v.free_symbols} for k,v in sol_dict.items()}
        sol_dict.update(zero_addendum)
        all_sols[ircombo] = sol_dict

    # This final dictionary is unnecessary but
    # simplifies calling the replacements onto
    # a symbolic expression.
    if verbose:
        msg = "Creating a dictionary with all the 4-symbol replacements ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    super_solution = {}
    for ircombo in all_sols:
        super_solution.update(all_sols[ircombo])

    def simplifier(qet):
        simp_ket = Qet({})
        for k,v in qet.dict.items():
            simp_ket += Qet({real_var_full_simplifier[k]:v})
        true_qet = Qet({})
        for k,v in simp_ket.dict.items():
            if k in super_solution:
                true_qet += v*Qet(super_solution[k])
            else:
                true_qet += Qet({k:v})
        return true_qet

    def simplify_config_matrix(ir1ir2, S, ir3):
        config_matrix = config_matrices[ir1ir2][(S,ir3)]
        num_rows = len(config_matrix)
        simple_matrix = [[simplifier(config_matrix[row][col]) for col in range(num_rows)] for row in range(num_rows)]
        return simple_matrix
    
    if verbose:
        msg = "Simplifying configuration matrices ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    simple_config_matrices = {k:{} for k in config_matrices}
    for ir1ir2 in config_matrices.keys():
        for term_key in config_matrices[ir1ir2]:
            S, Γ3 = term_key
            simple_config_matrices[ir1ir2][term_key] = simplify_config_matrix(ir1ir2, S, Γ3)

    twotuplerecovery = fourtuplerecovery
    if verbose:
        msg = "Creating all 2-symbol identities ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1

    # this   integral_identities  dictionary  will  have  as  keys
    # 2-tuples  of  irreps  and  its  values  will  be lists whose
    # elements  are  2-tuples whose first elements are 2-tuples of
    # irrep  components  and  whose values are qets whose keys are
    # 2-tuples  of  irrep components and whose values are numeric.
    # These 2-tuples represent a braket with the an invariant operator
    # operator in between.

    integral_identities_2 = {}
    ir_mats = group.irrep_matrices
    for ir1, ir2 in product(*([group.irrep_labels]*2)):
        # To simplify calculations this part
        # can only be done over quadruples in a standard
        # order.
        # Whatever is missed here is then brough back in
        # by the reality relations.
        altorder1 = (ir2, ir1, ir1, ir4)
        if (altorder1 in integral_identities_2):
            continue
        integral_identity_sector = []
        components = [component_labels[ir] for ir in [ir1, ir2]]
        comp_to_idx = [{c: idx for idx, c in enumerate(component_labels[ir])} for ir in [ir1, ir2]]
        for R in group.generators + [sp.Symbol('{Ee}')]:
            R_id = {}
            for γ1, γ2 in product(*components):
                for γ1p, γ2p in product(*components):
#                         val = sp.conjugate(ir_mats[ir1][R][comp_to_idx[0][γ1],comp_to_idx[0][γ1p]]) *\
#                               ir_mats[ir2][R][comp_to_idx[1][γ2],comp_to_idx[1][γ2p]]
                        val = sp.conjugate(ir_mats[ir1][R][comp_to_idx[0][γ1p],comp_to_idx[0][γ1]]) *\
                              ir_mats[ir2][R][comp_to_idx[1][γ2],comp_to_idx[1][γ2p]]
                        if val== 0:
                            continue
                        key = (γ1, γ2)
                        if key not in R_id.keys():
                            R_id[key] = []
                        R_id[key].append( Qet({(γ1p,γ2p): val}) )
            R_id_total = [(key, sum(R_id[key], Qet({}))) for key in R_id.keys()]
            R_id_total = [q for q in R_id_total if len(q[1].dict)>0]
            integral_identity_sector.extend(R_id_total)
        integral_identities_2[(ir1,ir2)] = integral_identity_sector

    # For solving the linear system it is convenient
    # to have everything on one side of the equation.
    if verbose:
        msg = "Creating set of 2 symbol identities ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    identities_2 = {}
    
    for ircombo in integral_identities_2:
        these_ids = []
        for v in integral_identities_2[ircombo]:
            lhs, rhs = v
            diff = Qet({lhs:1}) - rhs
            if len(diff.dict) > 0:
                these_ids.append(diff)
        identities_2[ircombo] = these_ids

    # If an equation has only one term, then
    # that immediately means that that term is zero.
    if verbose:
        msg = "Finding trivial zeros ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    # first determine which ones have to be zero
    better_identities_2 = {irc:[] for irc in identities_2}
    all_zeros_2 = {}
    for ircombo in identities_2:
        zeros = []
        for identity in identities_2[ircombo]:
            if len(identity.dict) == 1:
                zeros.append((list(identity.dict.keys())[0]))
            else:
                better_identities_2[ircombo].append(identity)
        all_zeros_2[ircombo] = zeros

    # use them to simplify things.
    if verbose:
        msg = "Using them to simplify things ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    great_identities_2 = {irc:[] for irc in identities_2}
    for ircombo in better_identities_2:
        for identity in better_identities_2[ircombo]:
            new_qet = Qet({})
            for k,v in identity.dict.items():
                if k in all_zeros_2[ircombo]:
                    continue
                else:
                    new_qet+= Qet({k: v})
            if len(new_qet.dict) == 0:
                continue
            great_identities_2[ircombo].append(new_qet)

    # Inside of a two-symbol braket one may do three exchanges
    # that must result in the same value. That if the wave
    # functions are assumed to be real-valued.
    if verbose:
        msg = "Creating reality identities ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    real_var_simplifiers_2 = {irc:[] for irc in great_identities_2}
    # this has to run over all the quadruples of irs
    kprimes = set()
    for ir1, ir2 in product(*([group.irrep_labels]*2)):
        real_var_simplifier = {}
        ircombo = (ir1, ir2)
        components = [component_labels[ir] for ir in ircombo]
        for γ1, γ2 in product(*components):
            k = (γ1, γ2)
            kalt = (γ2, γ1)
            # If for a given key I find that its
            # switched version has already been seen
            # Then that key has to be mapped to be
            # mapped to the key already present.
            if kalt in kprimes:
                real_var_simplifier[k] = kalt
            else:
                real_var_simplifier[k] = k
                kprimes.add(k)
        real_var_simplifiers_2[ircombo] = real_var_simplifier

    # For each 2-tuple of irreps
    # create a system of symbolic solutions
    # and let sympy solve that.
    # For each 2-tuple of irreps
    # the end result is a dictionary
    # whose keys represent the dependent
    # brakets and whose values are the
    # relation that those dependent values
    # have with the independent brakets.
    # As such, when these dictionaries are
    # used on an expression, everything should
    # then be given in terms of indepedent brakets.
    if verbose:
        msg = "Solving for independent 2-symbol brakets ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    all_sols_2 = {irc:[] for irc in great_identities_2}
    
    for ircombo in great_identities_2:
        zeros = all_zeros_2[ircombo]
        problem_vars = list(set(sum([list(identity.dict.keys()) for identity in great_identities_2[ircombo]],[])))
        big_ma = [awe.vec_in_basis(problem_vars) for awe in great_identities_2[ircombo]] 
        big_mat = sp.Matrix(big_ma)
        big_mat = sp.re(big_mat)+sp.I*sp.im(big_mat)
        rref_mat, pivots = big_mat.rref()
        num_rows = rref_mat.rows
        num_cols = rref_mat.cols
        rref_ma_non_zero = [rref_mat[row,:] for row in range(num_rows) if (sum(np.array(rref_mat[row,:])[0] == 0) != num_cols)]
        rref_mat_non_zero = sp.Matrix(rref_ma_non_zero)
        varvec = sp.Matrix(list(map(composite_symbol,problem_vars)))
        eqns = rref_mat_non_zero*varvec
        ssol = sp.solve(list(eqns), dict=True)
        all_sols[ircombo] = ssol
        assert len(ssol) in [0,1]
        if len(ssol) == 0:
            sol_dict = {}
        else:
            sol_dict = ssol[0]
        zero_addendum = {k:{} for k in all_zeros_2[ircombo]}
        sol_dict = {twotuplerecovery(k):{twotuplerecovery(s):v.coeff(s) for s in v.free_symbols} for k,v in sol_dict.items()}
        sol_dict.update(zero_addendum)
        all_sols_2[ircombo] = sol_dict

    # This final dictionary is unnecessary but
    # simplifies calling the replacements onto
    # a symbolic expression.
    if verbose:
        msg = "Creating a dictionary with all the 2-symbol replacements ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    super_solution_2 = {}
    for ircombo in all_sols_2:
        super_solution_2.update(all_sols_2[ircombo])

    # for a given electron config
    more_ids = {e_config:[] for e_config in simple_config_matrices}
    for e_config in simple_config_matrices:
        for term in simple_config_matrices[e_config]:
            the_matrix = simple_config_matrices[e_config][term]
            num_rows = len(the_matrix)
            num_cols = len(the_matrix)
            the_state_keys = config_supplement[e_config][term]
            the_key_matrix = {}
            for row_idx in range(num_rows):
                Γ3_row = the_state_keys[row_idx][2]
                γ3_row = the_state_keys[row_idx][3]
                for col_idx in range(num_cols):
                    Γ3_col = the_state_keys[col_idx][2]
                    γ3_col = the_state_keys[col_idx][3]
                    matrix_val = the_matrix[row_idx][col_idx]
                    the_key_matrix[(γ3_row,γ3_col)] = matrix_val
            # now go over the keys of super_solution_2
            # and if one of those keys matches with a key in the_key_matrix
            # do something about it
            for k in super_solution_2:
                if k in the_key_matrix:
                    v = super_solution_2[k]
                    # this matrix element
                    matrix_element = the_key_matrix[k]
                    # must be identified with the sum
                    # as given in v
                    matrix_equiv = sum([kv*the_key_matrix[km] for km, kv in v.items()],Qet({}))
                    identity = (matrix_element - matrix_equiv) #=0
                    if len(identity.dict) != 0:
                        more_ids[e_config].append(identity)
                        problem_vars = list(set(sum([list(identity.dict.keys()) for identity in more_ids[e_config]],[])))
    if verbose:
        msg = "Simplifying numeric values of qets ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    more_ids = {e_config:list(filter(lambda x: len(x.dict) > 0,list(map(simplify_qet,more_ids[e_config])))) for e_config in more_ids}
    
    if verbose:
        msg = "Solving for dependent vars in terms of independent ones ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    all_sols_2_4 = {irc:[] for irc in more_ids}
    
    for e_config in more_ids:
        problem_vars = list(set(sum([list(identity.dict.keys()) for identity in more_ids[e_config]],[])))
        big_ma = [awe.vec_in_basis(problem_vars) for awe in more_ids[e_config]] 
        big_mat = sp.Matrix(big_ma)
        big_mat = sp.re(big_mat)+sp.I*sp.im(big_mat)
        rref_mat, pivots = big_mat.rref()
        num_rows = rref_mat.rows
        num_cols = rref_mat.cols
        rref_ma_non_zero = [rref_mat[row,:] for row in range(num_rows) if (sum(np.array(rref_mat[row,:])[0] == 0) != num_cols)]
        rref_mat_non_zero = sp.Matrix(rref_ma_non_zero)
        varvec = sp.Matrix(list(map(composite_symbol, problem_vars)))
        eqns = rref_mat_non_zero*varvec
        ssol = sp.solve(list(eqns), dict=True)
        all_sols_2_4[e_config] = ssol
        assert len(ssol) in [0,1]
        if len(ssol) == 0:
            sol_dict = {}
        else:
            sol_dict = ssol[0]
        sol_dict = {twotuplerecovery(k):{twotuplerecovery(s):v.coeff(s) for s in v.free_symbols} for k,v in sol_dict.items()}
        all_sols_2_4[e_config] = sol_dict

    # flatten into a single dictionary of replacements
    fab_solution_2_4 = {}
    for e_config in all_sols_2_4:
        fab_solution_2_4.update(all_sols_2_4[e_config])
    if verbose:
        msg = "There are %d less independent variables ..."
        pbar.set_description("There are %d less independent variables ..." % len(fab_solution_2_4))
        pbar.refresh()
        pbar.n += 1
    
    def simplifier_f(qet):
        true_qet = Qet({})
        for k,v in qet.dict.items():
            if k in fab_solution_2_4:
                true_qet += v*Qet(fab_solution_2_4[k])
            else:
                true_qet += Qet({k:v})
        return true_qet
    
    def simplify_config_matrix_f(ir1ir2, S, ir3):
        config_matrix = simple_config_matrices[ir1ir2][(S,ir3)]
        num_rows = len(config_matrix)
        simple_matrix = [[simplifier_f(config_matrix[row][col]) for col in range(num_rows)] for row in range(num_rows)]
        #simple_matrix = sp.Matrix(simple_matrix)
        return simple_matrix
    
    if verbose:
        msg = "Making final simplifications ..."
        pbar.set_description(msg)
        pbar.refresh()
        pbar.n += 1
    
    final_config_matrices = {k:{} for k in config_matrices}
    for ir1ir2 in config_matrices.keys():
        for term_key in config_matrices[ir1ir2]:
            S, Γ3 = term_key
            final_config_matrices[ir1ir2][term_key] = simplify_config_matrix_f(ir1ir2, S, Γ3)
    
    if verbose:
        msg = "Finished"
        pbar.set_description(msg)
        pbar.refresh()
    
    return final_config_matrices

In [18]:
all_term_energies = {}
for group_label in CPGs.all_group_labels:
    if group_label in all_term_energies:
        continue
    final_config_matrices = term_energies(group_label)
#     print("Printing ...")
#     for counter, ir1ir2 in enumerate(final_config_matrices):
#         print('-'*10)
#         display(ir1ir2)
#         for term_key in final_config_matrices[ir1ir2]:
#             display(term_key)
#             display(sp.Matrix([[el.as_braket() for el in row] for row in final_config_matrices[ir1ir2][term_key]]))
#     print("Saving ...")
    all_term_energies[group_label] = final_config_matrices

(0/19) [00:00] 

(0/19) [00:00] 

(0/19) [00:00] 

(0/19) [00:00] 

(0/19) [00:00] 

...

(0/19) [00:00] 

(0/19) [00:00] 

(0/19) [00:00] 

(0/19) [00:00] 

(0/19) [00:00] 

In [19]:
def simplify_qet_values(qet):
    sqetdict = {k:sp.simplify(v) for k,v in qet.dict.items()}
    return Qet(sqetdict)

In [20]:
term_energies_for_printout = {}
for group_label, term_energies in all_term_energies.items():
    for e_config, terms  in term_energies.items():
        for term_key, term_matrix in terms.items():
            term_energy = term_matrix[0][0]
            term_energy = simplify_qet_values(term_energy)
            term_energies_for_printout[(group_label, e_config, term_key)] = term_energy
#             out = sp.simplify(term_energy.as_braket())
#             if len(sp.latex(out))> 200:
#                 if 'sqrt' in sp.latex(out) and 'i' in sp.latex(out):
#                     display(term_key)
#                     display(sp.simplify(term_energy.as_braket()))

In [21]:
pickle.dump(term_energies_for_printout, open('./data/term_energies_for_print.pkl','wb'))

In [9]:
!beep

#### Single group

In [36]:
group_label = 'O_{h}'
terms = twoe_terms[group_label]
group = CPGs.get_group_by_label(group_label)
component_labels = {k:list(v.values()) for k,v in new_labels[group_label].items()}

In [37]:
# for a given set of terms
# split all the enclosed states
# in groups of electron configurations
configs = {}
config_supplement = {}
for term_key, term in terms.items():
    for state_key, state in zip(term.state_keys, term.states):
        (Γ1, Γ2, Γ3, γ3, S, mSz) = state_key
        α = sp.Symbol(sp.latex(Γ1).lower())*sp.Symbol(sp.latex(Γ2).lower())
        if α not in configs.keys():
            configs[α] = {}
            config_supplement[α] = {}
        if (S,Γ3) not in configs[α].keys():
            configs[α][(S,Γ3)] = []
            config_supplement[α][(S,Γ3)] = []
        configs[α][(S,Γ3)].append(state)
        config_supplement[α][(S,Γ3)].append(state_key)

In [38]:
def overlinesqueegee(s):
    '''
    Going back from the overline shorthand for spin down,
    acting on a single set of quantum numbers.
    '''
    if 'overline' in str(s):
        spin = -sp.S(1)/2
        comp = sp.Symbol(str(s).replace('\\overline{','')[:-1])
    else:
        spin = sp.S(1)/2
        comp = s
    return (comp, spin)
def spin_restoration(qet):
    '''
    Going back from the overline shorthand for spin down,
    acting on a qet.
    '''
    new_dict = {}
    for k,v in qet.dict.items():
        k = (*overlinesqueegee(k[0]),*overlinesqueegee(k[1]))
        new_dict[k] = v
    return Qet(new_dict)

# within each configuration
config_matrices = {}
for config in configs.keys():
    # display(config)
    # within each term
    term_matrices = {}
    for term_key, states in configs[config].items():
        # find the brackets between all possible pairs of states
        ham_matrix = []
        for state0 in states:
            ham_row = []
            state0 = spin_restoration(state0)
            for state1 in states:
                # recover the spin for the determinantal states
                pstate = state1
                state1 = spin_restoration(state1)
                # determine the det braket between these two states (an operator in between is assumed and omitted)
                detbraket = state0*state1
                # simplify this braket of detstates into brakets between regular states
                regbraket = Qet({})
                for k,v in detbraket.dict.items():
                    γ1, m1, γ2, m2, γ1p, m1p, γ2p, m2p = k
    #                 k0 = (γ1, m1, γ2, m2, γ1p, m1p, γ2p, m2p)
    #                 k1 = (γ1, m1, γ2, m2, γ2p, m2p, γ1p, m1p)
                    k0 = (γ1, γ2, γ1p, γ2p)
                    k1 = (γ1, γ2, γ2p, γ1p)
                    # enforce orthogonality wrt. spin
                    if (m1 == m1p) and (m2 == m2p):
                        qplus = Qet({k0:v}) 
                    else:
                        qplus = Qet({})
                    if (m1 == m2p) and (m2 == m1p):
                        qminus = Qet({k1:-v}) 
                    else:
                        qminus = Qet({})
                    regbraket += (qplus+qminus)
                ham_row.append(regbraket)
            ham_matrix.append(ham_row)
#         ham_matrix = sp.Matrix(ham_matrix)
        term_matrices[term_key] = ham_matrix
    config_matrices[config] = term_matrices

In [39]:
# def simplify_to_J_and_K(qet):
#     '''
#     This function takes a regular braket and simplifies the
#     expression in terms of J's and K's and coefficients it
#     also applies the JK simplification
#     '''
#     tot = sp.S(0)
#     for k,v in qet.dict.items():
#         γ1, γ2, γ3, γ4 = k
#         if (γ1 == γ4 and γ2 == γ3):
#             tot += v*sp.Symbol('K(%s,%s)' % (sp.latex(γ1),sp.latex(γ2)))
#         elif (γ1 == γ2 == γ3 == γ4):
#             tot += v*sp.Symbol('J(%s,%s)' % (sp.latex(γ1),sp.latex(γ2)))
#         elif (γ1 == γ2 and γ3 == γ4):
#             tot += v*sp.Symbol('J(%s,%s)' % (sp.latex(γ1),sp.latex(γ3)))
#     return tot

<span style="color:lime">
2. Many of the brakets in these matrices are related to one another. Use the generators of the group to find the smallest set of independent integrals needed in order to evaluate them. This requires considering equivalences coming from the symmetries of four symbol brakets, and also two symbol brakets. The four symbol brakets would apply to the elements of the configuration matrices, while the two symbol brakets would relate elements within these matrices.
</span>

In [40]:
def composite_symbol(x):
    return sp.Symbol('(%s)'%(','.join(list(map(sp.latex,x)))))
def fourtuplerecovery(ft):
    '''the inverse of composite_symbol'''
    return tuple(map(sp.Symbol,sp.latex(ft)[1:-1].split(',')))

In [41]:
print("Creating all 4-symbol identities using the generators of the group ...")

# this   integral_identities  dictionary  will  have  as  keys
# 4-tuples  of  irreps  and  its  values  will  be lists whose
# elements  are  2-tuples whose first elements are 4-tuples of
# irrep  components  and  whose values are qets whose keys are
# 4-tuples  of  irrep components and whose values are numeric.
# These 4-tuples represent a braket with the Coulomb repulsion
# operator in between.

integral_identities = {}
ir_mats = group.irrep_matrices
for ir1, ir2, ir3, ir4 in product(*([group.irrep_labels]*4)):
    # To simplify calculations this part
    # may only be done over quadruples in a standard
    # order.
    # Whatever is missed here is then brough back in
    # by the reality relations.
    altorder1 = (ir3, ir2, ir1, ir4)
    altorder2 = (ir1, ir4, ir3, ir2)
    altorder3 = (ir3, ir4, ir1, ir2)
    if (altorder1 in integral_identities) or\
       (altorder2 in integral_identities) or\
       (altorder3 in integral_identities):
        continue
    integral_identity_sector = []
    components = [component_labels[ir] for ir in [ir1, ir2, ir3, ir4]]
    comp_to_idx = [{c: idx for idx, c in enumerate(component_labels[ir])} for ir in [ir1, ir2, ir3, ir4]]
    for R in group.generators + [sp.Symbol('{Ee}')]:
        R_id = {}
        for γ1, γ2, γ3, γ4 in product(*components):
            for γ1p, γ2p, γ3p, γ4p in product(*components):
                    val = sp.conjugate(ir_mats[ir1][R][comp_to_idx[0][γ1],comp_to_idx[0][γ1p]]) *\
                          sp.conjugate(ir_mats[ir2][R][comp_to_idx[1][γ2],comp_to_idx[1][γ2p]]) *\
                          ir_mats[ir3][R][comp_to_idx[2][γ3],comp_to_idx[2][γ3p]] *\
                          ir_mats[ir4][R][comp_to_idx[3][γ4],comp_to_idx[3][γ4p]]
                    if val== 0:
                        continue
                    key = (γ1, γ2, γ3, γ4)
                    if key not in R_id.keys():
                        R_id[key] = []
                    R_id[key].append( Qet({(γ1p,γ2p,γ3p,γ4p): val}) )
        R_id_total = [(key, sum(R_id[key], Qet({}))) for key in R_id.keys()]
        R_id_total = [q for q in R_id_total if len(q[1].dict)>0]
        integral_identity_sector.extend(R_id_total)
    integral_identities[(ir1,ir2,ir3,ir4)] = integral_identity_sector

# For solving the linear system it is convenient
# to have everything on one side of the equation.

print("Creating set of identities ...")
identities = {}
for ircombo in integral_identities:
    these_ids = []
    for v in integral_identities[ircombo]:
        lhs, rhs = v
        diff = Qet({lhs:1}) - rhs
        if len(diff.dict) > 0:
            these_ids.append(diff)
    identities[ircombo] = these_ids

# If an equation has only one key, then
# that immediately means that that braket is zero.

print("Finding trivial zeros ...")
# first determine which ones have to be zero
better_identities = {irc:[] for irc in identities}
all_zeros = {}
for ircombo in identities:
    zeros = []
    for identity in identities[ircombo]:
        if len(identity.dict) == 1:
            zeros.append((list(identity.dict.keys())[0]))
        else:
            better_identities[ircombo].append(identity)
    all_zeros[ircombo] = zeros

# use them to simplify things.    
print("Using them to simplify things ...")
great_identities = {irc:[] for irc in identities}
for ircombo in better_identities:
    for identity in better_identities[ircombo]:
        new_qet = Qet({})
        for k,v in identity.dict.items():
            if k in all_zeros[ircombo]:
                continue
            else:
                new_qet+= Qet({k: v})
        if len(new_qet.dict) == 0:
            continue
        great_identities[ircombo].append(new_qet)

# Inside of a four-symbol braket one may do three exchanges
# that must result in the same value. That if the wave
# functions are assumed to be real-valued.
print("Creating reality identities ...")
real_var_simplifiers = {irc:[] for irc in great_identities}
kprimes = set()
# this has to run over all the quadruples of irs
for ir1, ir2, ir3, ir4 in product(*([group.irrep_labels]*4)):
    real_var_simplifier = {}
    ircombo = (ir1, ir2, ir3, ir4)
    components = [component_labels[ir] for ir in ircombo]
    for γ1, γ2, γ3, γ4 in product(*components):
        k = (γ1, γ2, γ3, γ4)
        kalt1 = (γ3, γ2, γ1, γ4)
        kalt2 = (γ1, γ4, γ3, γ2)
        kalt3 = (γ3, γ4, γ1, γ2)
        if kalt1 in kprimes:
            real_var_simplifier[k] = kalt1
        elif kalt2 in kprimes:
            real_var_simplifier[k] = kalt2
        elif kalt3 in kprimes:
            real_var_simplifier[k] = kalt3
        else:
            real_var_simplifier[k] = k
            kprimes.add(k)
    real_var_simplifiers[ircombo] = real_var_simplifier

real_var_full_simplifier = {}
for ircombo in real_var_simplifiers:
    real_var_full_simplifier.update(real_var_simplifiers[ircombo])

Creating all 4-symbol identities using the generators of the group ...
Creating set of identities ...
Finding trivial zeros ...
Using them to simplify things ...
Creating reality identities ...


In [42]:
!beep

In [44]:
# For each 4-tuple of irreps
# create a system of symbolic solutions
# and let sympy solve that.
# For each 4-tuple of irreps
# the end result is a dictionary
# whose keys represent the dependent
# brakets and whose values are the
# relation that those dependent values
# have with the independent brakets.
# As such, when these dictionaries are
# used on an expression, everything should
# then be given in terms of indepedent brakets.

print("Solving all ...")
all_sols = {irc:[] for irc in great_identities}
for ircombo in great_identities:
    zeros = all_zeros[ircombo]
    problem_vars = list(set(sum([list(identity.dict.keys()) for identity in great_identities[ircombo]],[])))
    big_ma = [awe.vec_in_basis(problem_vars) for awe in great_identities[ircombo]] 
    big_mat = sp.Matrix(big_ma)
    rref_mat, pivots = big_mat.rref()
    num_rows = rref_mat.rows
    num_cols = rref_mat.cols
    rref_ma_non_zero = [rref_mat[row,:] for row in range(num_rows) if (sum(np.array(rref_mat[row,:])[0] == 0) != num_cols)]
    rref_mat_non_zero = sp.Matrix(rref_ma_non_zero)
    varvec = sp.Matrix(list(map(composite_symbol,problem_vars)))
    eqns = rref_mat_non_zero*varvec
    ssol = sp.solve(list(eqns), dict=True)
    all_sols[ircombo] = ssol
    assert len(ssol) in [0,1]
    if len(ssol) == 0:
        sol_dict = {}
    else:
        sol_dict = ssol[0]
    zero_addendum = {k:{} for k in all_zeros[ircombo]}
    sol_dict = {fourtuplerecovery(k):{fourtuplerecovery(s):v.coeff(s) for s in v.free_symbols} for k,v in sol_dict.items()}
    sol_dict.update(zero_addendum)
    all_sols[ircombo] = sol_dict

# This final dictionary is unnecessary but
# simplifies calling the replacements onto
# a symbolic expression.

print("Creating a dictionary with all the 4-symbol replacements ...")
super_solution = {}
for ircombo in all_sols:
    super_solution.update(all_sols[ircombo])

def simplifier(qet):
    simp_ket = Qet({})
    for k,v in qet.dict.items():
        simp_ket += Qet({real_var_full_simplifier[k]:v})
    true_qet = Qet({})
    for k,v in simp_ket.dict.items():
        if k in super_solution:
            true_qet += v*Qet(super_solution[k])
        else:
            true_qet += Qet({k:v})
    return true_qet#.as_braket()
def simplify_config_matrix(ir1ir2, S, ir3):
    config_matrix = config_matrices[ir1ir2][(S,ir3)]
    num_rows = len(config_matrix)
    simple_matrix = [[simplifier(config_matrix[row][col]) for col in range(num_rows)] for row in range(num_rows)]
    #simple_matrix = sp.Matrix(simple_matrix)
    return simple_matrix

print("Simplifying configuration matrices ...")
simple_config_matrices = {k:{} for k in config_matrices}
for ir1ir2 in config_matrices.keys():
    for term_key in config_matrices[ir1ir2]:
        S, Γ3 = term_key
        simple_config_matrices[ir1ir2][term_key] = simplify_config_matrix(ir1ir2, S, Γ3)

Solving all ...
Creating a dictionary with all the 4-symbol replacements ...
Simplifying configuration matrices ...


In [45]:
!beep

In [47]:
twotuplerecovery = fourtuplerecovery

print("Creating all 2-symbol identities using the generators of the group ...")

# this   integral_identities  dictionary  will  have  as  keys
# 2-tuples  of  irreps  and  its  values  will  be lists whose
# elements  are  2-tuples whose first elements are 2-tuples of
# irrep  components  and  whose values are qets whose keys are
# 2-tuples  of  irrep components and whose values are numeric.
# These 2-tuples represent a braket with the an invariant operator
# operator in between.

integral_identities_2 = {}
ir_mats = group.irrep_matrices
for ir1, ir2 in product(*([group.irrep_labels]*2)):
    # To simplify calculations this part
    # can only be done over quadruples in a standard
    # order.
    # Whatever is missed here is then brough back in
    # by the reality relations.
    altorder1 = (ir2, ir1, ir1, ir4)
    if (altorder1 in integral_identities_2):
        continue
    integral_identity_sector = []
    components = [component_labels[ir] for ir in [ir1, ir2]]
    comp_to_idx = [{c: idx for idx, c in enumerate(component_labels[ir])} for ir in [ir1, ir2]]
    for R in group.generators + [sp.Symbol('{Ee}')]:
        R_id = {}
        for γ1, γ2 in product(*components):
            for γ1p, γ2p in product(*components):
                    val = sp.conjugate(ir_mats[ir1][R][comp_to_idx[0][γ1],comp_to_idx[0][γ1p]]) *\
                          ir_mats[ir2][R][comp_to_idx[1][γ2],comp_to_idx[1][γ2p]]
                    if val== 0:
                        continue
                    key = (γ1, γ2)
                    if key not in R_id.keys():
                        R_id[key] = []
                    R_id[key].append( Qet({(γ1p,γ2p): val}) )
        R_id_total = [(key, sum(R_id[key], Qet({}))) for key in R_id.keys()]
        R_id_total = [q for q in R_id_total if len(q[1].dict)>0]
        integral_identity_sector.extend(R_id_total)
    integral_identities_2[(ir1,ir2)] = integral_identity_sector

# For solving the linear system it is convenient
# to have everything on one side of the equation.

print("Creating set of identities ...")
identities_2 = {}
for ircombo in integral_identities_2:
    these_ids = []
    for v in integral_identities_2[ircombo]:
        lhs, rhs = v
        diff = Qet({lhs:1}) - rhs
        if len(diff.dict) > 0:
            these_ids.append(diff)
    identities_2[ircombo] = these_ids
    
# If an equation has only one term, then
# that immediately means that that term is zero.

print("Finding trivial zeros ...")
# first determine which ones have to be zero
better_identities_2 = {irc:[] for irc in identities_2}
all_zeros_2 = {}
for ircombo in identities_2:
    zeros = []
    for identity in identities_2[ircombo]:
        if len(identity.dict) == 1:
            zeros.append((list(identity.dict.keys())[0]))
        else:
            better_identities_2[ircombo].append(identity)
    all_zeros_2[ircombo] = zeros

# use them to simplify things.    
print("Using them to simplify things ...")
great_identities_2 = {irc:[] for irc in identities_2}
for ircombo in better_identities_2:
    for identity in better_identities_2[ircombo]:
        new_qet = Qet({})
        for k,v in identity.dict.items():
            if k in all_zeros_2[ircombo]:
                continue
            else:
                new_qet+= Qet({k: v})
        if len(new_qet.dict) == 0:
            continue
        great_identities_2[ircombo].append(new_qet)

# Inside of a two-symbol braket one may do three exchanges
# that must result in the same value. That if the wave
# functions are assumed to be real-valued.

print("Creating reality identities ...")
real_var_simplifiers_2 = {irc:[] for irc in great_identities_2}
# this has to run over all the quadruples of irs
kprimes = set()
for ir1, ir2 in product(*([group.irrep_labels]*2)):
    real_var_simplifier = {}
    ircombo = (ir1, ir2)
    components = [component_labels[ir] for ir in ircombo]
    for γ1, γ2 in product(*components):
        k = (γ1, γ2)
        kalt = (γ2, γ1)
        # If for a given key I find that its
        # switched version has already been seen
        # Then that key has to be mapped to be
        # mapped to the key already present.
        if kalt in kprimes:
            real_var_simplifier[k] = kalt
        else:
            real_var_simplifier[k] = k
            kprimes.add(k)
    real_var_simplifiers_2[ircombo] = real_var_simplifier

Creating all 2-symbol identities using the generators of the group ...
Creating set of identities ...
Finding trivial zeros ...
Using them to simplify things ...
Creating reality identities ...


In [49]:
# For each 2-tuple of irreps
# create a system of symbolic solutions
# and let sympy solve that.
# For each 2-tuple of irreps
# the end result is a dictionary
# whose keys represent the dependent
# brakets and whose values are the
# relation that those dependent values
# have with the independent brakets.
# As such, when these dictionaries are
# used on an expression, everything should
# then be given in terms of indepedent brakets.

print("Solving all ...")
all_sols_2 = {irc:[] for irc in great_identities_2}
for ircombo in great_identities_2:
    zeros = all_zeros_2[ircombo]
    problem_vars = list(set(sum([list(identity.dict.keys()) for identity in great_identities_2[ircombo]],[])))
    big_ma = [awe.vec_in_basis(problem_vars) for awe in great_identities_2[ircombo]] 
    big_mat = sp.Matrix(big_ma)
    rref_mat, pivots = big_mat.rref()
    num_rows = rref_mat.rows
    num_cols = rref_mat.cols
    rref_ma_non_zero = [rref_mat[row,:] for row in range(num_rows) if (sum(np.array(rref_mat[row,:])[0] == 0) != num_cols)]
    rref_mat_non_zero = sp.Matrix(rref_ma_non_zero)
    varvec = sp.Matrix(list(map(composite_symbol,problem_vars)))
    eqns = rref_mat_non_zero*varvec
    ssol = sp.solve(list(eqns), dict=True)
    all_sols[ircombo] = ssol
    assert len(ssol) in [0,1]
    if len(ssol) == 0:
        sol_dict = {}
    else:
        sol_dict = ssol[0]
    zero_addendum = {k:{} for k in all_zeros_2[ircombo]}
    sol_dict = {twotuplerecovery(k):{twotuplerecovery(s):v.coeff(s) for s in v.free_symbols} for k,v in sol_dict.items()}
    sol_dict.update(zero_addendum)
    all_sols_2[ircombo] = sol_dict

Solving all ...


In [51]:
# This final dictionary is unnecessary but
# simplifies calling the replacements onto
# a symbolic expression.

print("Creating a dictionary with all the 2-symbol replacements ...")
super_solution_2 = {}
for ircombo in all_sols_2:
    super_solution_2.update(all_sols_2[ircombo])
    
# for a given electron config
more_ids = {e_config:[] for e_config in simple_config_matrices}
for e_config in simple_config_matrices:
    for term in simple_config_matrices[e_config]:
        the_matrix = simple_config_matrices[e_config][term]
        num_rows = len(the_matrix)
        num_cols = len(the_matrix)
        the_state_keys = config_supplement[e_config][term]
        the_key_matrix = {}
        for row_idx in range(num_rows):
            Γ3_row = the_state_keys[row_idx][2]
            γ3_row = the_state_keys[row_idx][3]
            for col_idx in range(num_cols):
                Γ3_col = the_state_keys[col_idx][2]
                γ3_col = the_state_keys[col_idx][3]
                matrix_val = the_matrix[row_idx][col_idx]
                the_key_matrix[(γ3_row,γ3_col)] = matrix_val
        # now go over the keys of super_solution_2
        # and if one of those keys matches with a key in the_key_matrix
        # do something about it
        for k in super_solution_2:
            if k in the_key_matrix:
                v = super_solution_2[k]
                # this matrix element
                matrix_element = the_key_matrix[k]
                # must be identified with the sum
                # as given in v
                matrix_equiv = sum([kv*the_key_matrix[km] for km, kv in v.items()],Qet({}))
                identity = (matrix_element - matrix_equiv) #=0
                if len(identity.dict) != 0:
                    more_ids[e_config].append(identity)
                    problem_vars = list(set(sum([list(identity.dict.keys()) for identity in more_ids[e_config]],[])))

Creating a dictionary with all the 2-symbol replacements ...


In [55]:
for e_config in more_ids:
    if len(more_ids[e_config]) > 1:
        break

In [71]:
def simplify_qet(qet):
    new_dict = {k:sp.simplify(v) for k,v in qet.dict.items()}
    return Qet(new_dict)

In [72]:
list(map(simplify_qet,more_ids[e_config]))

[Qet({}),
 Qet({({\nu}_{g}, {\nu}_{g}, {\nu}_{g}, {\nu}_{g}): -1/2 + sqrt(3)*I/2, ({\nu}_{g}, {\nu}_{g}, {\mu}_{g}, {\mu}_{g}): 1/2 - sqrt(3)*I/2}),
 Qet({({\nu}_{g}, {\nu}_{g}, {\nu}_{g}, {\nu}_{g}): -1/2 + sqrt(3)*I/2, ({\nu}_{g}, {\nu}_{g}, {\mu}_{g}, {\mu}_{g}): 1/2 - sqrt(3)*I/2}),
 Qet({})]

In [73]:
uber_ids = {e_config:list(filter(lambda x: len(x.dict) > 0,list(map(simplify_qet,more_ids[e_config])))) for e_config in more_ids}

TODO:

- add simplifcation of numeric values of qets in more_ids
- add complex simplification of big_mat

In [ ]:
print("Simplifying numeric values of qets ...")
more_ids = {e_config:list(filter(lambda x: len(x.dict) > 0,list(map(simplify_qet,more_ids[e_config])))) for e_config in more_ids}

In [96]:
big_mat = sp.Matrix(big_ma)
big_mat = sp.re(big_mat) + sp.I*sp.im(big_mat)
rref_mat, pivots = big_mat.rref()
num_rows = rref_mat.rows
num_cols = rref_mat.cols

In [97]:
print("Solving for dependent vars in terms of independent ones ...")
all_sols_2_4 = {irc:[] for irc in more_ids}
for e_config in more_ids:
    print(e_config)
    problem_vars = list(set(sum([list(identity.dict.keys()) for identity in more_ids[e_config]],[])))
    big_ma = [awe.vec_in_basis(problem_vars) for awe in more_ids[e_config]] 
    big_mat = sp.Matrix(big_ma)
    big_mat = sp.re(big_mat) + sp.I*sp.im(big_mat)
    rref_mat, pivots = big_mat.rref()
    num_rows = rref_mat.rows
    num_cols = rref_mat.cols
    rref_ma_non_zero = [rref_mat[row,:] for row in range(num_rows) if (sum(np.array(rref_mat[row,:])[0] == 0) != num_cols)]
    rref_mat_non_zero = sp.Matrix(rref_ma_non_zero)
    varvec = sp.Matrix(list(map(composite_symbol, problem_vars)))
    eqns = rref_mat_non_zero*varvec
    ssol = sp.solve(list(eqns), dict=True)
    all_sols_2_4[e_config] = ssol
    assert len(ssol) in [0,1]
    if len(ssol) == 0:
        sol_dict = {}
    else:
        sol_dict = ssol[0]
    sol_dict = {twotuplerecovery(k):{twotuplerecovery(s):v.coeff(s) for s in v.free_symbols} for k,v in sol_dict.items()}
    all_sols_2_4[e_config] = sol_dict

Solving for dependent vars in terms of independent ones ...
a_{{1g}}**2
a_{{2u}}**2
a_{{1u}}**2
a_{{2g}}**2
e_{u}**2
e_{g}**2
t_{{1g}}**2
t_{{2g}}**2
t_{{1u}}**2
t_{{2u}}**2
a_{{1g}}*a_{{2u}}
a_{{1u}}*a_{{2g}}
e_{g}*e_{u}
t_{{1g}}*t_{{2u}}
t_{{1u}}*t_{{2g}}
a_{{1g}}*a_{{1u}}
a_{{2g}}*a_{{2u}}
t_{{1g}}*t_{{1u}}
t_{{2g}}*t_{{2u}}
a_{{1g}}*a_{{2g}}
a_{{1u}}*a_{{2u}}
t_{{1g}}*t_{{2g}}
t_{{1u}}*t_{{2u}}
a_{{1g}}*e_{u}
a_{{2u}}*e_{g}
a_{{1u}}*e_{g}
a_{{2g}}*e_{u}
a_{{1g}}*e_{g}
a_{{2u}}*e_{u}
a_{{1u}}*e_{u}
a_{{2g}}*e_{g}
a_{{1g}}*t_{{1g}}
a_{{2u}}*t_{{2u}}
a_{{1u}}*t_{{1u}}
a_{{2g}}*t_{{2g}}
e_{u}*t_{{1u}}
e_{u}*t_{{2u}}
e_{g}*t_{{1g}}
e_{g}*t_{{2g}}
a_{{1g}}*t_{{2g}}
a_{{2u}}*t_{{1u}}
a_{{1u}}*t_{{2u}}
a_{{2g}}*t_{{1g}}
a_{{1g}}*t_{{1u}}
a_{{2u}}*t_{{2g}}
a_{{1u}}*t_{{1g}}
a_{{2g}}*t_{{2u}}
e_{u}*t_{{1g}}
e_{u}*t_{{2g}}
e_{g}*t_{{1u}}
e_{g}*t_{{2u}}
a_{{1g}}*t_{{2u}}
a_{{2u}}*t_{{1g}}
a_{{1u}}*t_{{2g}}
a_{{2g}}*t_{{1u}}


In [ ]:
# flatten into a single dictionary of replacements
fab_solution_2_4 = {}
for e_config in all_sols_2_4:
    fab_solution_2_4.update(all_sols_2_4[e_config])

print("There are %d less independent variables ..." % len(fab_solution_2_4))    

def simplifier_f(qet):
    true_qet = Qet({})
    for k,v in qet.dict.items():
        if k in fab_solution_2_4:
            true_qet += v*Qet(fab_solution_2_4[k])
        else:
            true_qet += Qet({k:v})
    return true_qet#.as_braket()
def simplify_config_matrix_f(ir1ir2, S, ir3):
    config_matrix = simple_config_matrices[ir1ir2][(S,ir3)]
    num_rows = len(config_matrix)
    simple_matrix = [[simplifier_f(config_matrix[row][col]) for col in range(num_rows)] for row in range(num_rows)]
    #simple_matrix = sp.Matrix(simple_matrix)
    return simple_matrix

print("Making final simplifications ...")
final_config_matrices = {k:{} for k in config_matrices}
for ir1ir2 in config_matrices.keys():
    for term_key in config_matrices[ir1ir2]:
        S, Γ3 = term_key
        final_config_matrices[ir1ir2][term_key] = simplify_config_matrix_f(ir1ir2, S, Γ3)

print("Printing ...")
for counter, ir1ir2 in enumerate(final_config_matrices):
    print('-'*10)
    display(ir1ir2)
    for term_key in final_config_matrices[ir1ir2]:
        display(term_key)
        display(sp.Matrix([[el.as_braket() for el in row] for row in final_config_matrices[ir1ir2][term_key]]))

In [ ]:
!beep

In [26]:
set(sum([[el.as_braket() for el in row if el.as_braket()!=0] for row in final_config_matrices[ir1ir2][term_key]],[]))

{-3*<{\zeta}{\zeta}|{\gamma}{\gamma}> + <{\zeta}{\zeta}|{\zeta}{\zeta}>}

In [27]:
print("Counting ...")
for counter, ir1ir2 in enumerate(final_config_matrices):
    print('-'*10)
    display(ir1ir2)
    for term_key in final_config_matrices[ir1ir2]:
        display(term_key)
        num_different = len(set(sum([[el.as_braket() for el in row if el.as_braket()!=0] for row in final_config_matrices[ir1ir2][term_key]],[])))
        display(num_different)
        display(sp.Matrix([[el.as_braket() for el in row] for row in final_config_matrices[ir1ir2][term_key]]))

Counting ...
----------


a_{1}**2

(0, A_1)

1

Matrix([[<{\alpha}{\alpha}|{\alpha}{\alpha}>]])

...

1

Matrix([
[-<{\beta}{\beta}|{\nu}{\nu}> + <{\beta}{\nu}|{\beta}{\nu}>,                                                          0,                                                          0,                                                          0,                                                          0,                                                          0,                                                          0,                                                          0,                                                          0],
[                                                         0, -<{\beta}{\beta}|{\nu}{\nu}> + <{\beta}{\nu}|{\beta}{\nu}>,                                                          0,                                                          0,                                                          0,                                                          0,                                                          0,                             

(0, T_2)

1

Matrix([
[<{\beta}{\beta}|{\nu}{\nu}> + <{\beta}{\nu}|{\beta}{\nu}>,                                                         0,                                                         0],
[                                                        0, <{\beta}{\beta}|{\nu}{\nu}> + <{\beta}{\nu}|{\beta}{\nu}>,                                                         0],
[                                                        0,                                                         0, <{\beta}{\beta}|{\nu}{\nu}> + <{\beta}{\nu}|{\beta}{\nu}>]])

In [ ]:
!beep

In [1607]:
# # test it out on a single element of one of the matrices
# brak = config_matrices[sp.Symbol('t_{2}')*sp.Symbol('t_{2}')][(0,sp.Symbol('A_1'))][0][0]
# simp_ket = Qet({})
# for k,v in brak.dict.items():
#     simp_ket += Qet({real_var_full_simplifier[k]:v})
# true_qet = Qet({})
# for k,v in simp_ket.dict.items():
#     if k in super_solution:
#         true_qet += v*Qet(super_solution[k])
#     else:
#         true_qet += Qet({k:v})
# true_qet.as_braket()

In [1611]:
# list(simple_config_matrices[sp.Symbol('e')*sp.Symbol('t_{2}')][(0,sp.Symbol('T_2'))])[0][0].as_braket()
# config_matrices[sp.Symbol('e')*sp.Symbol('t_{2}')][(0,sp.Symbol('T_2'))][0][0].as_braket()

In [ ]:
# # print them out
# for ir1ir2 in simple_config_matrices.keys():
#     print('-'*10)
#     display(ir1ir2)
#     for term_key in simple_config_matrices[ir1ir2]:
#         display(term_key)
#         display(simple_config_matrices[ir1ir2][term_key])

In [1234]:
# nullspace approach doesn't solve for
# all redundancies
# # eqns = list(rref_mat_non_zero*varvec)
# print("Finding the nullspace ...")
# nullspace = rref_mat_non_zero.nullspace()
# print("Converting into solution dictionary ...")
# normal_nullspace = []
# sols = {}
# for row in nullspace:
#     row = list(row)
#     for element_idx, element in enumerate(row):
#         if element !=0:
#             break
#     normalizer = sp.S(element)
#     if normalizer == 1:
#         sols[problem_vars[element_idx]] = {problem_vars[idx]:row[idx] for idx in range(len(row)) if (idx != element_idx and row[idx]!=0)}
#     else:
#         normal_row = [el/normalizer for el in row]
#         if problem_vars[element_idx] in sols:
#             print("huh?",end='|')
# #             print(problem_vars[element_idx])
# #             print({problem_vars[idx]:normal_row[idx] for idx in range(len(row)) if (idx != element_idx and row[idx]!=0)})
#         sols[problem_vars[element_idx]] = {problem_vars[idx]:normal_row[idx] for idx in range(len(row)) if (idx != element_idx and row[idx]!=0)}

<span style="color:lime">
2.2 --- Two symbol identities.
</span>

<span style="color:red"> JOINT APPROACH
<span>
<span style="color:lime">
2. Many of the brakets in these matrices are related to one another. Use the generators of the group to find the smallest set of independent integrals needed in order to evaluate them. This requires considering equivalences coming from the symmetries of four symbol brakets, and also two symbol brakets. The four symbol brakets would apply to the elements of the configuration matrices, while the two symbol brakets would relate elements within these matrices.
</span>

In [1342]:
def composite_symbol(x):
    return sp.Symbol('(%s)'%(','.join(list(map(sp.latex,x)))))
def fourtuplerecovery(ft):
    '''the inverse of composite_symbol'''
    return tuple(map(sp.Symbol,sp.latex(ft)[1:-1].split(',')))

<span style="color:lime">
2.1 --- Four symbol identities.
</span>

In [1343]:
print("Creating all 4-symbol identities using the generators of the group ...")

# this   integral_identities  dictionary  will  have  as  keys
# 4-tuples  of  irreps  and  its  values  will  be lists whose
# elements  are  2-tuples whose first elements are 4-tuples of
# irrep  components  and  whose values are qets whose keys are
# 4-tuples  of  irrep components and whose values are numeric.
# These 4-tuples represent a braket with the Coulomb repulsion
# operator in between.

integral_identities = {}
ir_mats = group.irrep_matrices
for ir1, ir2, ir3, ir4 in product(*([group.irrep_labels]*4)):
    # To simplify calculations this part
    # can only be done over quadruples in a standard
    # order.
    # Whatever is missed here is then brough back in
    # by the reality relations.
    altorder1 = (ir3, ir2, ir1, ir4)
    altorder2 = (ir1, ir4, ir3, ir2)
    altorder3 = (ir3, ir4, ir1, ir2)
    if (altorder1 in integral_identities) or\
       (altorder2 in integral_identities) or\
       (altorder3 in integral_identities):
        continue
    integral_identity_sector = []
    components = [component_labels[ir] for ir in [ir1, ir2, ir3, ir4]]
    comp_to_idx = [{c: idx for idx, c in enumerate(component_labels[ir])} for ir in [ir1, ir2, ir3, ir4]]
    for R in group.generators + [sp.Symbol('{Ee}')]:
        R_id = {}
        for γ1, γ2, γ3, γ4 in product(*components):
            for γ1p, γ2p, γ3p, γ4p in product(*components):
                    val = sp.conjugate(ir_mats[ir1][R][comp_to_idx[0][γ1],comp_to_idx[0][γ1p]]) *\
                          sp.conjugate(ir_mats[ir2][R][comp_to_idx[1][γ2],comp_to_idx[1][γ2p]]) *\
                          ir_mats[ir3][R][comp_to_idx[2][γ3],comp_to_idx[2][γ3p]] *\
                          ir_mats[ir4][R][comp_to_idx[3][γ4],comp_to_idx[3][γ4p]]
                    if val== 0:
                        continue
                    key = (γ1, γ2, γ3, γ4)
                    if key not in R_id.keys():
                        R_id[key] = []
                    R_id[key].append( Qet({(γ1p,γ2p,γ3p,γ4p): val}) )
        R_id_total = [(key, sum(R_id[key], Qet({}))) for key in R_id.keys()]
        R_id_total = [q for q in R_id_total if len(q[1].dict)>0]
        integral_identity_sector.extend(R_id_total)
    integral_identities[(ir1,ir2,ir3,ir4)] = integral_identity_sector

# For solving the linear system it is convenient
# to have everything on one side of the equation.

print("Creating set of identities ...")
identities = {}
for ircombo in integral_identities:
    these_ids = []
    for v in integral_identities[ircombo]:
        lhs, rhs = v
        diff = Qet({lhs:1}) - rhs
        if len(diff.dict) > 0:
            these_ids.append(diff)
    identities[ircombo] = these_ids

# If an equation has only one term, then
# that immediately means that that term is zero.

print("Finding trivial zeros ...")
# first determine which ones have to be zero
better_identities = {irc:[] for irc in identities}
all_zeros = {}
for ircombo in identities:
    zeros = []
    for identity in identities[ircombo]:
        if len(identity.dict) == 1:
            zeros.append((list(identity.dict.keys())[0]))
        else:
            better_identities[ircombo].append(identity)
    all_zeros[ircombo] = zeros

# use them to simplify things.    
print("Using them to simplify things...")
great_identities = {irc:[] for irc in identities}
for ircombo in better_identities:
    for identity in better_identities[ircombo]:
        new_qet = Qet({})
        for k,v in identity.dict.items():
            if k in all_zeros[ircombo]:
                continue
            else:
                new_qet+= Qet({k: v})
        if len(new_qet.dict) == 0:
            continue
        great_identities[ircombo].append(new_qet)

# Inside of a four-symbol braket one may three exchanges
# that must result in the same value. That if the wave
# functions are assumed to be real-valued.

print("Creating reality identities ...")
real_var_simplifiers = {irc:[] for irc in great_identities}
kprimes = set()
# this has to run over all the quadruples of irs
for ir1, ir2, ir3, ir4 in product(*([group.irrep_labels]*4)):
    real_var_simplifier = {}
    ircombo = (ir1, ir2, ir3, ir4)
    components = [component_labels[ir] for ir in ircombo]
    for γ1, γ2, γ3, γ4 in product(*components):
        k = (γ1, γ2, γ3, γ4)
        kalt1 = (γ3, γ2, γ1, γ4)
        kalt2 = (γ1, γ4, γ3, γ2)
        kalt3 = (γ3, γ4, γ1, γ2)
        if kalt1 in kprimes:
            real_var_simplifier[kalt2] = kalt1
            real_var_simplifier[kalt3] = kalt1
        elif kalt2 in kprimes:
            real_var_simplifier[kalt1] = kalt2
            real_var_simplifier[kalt3] = kalt2
        elif kalt3 in kprimes:
            real_var_simplifier[kalt1] = kalt3
            real_var_simplifier[kalt2] = kalt3
        else:
            real_var_simplifier[k] = k
            real_var_simplifier[kalt1] = k
            real_var_simplifier[kalt2] = k
            real_var_simplifier[kalt3] = k
            kprimes.add(k)
    real_var_simplifiers[ircombo] = real_var_simplifier

# For each 4-tuple of irreps
# create a system of symbolic solutions
# and let sympy solve that.
# For each 4-tuple of irreps
# the end result is a dictionary
# whose keys represent the dependent
# brakets and whose values are the
# relation that those dependent values
# have with the independent brakets.
# As such, when these dictionaries are
# used on an expression, everything should
# then be given in terms of indepedent brakets.

In [1471]:
twotuplerecovery = fourtuplerecovery

print("Creating all 2-symbol identities using the generators of the group ...")

# this   integral_identities  dictionary  will  have  as  keys
# 2-tuples  of  irreps  and  its  values  will  be lists whose
# elements  are  2-tuples whose first elements are 2-tuples of
# irrep  components  and  whose values are qets whose keys are
# 2-tuples  of  irrep components and whose values are numeric.
# These 2-tuples represent a braket with the an invariant operator
# operator in between.

integral_identities_2 = {}
ir_mats = group.irrep_matrices
for ir1, ir2 in product(*([group.irrep_labels]*2)):
    # To simplify calculations this part
    # can only be done over quadruples in a standard
    # order.
    # Whatever is missed here is then brough back in
    # by the reality relations.
    altorder1 = (ir2, ir1, ir1, ir4)
    if (altorder1 in integral_identities_2):
        continue
    integral_identity_sector = []
    components = [component_labels[ir] for ir in [ir1, ir2]]
    comp_to_idx = [{c: idx for idx, c in enumerate(component_labels[ir])} for ir in [ir1, ir2]]
    for R in group.generators + [sp.Symbol('{Ee}')]:
        R_id = {}
        for γ1, γ2 in product(*components):
            for γ1p, γ2p in product(*components):
                    val = sp.conjugate(ir_mats[ir1][R][comp_to_idx[0][γ1],comp_to_idx[0][γ1p]]) *\
                          ir_mats[ir2][R][comp_to_idx[1][γ2],comp_to_idx[1][γ2p]]
                    if val== 0:
                        continue
                    key = (γ1, γ2)
                    if key not in R_id.keys():
                        R_id[key] = []
                    R_id[key].append( Qet({(γ1p,γ2p): val}) )
        R_id_total = [(key, sum(R_id[key], Qet({}))) for key in R_id.keys()]
        R_id_total = [q for q in R_id_total if len(q[1].dict)>0]
        integral_identity_sector.extend(R_id_total)
    integral_identities_2[(ir1,ir2)] = integral_identity_sector

# For solving the linear system it is convenient
# to have everything on one side of the equation.

print("Creating set of identities ...")
identities_2 = {}
for ircombo in integral_identities_2:
    these_ids = []
    for v in integral_identities_2[ircombo]:
        lhs, rhs = v
        diff = Qet({lhs:1}) - rhs
        if len(diff.dict) > 0:
            these_ids.append(diff)
    identities_2[ircombo] = these_ids
    
# If an equation has only one term, then
# that immediately means that that term is zero.

print("Finding trivial zeros ...")
# first determine which ones have to be zero
better_identities_2 = {irc:[] for irc in identities_2}
all_zeros_2 = {}
for ircombo in identities_2:
    zeros = []
    for identity in identities_2[ircombo]:
        if len(identity.dict) == 1:
            zeros.append((list(identity.dict.keys())[0]))
        else:
            better_identities_2[ircombo].append(identity)
    all_zeros_2[ircombo] = zeros

# use them to simplify things.    
print("Using them to simplify things...")
great_identities_2 = {irc:[] for irc in identities_2}
for ircombo in better_identities_2:
    for identity in better_identities_2[ircombo]:
        new_qet = Qet({})
        for k,v in identity.dict.items():
            if k in all_zeros_2[ircombo]:
                continue
            else:
                new_qet+= Qet({k: v})
        if len(new_qet.dict) == 0:
            continue
        great_identities_2[ircombo].append(new_qet)

# Inside of a four-symbol braket one may three exchanges
# that must result in the same value. That if the wave
# functions are assumed to be real-valued.

print("Creating reality identities ...")
real_var_simplifiers_2 = {irc:[] for irc in great_identities_2}
# this has to run over all the quadruples of irs
kprimes = set()
for ir1, ir2 in product(*([group.irrep_labels]*2)):
    real_var_simplifier = {}
    ircombo = (ir1, ir2)
    components = [component_labels[ir] for ir in ircombo]
    for γ1, γ2 in product(*components):
        k = (γ1, γ2)
        kalt = (γ2, γ1)
        # If for a given key I find that its
        # switched version has already been seen
        # Then that key has to be mapped to be
        # mapped to the key already present.
        if kalt in kprimes:
            real_var_simplifier[k] = kalt
        else:
            real_var_simplifier[k] = k
            kprimes.add(k)
    real_var_simplifiers_2[ircombo] = real_var_simplifier

##############

# For each 4-tuple of irreps
# create a system of symbolic solutions
# and let sympy solve that.
# For each 4-tuple of irreps
# the end result is a dictionary
# whose keys represent the dependent
# brakets and whose values are the
# relation that those dependent values
# have with the independent brakets.
# As such, when these dictionaries are
# used on an expression, everything should
# then be given in terms of indepedent brakets.

print("Solving all...")
all_sols_2 = {irc:[] for irc in great_identities_2}
for ircombo in great_identities_2:
    zeros = all_zeros_2[ircombo]
    problem_vars = list(set(sum([list(identity.dict.keys()) for identity in great_identities_2[ircombo]],[])))
    big_ma = [awe.vec_in_basis(problem_vars) for awe in great_identities_2[ircombo]] 
    big_mat = sp.Matrix(big_ma)
    rref_mat, pivots = big_mat.rref()
    num_rows = rref_mat.rows
    num_cols = rref_mat.cols
    rref_ma_non_zero = [rref_mat[row,:] for row in range(num_rows) if (sum(np.array(rref_mat[row,:])[0] == 0) != num_cols)]
    rref_mat_non_zero = sp.Matrix(rref_ma_non_zero)
    varvec = sp.Matrix(list(map(composite_symbol,problem_vars)))
    eqns = rref_mat_non_zero*varvec
    ssol = sp.solve(list(eqns), dict=True)
    all_sols[ircombo] = ssol
    assert len(ssol) in [0,1]
    if len(ssol) == 0:
        sol_dict = {}
    else:
        sol_dict = ssol[0]
    zero_addendum = {k:{} for k in all_zeros_2[ircombo]}
    sol_dict = {twotuplerecovery(k):{twotuplerecovery(s):v.coeff(s) for s in v.free_symbols} for k,v in sol_dict.items()}
    sol_dict.update(zero_addendum)
    all_sols_2[ircombo] = sol_dict

# This final dictionary is unnecessary but
# simplifies calling the replacements onto
# a symbolic expression.

print("Creating a dictionary with all the 2-symbol replacements ...")
super_solution_2 = {}
for ircombo in all_sols_2:
    super_solution_2.update(all_sols_2[ircombo])

# for a given electron config xxx
more_ids = {ec:[] for ec in config_matrices}
for e_config in simple_config_matrices:
    # for a given term
    for term in config_matrices[e_config]:
#         more_ids[e_config][term] = []
        the_matrix = config_matrices[e_config][term]
        num_rows = len(the_matrix)
        num_cols = len(the_matrix)
        the_state_keys = config_supplement[e_config][term]
        the_key_matrix = {}
        for row_idx in range(num_rows):
            Γ3_row = the_state_keys[row_idx][2]
            γ3_row = the_state_keys[row_idx][3]
            for col_idx in range(num_cols):
                Γ3_col = the_state_keys[col_idx][2]
                γ3_col = the_state_keys[col_idx][3]
                matrix_val = the_matrix[row_idx][col_idx]
                the_key_matrix[(γ3_row,γ3_col)] = matrix_val
        # now go over the keys of super_solution_2
        # and if one of those keys matches with a key in the_key_matrix
        # do something about it
        for k in super_solution_2:
            if k in the_key_matrix:
                v = super_solution_2[k]
                # this matrix element
                matrix_element = the_key_matrix[k]
                # must be identified with the sum
                # as given in v
                matrix_equiv = sum([kv*the_key_matrix[km] for km, kv in v.items()],Qet({}))
                identity = (matrix_element - matrix_equiv) #=0
                if len(identity.dict) != 0:
                    more_ids[e_config].append(identity)

Creating all 2-symbol identities using the generators of the group ...
Creating set of identities ...
Finding trivial zeros ...
Using them to simplify things...
Creating reality identities ...
Solving all...
Creating a dictionary with all the 2-symbol replacements ...


In [ ]:
for e_config in more_ids:

In [1343]:
print("Solving all...")
all_sols = {irc:[] for irc in great_identities}
for ircombo in great_identities:
    zeros = all_zeros[ircombo]
    problem_vars = list(set(sum([list(identity.dict.keys()) for identity in great_identities[ircombo]],[])))
    big_ma = [awe.vec_in_basis(problem_vars) for awe in great_identities[ircombo]] 
    big_mat = sp.Matrix(big_ma)
    rref_mat, pivots = big_mat.rref()
    num_rows = rref_mat.rows
    num_cols = rref_mat.cols
    rref_ma_non_zero = [rref_mat[row,:] for row in range(num_rows) if (sum(np.array(rref_mat[row,:])[0] == 0) != num_cols)]
    rref_mat_non_zero = sp.Matrix(rref_ma_non_zero)
    varvec = sp.Matrix(list(map(composite_symbol,problem_vars)))
    eqns = rref_mat_non_zero*varvec
    ssol = sp.solve(list(eqns), dict=True)
    all_sols[ircombo] = ssol
    assert len(ssol) in [0,1]
    if len(ssol) == 0:
        sol_dict = {}
    else:
        sol_dict = ssol[0]
    zero_addendum = {k:{} for k in all_zeros[ircombo]}
    sol_dict = {fourtuplerecovery(k):{fourtuplerecovery(s):v.coeff(s) for s in v.free_symbols} for k,v in sol_dict.items()}
    sol_dict.update(zero_addendum)
    all_sols[ircombo] = sol_dict

# This final dictionary is unnecessary but
# simplifies calling the replacements onto
# a symbolic expression.

print("Creating a dictionary with all the 4-symbol replacements ...")
super_solution = {}
for ircombo in all_sols:
    super_solution.update(all_sols[ircombo])

Creating all 4-symbol identities using the generators of the group ...
Creating set of identities ...
Finding trivial zeros ...
Using them to simplify things...
Creating reality identities ...
Solving all...
Creating a dictionary with all the 4-symbol replacements ...


In [1445]:
# test it out on a single element of one of the matrices
brak = config_matrices[sp.Symbol('t_{2}')*sp.Symbol('t_{2}')][(0,sp.Symbol('A_1'))][0][0]
simp_ket = Qet({})
for k,v in brak.dict.items():
    simp_ket += Qet({real_var_full_simplifier[k]:v})
true_qet = Qet({})
for k,v in simp_ket.dict.items():
    if k in super_solution:
        true_qet += v*Qet(super_solution[k])
    else:
        true_qet += Qet({k:v})
true_qet.as_braket()

2*<{\xi}{\xi}|{\phi}{\phi}> + <{\xi}{\xi}|{\xi}{\xi}>

In [1351]:
def simplifier(qet):
    simp_ket = Qet({})
    for k,v in qet.dict.items():
        simp_ket += Qet({real_var_full_simplifier[k]:v})
    true_qet = Qet({})
    for k,v in simp_ket.dict.items():
        if k in super_solution:
            true_qet += v*Qet(super_solution[k])
        else:
            true_qet += Qet({k:v})
    return true_qet#.as_braket()
def simplify_config_matrix(ir1ir2, S, ir3):
    config_matrix = config_matrices[ir1ir2][(S,ir3)]
    num_rows = len(config_matrix)
    simple_matrix = [[simplifier(config_matrix[row][col]) for col in range(num_rows)] for row in range(num_rows)]
    #simple_matrix = sp.Matrix(simple_matrix)
    return simple_matrix

In [1352]:
simple_config_matrices = {k:{} for k in config_matrices}
for ir1ir2 in config_matrices.keys():
    for term_key in config_matrices[ir1ir2]:
        S, Γ3 = term_key
        simple_config_matrices[ir1ir2][term_key] = simplify_config_matrix(ir1ir2, S, Γ3)

In [1347]:
list(simple_config_matrices[sp.Symbol('e')*sp.Symbol('t_{2}')][(0,sp.Symbol('T_2'))])[0][0].as_braket()

4*<{\zeta}{\phi}|{\zeta}{\phi}>/3 - <{\zeta}{\xi}|{\zeta}{\xi}>/3 + 4*<{\zeta}{\zeta}|{\phi}{\phi}>/3 - <{\zeta}{\zeta}|{\xi}{\xi}>/3

In [1335]:
config_matrices[sp.Symbol('e')*sp.Symbol('t_{2}')][(0,sp.Symbol('T_2'))][0][0].as_braket()

<{\gamma}{\xi}|{\gamma}{\xi}> + <{\gamma}{\xi}|{\xi}{\gamma}>

In [ ]:
# print them out
for ir1ir2 in simple_config_matrices.keys():
    print('-'*10)
    display(ir1ir2)
    for term_key in simple_config_matrices[ir1ir2]:
        display(term_key)
        display(simple_config_matrices[ir1ir2][term_key])

In [1234]:
# nullspace approach doesn't solve for
# all redundancies
# # eqns = list(rref_mat_non_zero*varvec)
# print("Finding the nullspace ...")
# nullspace = rref_mat_non_zero.nullspace()
# print("Converting into solution dictionary ...")
# normal_nullspace = []
# sols = {}
# for row in nullspace:
#     row = list(row)
#     for element_idx, element in enumerate(row):
#         if element !=0:
#             break
#     normalizer = sp.S(element)
#     if normalizer == 1:
#         sols[problem_vars[element_idx]] = {problem_vars[idx]:row[idx] for idx in range(len(row)) if (idx != element_idx and row[idx]!=0)}
#     else:
#         normal_row = [el/normalizer for el in row]
#         if problem_vars[element_idx] in sols:
#             print("huh?",end='|')
# #             print(problem_vars[element_idx])
# #             print({problem_vars[idx]:normal_row[idx] for idx in range(len(row)) if (idx != element_idx and row[idx]!=0)})
#         sols[problem_vars[element_idx]] = {problem_vars[idx]:normal_row[idx] for idx in range(len(row)) if (idx != element_idx and row[idx]!=0)}

<span style="color:lime">
2.2 --- Two symbol identities.
</span>

In [1353]:
twotuplerecovery = fourtuplerecovery

In [1354]:
print("Creating all 2-symbol identities using the generators of the group ...")

# this   integral_identities  dictionary  will  have  as  keys
# 2-tuples  of  irreps  and  its  values  will  be lists whose
# elements  are  2-tuples whose first elements are 2-tuples of
# irrep  components  and  whose values are qets whose keys are
# 2-tuples  of  irrep components and whose values are numeric.
# These 2-tuples represent a braket with the an invariant operator
# operator in between.

integral_identities_2 = {}
ir_mats = group.irrep_matrices
for ir1, ir2 in product(*([group.irrep_labels]*2)):
    # To simplify calculations this part
    # can only be done over quadruples in a standard
    # order.
    # Whatever is missed here is then brough back in
    # by the reality relations.
    altorder1 = (ir2, ir1, ir1, ir4)
    if (altorder1 in integral_identities_2):
        continue
    integral_identity_sector = []
    components = [component_labels[ir] for ir in [ir1, ir2]]
    comp_to_idx = [{c: idx for idx, c in enumerate(component_labels[ir])} for ir in [ir1, ir2]]
    for R in group.generators + [sp.Symbol('{Ee}')]:
        R_id = {}
        for γ1, γ2 in product(*components):
            for γ1p, γ2p in product(*components):
                    val = sp.conjugate(ir_mats[ir1][R][comp_to_idx[0][γ1],comp_to_idx[0][γ1p]]) *\
                          ir_mats[ir2][R][comp_to_idx[1][γ2],comp_to_idx[1][γ2p]]
                    if val== 0:
                        continue
                    key = (γ1, γ2)
                    if key not in R_id.keys():
                        R_id[key] = []
                    R_id[key].append( Qet({(γ1p,γ2p): val}) )
        R_id_total = [(key, sum(R_id[key], Qet({}))) for key in R_id.keys()]
        R_id_total = [q for q in R_id_total if len(q[1].dict)>0]
        integral_identity_sector.extend(R_id_total)
    integral_identities_2[(ir1,ir2)] = integral_identity_sector

Creating all 2-symbol identities using the generators of the group ...


In [1355]:
# For solving the linear system it is convenient
# to have everything on one side of the equation.

print("Creating set of identities ...")
identities_2 = {}
for ircombo in integral_identities_2:
    these_ids = []
    for v in integral_identities_2[ircombo]:
        lhs, rhs = v
        diff = Qet({lhs:1}) - rhs
        if len(diff.dict) > 0:
            these_ids.append(diff)
    identities_2[ircombo] = these_ids

Creating set of identities ...


In [1356]:
# If an equation has only one term, then
# that immediately means that that term is zero.

print("Finding trivial zeros ...")
# first determine which ones have to be zero
better_identities_2 = {irc:[] for irc in identities_2}
all_zeros_2 = {}
for ircombo in identities_2:
    zeros = []
    for identity in identities_2[ircombo]:
        if len(identity.dict) == 1:
            zeros.append((list(identity.dict.keys())[0]))
        else:
            better_identities_2[ircombo].append(identity)
    all_zeros_2[ircombo] = zeros

Finding trivial zeros ...


In [1357]:
# use them to simplify things.    
print("Using them to simplify things...")
great_identities_2 = {irc:[] for irc in identities_2}
for ircombo in better_identities_2:
    for identity in better_identities_2[ircombo]:
        new_qet = Qet({})
        for k,v in identity.dict.items():
            if k in all_zeros_2[ircombo]:
                continue
            else:
                new_qet+= Qet({k: v})
        if len(new_qet.dict) == 0:
            continue
        great_identities_2[ircombo].append(new_qet)

Using them to simplify things...


In [1358]:
# Inside of a four-symbol braket one may three exchanges
# that must result in the same value. That if the wave
# functions are assumed to be real-valued.

print("Creating reality identities ...")
real_var_simplifiers_2 = {irc:[] for irc in great_identities_2}
# this has to run over all the quadruples of irs
kprimes = set()
for ir1, ir2 in product(*([group.irrep_labels]*2)):
    real_var_simplifier = {}
    ircombo = (ir1, ir2)
    components = [component_labels[ir] for ir in ircombo]
    for γ1, γ2 in product(*components):
        k = (γ1, γ2)
        kalt = (γ2, γ1)
        # If for a given key I find that its
        # switched version has already been seen
        # Then that key has to be mapped to be
        # mapped to the key already present.
        if kalt in kprimes:
            real_var_simplifier[k] = kalt
        else:
            real_var_simplifier[k] = k
            kprimes.add(k)
    real_var_simplifiers_2[ircombo] = real_var_simplifier

Creating reality identities ...


In [1359]:
# For each 4-tuple of irreps
# create a system of symbolic solutions
# and let sympy solve that.
# For each 4-tuple of irreps
# the end result is a dictionary
# whose keys represent the dependent
# brakets and whose values are the
# relation that those dependent values
# have with the independent brakets.
# As such, when these dictionaries are
# used on an expression, everything should
# then be given in terms of indepedent brakets.

print("Solving all...")
all_sols_2 = {irc:[] for irc in great_identities_2}
for ircombo in great_identities_2:
    zeros = all_zeros_2[ircombo]
    problem_vars = list(set(sum([list(identity.dict.keys()) for identity in great_identities_2[ircombo]],[])))
    big_ma = [awe.vec_in_basis(problem_vars) for awe in great_identities_2[ircombo]] 
    big_mat = sp.Matrix(big_ma)
    rref_mat, pivots = big_mat.rref()
    num_rows = rref_mat.rows
    num_cols = rref_mat.cols
    rref_ma_non_zero = [rref_mat[row,:] for row in range(num_rows) if (sum(np.array(rref_mat[row,:])[0] == 0) != num_cols)]
    rref_mat_non_zero = sp.Matrix(rref_ma_non_zero)
    varvec = sp.Matrix(list(map(composite_symbol,problem_vars)))
    eqns = rref_mat_non_zero*varvec
    ssol = sp.solve(list(eqns), dict=True)
    all_sols[ircombo] = ssol
    assert len(ssol) in [0,1]
    if len(ssol) == 0:
        sol_dict = {}
    else:
        sol_dict = ssol[0]
    zero_addendum = {k:{} for k in all_zeros_2[ircombo]}
    sol_dict = {twotuplerecovery(k):{twotuplerecovery(s):v.coeff(s) for s in v.free_symbols} for k,v in sol_dict.items()}
    sol_dict.update(zero_addendum)
    all_sols_2[ircombo] = sol_dict

# This final dictionary is unnecessary but
# simplifies calling the replacements onto
# a symbolic expression.

print("Creating a dictionary with all the 2-symbol replacements ...")
super_solution_2 = {}
for ircombo in all_sols_2:
    super_solution_2.update(all_sols_2[ircombo])

Solving all...
Creating a dictionary with all the 2-symbol replacements ...


In [1360]:
config_supplement[e_config][term]

[(E, E, E, {\gamma}, 0, 0), (E, E, E, {\zeta}, 0, 0)]

In [1398]:
# for a given electron config
more_ids = {ec:[] for ec in simple_config_matrices}
for e_config in simple_config_matrices:
    # for a given term
    for term in simple_config_matrices[e_config]:
#         more_ids[e_config][term] = []
        the_matrix = simple_config_matrices[e_config][term]
        num_rows = len(the_matrix)
        num_cols = len(the_matrix)
        the_state_keys = config_supplement[e_config][term]
        the_key_matrix = {}
        for row_idx in range(num_rows):
            Γ3_row = the_state_keys[row_idx][2]
            γ3_row = the_state_keys[row_idx][3]
            for col_idx in range(num_cols):
                Γ3_col = the_state_keys[col_idx][2]
                γ3_col = the_state_keys[col_idx][3]
                matrix_val = the_matrix[row_idx][col_idx]
                the_key_matrix[(γ3_row,γ3_col)] = matrix_val
        # now go over the keys of super_solution_2
        # and if one of those keys matches with a key in the_key_matrix
        # do something about it
        for k in super_solution_2:
            if k in the_key_matrix:
                v = super_solution_2[k]
                # this matrix element
                matrix_element = the_key_matrix[k]
                # must be identified with the sum
                # as given in v
                matrix_equiv = sum([kv*the_key_matrix[km] for km, kv in v.items()],Qet({}))
                identity = (matrix_element - matrix_equiv) #=0
                if len(identity.dict) != 0:
                    more_ids[e_config].append(identity)

In [1427]:
# For each 4-tuple of irreps
# create a system of symbolic solutions
# and let sympy solve that.
# For each 4-tuple of irreps
# the end result is a dictionary
# whose keys represent the dependent
# brakets and whose values are the
# relation that those dependent values
# have with the independent brakets.
# As such, when these dictionaries are
# used on an expression, everything should
# then be given in terms of indepedent brakets.

print("Solving all...")
all_sols_2_4 = {irc:[] for irc in more_ids}
for ircombo in more_ids:
#     print(ircombo)
    problem_vars = list(set(sum([list(identity.dict.keys()) for identity in more_ids[ircombo]],[])))
    big_ma = [awe.vec_in_basis(problem_vars) for awe in more_ids[ircombo]] 
    big_mat = sp.Matrix(big_ma)
    rref_mat, pivots = big_mat.rref()
    num_rows = rref_mat.rows
    num_cols = rref_mat.cols
    rref_ma_non_zero = [rref_mat[row,:] for row in range(num_rows) if (sum(np.array(rref_mat[row,:])[0] == 0) != num_cols)]
    rref_mat_non_zero = sp.Matrix(rref_ma_non_zero)
    varvec = sp.Matrix(list(map(composite_symbol,problem_vars)))
    eqns = rref_mat_non_zero*varvec
#     display(eqns)
    ssol = sp.solve(list(eqns), dict=True)
    all_sols_2_4[ircombo] = ssol
    assert len(ssol) in [0,1]
    if len(ssol) == 0:
        sol_dict = {}
    else:
        sol_dict = ssol[0]
    sol_dict = {twotuplerecovery(k):{twotuplerecovery(s):v.coeff(s) for s in v.free_symbols} for k,v in sol_dict.items()}
    all_sols_2_4[ircombo] = sol_dict

fab_solution_2_4 = {}
for ircombo in all_sols_2_4:
    fab_solution_2_4.update(all_sols_2_4[ircombo])

Solving all...


In [1438]:
mega_solution = {}
new_info_keys = set(fab_solution_2_4.keys())
for k, v in super_solution.items():
    # if any of the rhs keys in v
    # is in all_sols_2_4
    # use it to simplify it
    v_keys = set(v.keys())
    if len(v_keys.intersection(new_info_keys)) != 0:
        new_qet = Qet({})
        print('+',end='|')
        for k,v in v.items():
            if v in fab_solution_2_4:
                for k0,v0 in fab_solution_2_4[v]:
                    new_qet += v*Qet({k0:v0})
            else:
                new_qet += Qet({k:v})
        new_dict = new_qet.dict
        mega_solution[k] = new_dict
    else:
        mega_solution[k] = v

-|-|-|-|-|-|-|-|

In [1441]:
def simplifier_f(qet):
    simp_ket = Qet({})
    for k,v in qet.dict.items():
        simp_ket += Qet({real_var_full_simplifier[k]:v})
    true_qet = Qet({})
    for k,v in simp_ket.dict.items():
        if k in mega_solution:
            true_qet += v*Qet(mega_solution[k])
        else:
            true_qet += Qet({k:v})
    return true_qet.as_braket()
def simplify_config_matrix_f(ir1ir2, S, ir3):
    config_matrix = config_matrices[ir1ir2][(S,ir3)]
    num_rows = len(config_matrix)
    simple_matrix = [[simplifier_f(config_matrix[row][col]) for col in range(num_rows)] for row in range(num_rows)]
    simple_matrix = sp.Matrix(simple_matrix)
    return simple_matrix

In [1442]:
final_config_matrices = {k:{} for k in config_matrices}
for ir1ir2 in config_matrices.keys():
    for term_key in config_matrices[ir1ir2]:
        S, Γ3 = term_key
        final_config_matrices[ir1ir2][term_key] = simplify_config_matrix_f(ir1ir2, S, Γ3)

In [1455]:
sp.latex(ir1ir2)

'e^{2}'

In [1464]:
simple_config_matrices[sp.Symbol('e')*sp.Symbol('e')][(0,sp.Symbol('A_1'))][0][0].as_braket()

<{\zeta}{\zeta}|{\gamma}{\gamma}> + <{\zeta}{\zeta}|{\zeta}{\zeta}>

In [1460]:
final_config_matrices[sp.Symbol('e')*sp.Symbol('e')][(0,sp.Symbol('A_1'))]

Matrix([[-<{\zeta}{\gamma}|{\zeta}{\gamma}>/2 + <{\zeta}{\zeta}|{\gamma}{\gamma}>/2 + <{\zeta}{\zeta}|{\zeta}{\zeta}>]])

In [1453]:
# print them out
for counter, ir1ir2 in enumerate(final_config_matrices):
    print('-'*10)
    display(ir1ir2)
    for term_key in final_config_matrices[ir1ir2]:
        display(term_key)
        display(final_config_matrices[ir1ir2][term_key])
        display(sp.Matrix([[k.as_braket() for k in row] for row in simple_config_matrices[ir1ir2][term_key]]))
    if counter == 2:
        break

----------


a_{1}**2

(0, A_1)

Matrix([[<{\alpha}{\alpha}|{\alpha}{\alpha}>]])

Matrix([[<{\alpha}{\alpha}|{\alpha}{\alpha}>]])

...

Matrix([
[<{\gamma}{\zeta}|{\gamma}{\zeta}> - <{\zeta}{\zeta}|{\gamma}{\gamma}>,                                                                                                             0,                                                                     0],
[                                                                    0, <{\gamma}{\zeta}|{\gamma}{\zeta}>/2 + <{\zeta}{\gamma}|{\zeta}{\gamma}>/2 - <{\zeta}{\zeta}|{\gamma}{\gamma}>,                                                                     0],
[                                                                    0,                                                                                                             0, <{\gamma}{\zeta}|{\gamma}{\zeta}> - <{\zeta}{\zeta}|{\gamma}{\gamma}>]])

Matrix([
[<{\zeta}{\gamma}|{\zeta}{\gamma}> - <{\zeta}{\zeta}|{\gamma}{\gamma}>,                                                                     0,                                                                     0],
[                                                                    0, <{\zeta}{\gamma}|{\zeta}{\gamma}> - <{\zeta}{\zeta}|{\gamma}{\gamma}>,                                                                     0],
[                                                                    0,                                                                     0, <{\zeta}{\gamma}|{\zeta}{\gamma}> - <{\zeta}{\zeta}|{\gamma}{\gamma}>]])

(0, E)

Matrix([
[-<{\zeta}{\gamma}|{\zeta}{\gamma}>/2 - 3*<{\zeta}{\zeta}|{\gamma}{\gamma}>/2 + <{\zeta}{\zeta}|{\zeta}{\zeta}>,                                                                                                             0],
[                                                                                                             0, <{\gamma}{\zeta}|{\gamma}{\zeta}>/2 + <{\zeta}{\gamma}|{\zeta}{\gamma}>/2 + <{\zeta}{\zeta}|{\gamma}{\gamma}>]])

Matrix([
[-<{\zeta}{\zeta}|{\gamma}{\gamma}> + <{\zeta}{\zeta}|{\zeta}{\zeta}>,                                                                     0],
[                                                                   0, <{\zeta}{\gamma}|{\zeta}{\gamma}> + <{\zeta}{\zeta}|{\gamma}{\gamma}>]])

### Improving notation for components

In [640]:
# def notation_maker(group)
greek = 'alpha beta gamma zeta eta mu nu xi phi chi omega'
greek_alphabet = [sp.Symbol(r'\%s' % g) for g in greek.split(' ')]
new_labels = []
for group_label in CPGs.all_group_labels:
    group = CPGs.get_group_by_label(group_label)
    # get the irrep symbols
    irrep_symbols = group.irrep_labels
    # get the irrep dims
    irrep_dims = group.irrep_dims
    # separate them according to their dimension
    dims = list(set(irrep_dims.values()))
    irrep_symbols = OrderedDict([(dim, ([isymb for isymb in irrep_symbols if irrep_dims[isymb]==dim])) for dim in dims])
    irrep_symbols = OrderedDict([(k,sorted(v, key = lambda x: sp.latex(x))) for k,v in irrep_symbols.items()])
    irrep_symbols = sum(list(irrep_symbols.values()),[])
    asymb = str(irrep_symbols[0])
    if 'g' in asymb or 'u' in asymb:
        irrep_symbols_u = [i for i in irrep_symbols if 'u' in str(i)]
        irrep_symbols_v = [i for i in irrep_symbols if 'g' in str(i)]
        component_labels = {ir:OrderedDict() for ir in group.irrep_labels}
        counter = 0
        for irrep_symbol in irrep_symbols_u:
            old_components = group.component_labels[irrep_symbol]
            for old_component in old_components:
                component_labels[irrep_symbol][old_component] = sp.Symbol('{%s}_{u}' % (sp.latex(greek_alphabet[counter])))
                counter += 1
        counter = 0
        for irrep_symbol in irrep_symbols_v:
            old_components = group.component_labels[irrep_symbol]
            for old_component in old_components:
                component_labels[irrep_symbol][old_component] = sp.Symbol('{%s}_{g}' % (sp.latex(greek_alphabet[counter])))
                counter += 1
    elif group_label in ['C_{6h}','C_{4h}']:
        component_labels = {ir:OrderedDict() for ir in group.irrep_labels}
        counter = 0
        for irrep_symbol in irrep_symbols:
            old_components = group.component_labels[irrep_symbol]
            for old_component in old_components:
                component_labels[irrep_symbol][old_component] = sp.Symbol('{\gamma}_{%d}' % (counter+1))
                counter += 1
    else:
        component_labels = {ir:OrderedDict() for ir in group.irrep_labels}
        counter = 0
        for irrep_symbol in irrep_symbols:
            old_components = group.component_labels[irrep_symbol]
            for old_component in old_components:
                component_labels[irrep_symbol][old_component] = sp.Symbol('{%s}' % sp.latex(greek_alphabet[counter]))
                counter += 1
    new_labels.append((group_label, component_labels))
new_labels = OrderedDict(new_labels)

In [169]:
outlines = []
for group_label in CPGs.all_group_labels:
    outlines.append("\\subsection{Group : ${%s}$}" % group_label)
#     if group_label != 'O':
#         continue
    group = CPGs.get_group_by_label(group_label)
    irrep_symbols = group.irrep_labels
    components = group.component_labels
    for irrep in irrep_symbols:
        these_components = ','.join(list(map(sp.latex,components[irrep])))
        these_new_components = ','.join(list(map(lambda x: sp.latex(new_labels[group_label][irrep][x]),components[irrep])))
        outlines.append('\n\\noindent$%s: %s $\n\\vspace{0.1cm}\n' % (sp.latex(irrep),these_components))
        outlines.append('\n\\noindent$%s: %s $\n\\vspace{0.5cm}\n' % (sp.latex(irrep),these_new_components))
final_output = '\n'.join(outlines).replace("^'","^{'}").replace("^''","^{''}")
open('/Users/juan/Library/Mobile Documents/com~apple~CloudDocs/iCloudFiles/Theoretical Division/ComponentLabeling-rosetta.tex','w').write(final_output)

18473

### Spin-orbitals (see qdef-tables.tex)

In [3]:
new_labels = pickle.load(open('./Data/components_rosetta.pkl','rb'))

In [4]:
# with new labels
group_chunks = []
for group_label in CPGs.all_group_labels:
    group = CPGs.get_group_by_label(group_label)
    all_spin_combos = OrderedDict()
    spin_states = [sp.Symbol(r'\alpha'), sp.Symbol(r'\beta')]
    group.new_component_labels = {ir:list(new_labels[group_label][ir].values()) for ir in group.irrep_labels}
    for irrep in group.irrep_labels:
        spin_orbitals = []
        for component0, component1 in product(group.new_component_labels[irrep],group.new_component_labels[irrep]):
            for s0, s1 in product(spin_states,spin_states):
                spin_orbital = (component0, component1, s0, s1)
                if (component1, component0, s1, s0) not in spin_orbitals:
                    if (component1, s1) != (component0, s0):
                        spin_orbitals.append(spin_orbital)
        spin_orbital_symbols = {}
        for spin_orbital in spin_orbitals:
            component0, component1, s0, s1 = spin_orbital
            if s0 == spin_states[0]:
                cstar0 = sp.latex(component0)
            else:
                cstar0 = r'\overline{%s}' % sp.latex(component0)
            if s1 == spin_states[0]:
                cstar1 = sp.latex(component1)
            else:
                cstar1 = r'\overline{%s}' % sp.latex(component1)
            spin_orbital_symbols[spin_orbital] = sp.Symbol(r'$|%s%s|$' % (cstar0, cstar1))
        all_slater_symbols = []
        for s in list(spin_orbital_symbols.values()):
            all_slater_symbols.append(sp.latex(s))
        all_slater_symbols = ', '.join(all_slater_symbols)
        all_slater_symbols = '\\begin{center}\n%s\n\\end{center}' % all_slater_symbols
        all_slater_symbols = ('\\begin{center}\n $%s{\\cdot}%s:$ \n\\end{center}\n\n' % (sp.latex(irrep),sp.latex(irrep))) + all_slater_symbols
        all_spin_combos[irrep] = all_slater_symbols
    final_output = '\n\n'.join(all_spin_combos.values())
    final_output = ('\\hrulefill \n \\begin{center} Group $%s$ \\end{center} \n' % group_label) + final_output
    group_chunks.append(final_output)
super_output = '\n'.join(group_chunks)
super_output = super_output.replace("^'","^{'}").replace("^''","^{''}")
open('//Users/juan/Library/Mobile Documents/com~apple~CloudDocs/iCloudFiles/Theoretical Division/spin-orbitals.tex','w').write(super_output)

31183

In [638]:
pickle.dump(new_labels, open('./Data/components_rosetta.pkl','wb'))

In [639]:
# group_chunks = []
# for group_label in CPGs.all_group_labels:
#     group = CPGs.get_group_by_label(group_label)
#     all_spin_combos = OrderedDict()
#     spin_states = [sp.Symbol(r'\alpha'), sp.Symbol(r'\beta')]
#     for irrep in group.irrep_labels:
#         spin_orbitals = []
#         for component0, component1 in product(group.component_labels[irrep],group.component_labels[irrep]):
#             for s0, s1 in product(spin_states,spin_states):
#                 spin_orbital = (component0, component1, s0, s1)
#                 if (component1, component0, s1, s0) not in spin_orbitals:
#                     if (component1, s1) != (component0, s0):
#                         spin_orbitals.append(spin_orbital)
#         spin_orbital_symbols = {}
#         for spin_orbital in spin_orbitals:
#             component0, component1, s0, s1 = spin_orbital
#             if s0 == spin_states[0]:
#                 cstar0 = sp.latex(component0)
#             else:
#                 cstar0 = r'\overline{%s}' % sp.latex(component0)
#             if s1 == spin_states[0]:
#                 cstar1 = sp.latex(component1)
#             else:
#                 cstar1 = r'\overline{%s}' % sp.latex(component1)
#             spin_orbital_symbols[spin_orbital] = sp.Symbol(r'$|%s%s|$' % (cstar0, cstar1))
#         all_slater_symbols = []
#         for s in list(spin_orbital_symbols.values()):
#             all_slater_symbols.append(sp.latex(s))
#         all_slater_symbols = ', '.join(all_slater_symbols)
#         all_slater_symbols = '\\begin{center}\n%s\n\\end{center}' % all_slater_symbols
#         all_slater_symbols = ('\\begin{center}\n $%s{\\cdot}%s:$ \n\\end{center}\n\n' % (sp.latex(irrep),sp.latex(irrep))) + all_slater_symbols
#         all_spin_combos[irrep] = all_slater_symbols
#     final_output = '\n\n'.join(all_spin_combos.values())
#     final_output = ('\\hrulefill \n \\begin{center} Group $%s$ \\end{center} \n' % group_label) + final_output
#     group_chunks.append(final_output)
# super_output = '\n'.join(group_chunks)
# super_output = super_output.replace("^'","^{'}").replace("^''","^{''}")
# open('//Users/juan/Library/Mobile Documents/com~apple~CloudDocs/iCloudFiles/Theoretical Division/spin-orbitals.tex','w').write(super_output)

### 2e Terms and their wavefunctions

In [512]:
from sympy.physics.wigner import clebsch_gordan as ClebschG

In [513]:
term_energies_for_printout = pickle.load(open('./data/term_energies_for_print_1637619241.pkl','rb'))
# term_energies_for_printout = pickle.load(open('./data/term_energies_for_print_alt.pkl','rb'))

In [514]:
# for key, qet in term_energies_for_printout.items():
#     truth = list(map(lambda x: sp.im(x) == 0,qet.dict.values()))
#     if len(truth) > 1:
#         break
#     if not all(truth):
#         print("wups")
#         break

In [515]:
def det_simplify(qet):
    '''
    Simplification from antisymmetry of composing elements.
    '''
    qet_dict = qet.dict
    best_qet = {}
    for key, coeff in qet_dict.items():
        ikey = (*key[3:],*key[:3])
        current_keys = list(best_qet.keys())
        if ikey in current_keys:
            best_qet[ikey] += -coeff
            continue
        if key not in current_keys:
            best_qet[key] = coeff
        else:
            best_qet[key] += coeff
    return Qet(best_qet)

In [516]:
# this is now included in qdefcore
# class Term():
#     '''
#     To  represent  a  term,  this  object  holds the states that
#     correspond  to  it,  and offers a few methods in to view the
#     enclosed wave functions.
#     '''
#     def __init__(self, init_dict):
#         for k,v in init_dict.items():
#             setattr(self, k, v)
#         self.term_prototype = sp.Symbol(r'{{}}^{{{M}}}{ir}')
#         self.state_label_prototype = sp.Symbol(r'\Psi({α}, {S}, {{{Γ}}}, {M_s}, {γ})')
#     def term_symbol(self):
#         return sp.Symbol(str(self.term_prototype).format(M=str(2*self.S+1),ir=str(sp.latex(self.irrep))))
#     def make_state_symbols(self):
#         state_symbols = []
#         for qet, state_key in zip(self.states, self.state_keys):
#             (Γ1, Γ2, Γ3, γ3, S, mSz) = state_key
#             ket = qet.as_ket(fold_keys=True)
#             ket_symbol = sp.latex(ket).replace('\\right\\rangle','\\right|')
#             α = Γ1*Γ2
#             term_symb = self.term_symbol()
#             state_symbol = '\\Psi(%s,%s,M\!=\!%d,%s)' % (sp.latex(α).lower(), term_symb, mSz, sp.latex(γ3))
#             state_symbols.append((state_symbol, ket_symbol))
#         return state_symbols
#     def __str__(self):
#         return (self.term_symbol())
#     def __repr__(self):
#         return '%s: %d states' % (str(self.term_symbol()), len(self.states))

In [517]:
def as_determinantal_ket(qet):
    qdict = {}
    for k,v in qet.dict.items():
        if k[1] >= 0:
            k0 = sp.Symbol(str(k[0]))
        else:
            k0 = sp.Symbol('\\overline{%s}'%str(k[0]))
        if k[3] >= 0:
            k1 = sp.Symbol(str(k[2]))
        else:
            k1 = sp.Symbol('\\overline{%s}'%str(k[2]))
        qdict[(k0,k1)] = v
    qet = Qet(qdict)
    qet = qet*(1/qet.norm())
    ket = qet.as_ket(fold_keys=True)
    ket = sp.latex(ket).replace('\\right\\rangle','\\right|')
    return sp.Symbol(ket), qet

In [518]:
# qet = Qet({(str(i),str(i)*2):i for i in range(10)})
# chunk_size = 3
# qet_parts = list(qet.dict.items())
# qets = [Qet(dict(qet_parts[i:(i+chunk_size)])) for i in range(0,len(qet_parts),chunk_size)]
# lqets = ([sp.latex(qet.as_braket()).replace('|','||').replace(r'\right.',r'\right.\!\!') for qet in qets])

In [519]:
# with new labels and checking for num of waves
def format_empheq(qet):
    chunk_size = 3
    qet_parts = list(qet.items())
    qets = [Qet(dict(qp[i:(i+chunk_size)])) for i in range(0,len(qet_parts),chunk_size)]

def num_waves(ir0,ir1):
    if ir0 == ir1:
        return sp.binomial(group.irrep_dims[ir0]*2, 2)
    else:
        return group.irrep_dims[ir0]*group.irrep_dims[ir1]*4

def full_waves():
    done = []
    size = 0
    for k,v in group.product_table.odict.items():
        if (k[1],k[0]) in done:
            continue
        if k[1] == k[0]:
            size += sp.binomial(group.irrep_dims[k[1]]*2,2)
        else:
            size += sum(list(map(lambda x: group.irrep_dims[x]*4, v)))
        done.append(k)
    return size

def format_empheq(qet):
    if len(qet.dict) == 0:
        return r'''
\vspace{0.2cm}
\boxed{\Delta{E}=0}
\vspace{0.2cm}'''
    else:
        chunk_size = 3
        qet_parts = list(qet.dict.items())
        qets = [Qet(dict(qet_parts[i:(i+chunk_size)])) for i in range(0,len(qet_parts),chunk_size)]
        chunks = [sp.latex(qet.as_braket()).replace('|','||').replace(r'\right.',r'\right.\!\!') for qet in qets]
        chunks[0] = r'\Delta{E}='+chunks[0]
        for chunk_idx in range(1,len(chunks)):
            if chunks[chunk_idx][0] != '-':
                chunks[chunk_idx] = '+' + chunks[chunk_idx]
        lqets = '\\\\\n'.join(chunks)
        nice_output = r'''\vspace{-0.5cm}
\begin{empheq}[box=\fbox]{gather*}
%s
\end{empheq}''' % lqets
        return nice_output

all_final_outputs = []
tally_waves = {}
single_col_groups = ['C_{1}','C_{2}']
num_multicols = {group_label:2 for group_label in CPGs.all_group_labels}
for scg in single_col_groups:
    num_multicols[scg] = 1
archival_terms = {}
for group_counter, group_label in enumerate(CPGs.all_group_labels):
#     if group_label != 'C_{i}':
#         continue
    print("Working on group %s ..." % group_label)
    group = CPGs.get_group_by_label(group_label)
    print("computing partial sums...")
    s1, s2 = sp.S(1)/2, sp.S(1)/2
    Ss  = [0,1]
    m1s = [-sp.S(1)/2, sp.S(1)/2]
    m2s = [-sp.S(1)/2, sp.S(1)/2]
    group_CGs = group.CG_coefficients
    flat_labels = dict(sum([list(l.items()) for l in list(new_labels[group_label].values())],[]))
    group_CGs = {(flat_labels[k[0]], flat_labels[k[1]], flat_labels[k[2]]):v for k,v in group_CGs.items()}
    summands = OrderedDict()
    group.new_component_labels = OrderedDict([(ir, list(new_labels[group_label][ir].values())) for ir in group.irrep_labels])
    for Γ1, Γ2, Γ3, m1, m2, S in product(group.irrep_labels,
                                     group.irrep_labels,
                                     group.irrep_labels,
                                     m1s,
                                     m2s,
                                     Ss):
        for γ1, γ2, γ3 in product(group.new_component_labels[Γ1],
                                  group.new_component_labels[Γ2],
                                  group.new_component_labels[Γ3]):
            for mSz in range(S,-S-1,-1):
                sCG = ClebschG(s1, s2, S, m1, m2, mSz)
                if (γ1, γ2, γ3) not in group_CGs.keys():
                    continue
                else:
                    gCG = group_CGs[(γ1, γ2, γ3)]
                coeff = sCG*gCG
                key = (Γ1, Γ2, Γ3, γ3, S, mSz)
                if (Γ2,m2,γ2) == (Γ1,m1,γ1):
                    continue
                if coeff!=0:
                    if key not in summands.keys():
                        summands[key] = []
                    summands[key].append(Qet({(Γ1,γ1,m1,Γ2,γ2,m2):coeff}))
    print("summing out inner parts ...")
    total_qets = OrderedDict([(k, sum(v,Qet({}))) for k,v in summands.items()])
    best_qets = OrderedDict([(k, det_simplify(v)) for k,v in total_qets.items()])
    best_qets = OrderedDict([(k, v) for k,v in best_qets.items() if len(v.dict)!=0])
    print("term collection truck ...")
    terms = OrderedDict()
    done_keys = []
    for k, v in best_qets.items():
        (Γ1, Γ2, Γ3, γ3, S, mSz) = k
        equiv_key = (Γ2, Γ1, Γ3, γ3, S, mSz)
        term_pair = (Γ3, S)
        if term_pair not in terms.keys():
            terms[term_pair] = OrderedDict()
        if equiv_key in done_keys:
            continue
        terms[term_pair][k] = v
        done_keys.append(k)
    final_terms = OrderedDict()
    for term in terms:
        ir, S = term
        states = terms[term]
        one_term = Term({'irrep': ir, 'S': S, 'states': states})
        final_terms[term] = one_term
    print("Manufacturing latex output ....")
    max_counter = 3
    if group_counter == 0:
        print_outs = ['\n\\newpage\n\n\\section{Terms and wave functions}\n\\subsection{Group $%s$}\n\n\\begin{center} \\underline{Component labels} \n\\vspace{0.2cm}\n' % group_label]
    else:
        print_outs = ['\n\\doublerulefill\n\\subsection{Group $%s$} \n\n\\begin{center}\n\n\\underline{Component labels} \n\\vspace{0.2cm}\n' % group_label]
    for running_idx, irrep_symbol in enumerate(group.irrep_labels):
        components_for_printing = ','.join(list(map(sp.latex, group.new_component_labels[irrep_symbol])))
        if running_idx < (len(group.irrep_labels)-1):
            print_outs.append('$%s:\\{%s\\}$ || ' % (sp.latex(irrep_symbol), components_for_printing))
        else:
            print_outs.append('$%s:\\{%s\\}$' % (sp.latex(irrep_symbol), components_for_printing))
    if num_multicols[group_label] == 1:
        print_outs.append('\\end{center}\n\n')
    else:
        print_outs.append('\\end{center}\n\n\\begin{multicols}{%d}\n\n' % num_multicols[group_label])
    supreme_states = {}
    total_waves = 0
    end_terms = {}
    for one_term_k, one_term in final_terms.items():
        term_symb = r'{{}}^{{{M}}}\!{ir}'.format(M=(2*one_term_k[1]+1), ir = sp.latex(one_term_k[0]))
        term_symb = sp.Symbol(term_symb)
        print_outs.append('\n\\hrulefill\n\\subsubsection{$%s$}\n\\vspace{0.25cm}\n \\begin{center} \n' % sp.latex(term_symb))
        counter = 0
        prev_α = ''
        end_terms[one_term_k] = []
        origins = []
        for state_key, state in one_term.states.items():
            (Γ1, Γ2, Γ3, γ3, S, mSz) = state_key
            αl = sp.Symbol(sp.latex(Γ1).lower())*sp.Symbol(sp.latex(Γ2).lower())
            α = Γ1 * Γ2
            multiplicity = 2*S+1
            term_symb = r'{{}}^{{{M}}}\!{ir}'.format(M=(2*S+1), ir = sp.latex(Γ3))
            state_symbol = '\\Psi_{%d}(%s,%s,M\!=\!%d,%s)' % (counter+1, sp.latex(α).lower(), term_symb, mSz, sp.latex(γ3))
            state_symbol = sp.Symbol(state_symbol)
            v_simple = Qet({(k[1],k[2],k[4],k[5]):v for k,v in state.dict.items()})
            v_det, q_qet = as_determinantal_ket(v_simple)
            end_terms[one_term_k].append(q_qet)
            origins.append(state_key)
            sup_key = α
            if sup_key not in supreme_states.keys():
                supreme_states[sup_key] = []
            supreme_states[sup_key].append(v_det)
            pout = "$\\textcolor{blue}{%s} = %s$\\vspace{0.1cm}\n" % (sp.latex(state_symbol), sp.latex(v_det))
            pout = sp.Symbol(pout)
            if prev_α != α:
                if prev_α == '':
                    print_outs.append('\n\n $\\textcolor{red}{%s}$ \n\n' % (sp.latex(α).lower()))
                else:
                    print_outs.append('\n\\vspace{0.25cm}\n\n $\\textcolor{red}{%s}$ \n\n' % (sp.latex(α).lower()))
                term_energy = term_energies_for_printout[(group_label, αl, one_term_k[-1::-1])]
                print_outs.append('\n\n %s \n\n' % (format_empheq(term_energy)))
#                 term_energy = term_energies_for_printout[(group_label, αl, one_term_k[-1::-1])].as_braket()
#                 term_energy = sp.latex(term_energy).replace('|','||').replace(r'\right.',r'\right.\!\!')
#                 print_outs.append('\n\n\\boxed{%s}\n\n' % (term_energy))
            prev_α = α
            print_outs.append(sp.latex(pout))
            counter += 1
            total_waves += 1
            done_keys.append(state_key)
        end_terms[one_term_k] = Term({'irrep': one_term_k[0], 'S': one_term_k[1], 'states': end_terms[one_term_k], 'state_keys': origins})
        print_outs.append('\n\\end{center}\n')
    archival_terms[group_label] = end_terms
    if num_multicols[group_label] != 1:
        print_outs.append('\n\\end{multicols}\n')
    irrep_combos = list(combinations_with_replacement(group.irrep_labels,2))
    total_waves_groundtruth = sum([num_waves(ir0,ir1) for ir0, ir1 in irrep_combos])
    tally_waves[group_label] = (total_waves,total_waves_groundtruth,full_waves())
    all_wavefunctions = '\n'.join(print_outs)
    all_wavefunctions = all_wavefunctions.replace("^'","^{'}").replace("^''","^{''}")
    all_final_outputs.append(all_wavefunctions)
print("saving to file ...")
super_final_output = '\n'.join(all_final_outputs)
open('/Users/juan/Library/Mobile Documents/com~apple~CloudDocs/iCloudFiles/Theoretical Division/termwaves.tex','w').write(super_final_output)
# !beep
print("Saving to pickle ...")
pickle.dump(archival_terms, open('./Data/2e-terms.pkl','wb'))

Working on group C_{1} ...
computing partial sums...
summing out inner parts ...
term collection truck ...
Manufacturing latex output ....
Working on group C_{i} ...
computing partial sums...
summing out inner parts ...
term collection truck ...
Manufacturing latex output ....
Working on group C_{2} ...
computing partial sums...
summing out inner parts ...
term collection truck ...
Manufacturing latex output ....
Working on group C_{s} ...
computing partial sums...
summing out inner parts ...
term collection truck ...
Manufacturing latex output ....
Working on group C_{2h} ...
computing partial sums...
summing out inner parts ...
term collection truck ...
Manufacturing latex output ....
Working on group D_{2} ...
computing partial sums...
summing out inner parts ...
term collection truck ...
Manufacturing latex output ....
Working on group C_{2v} ...
computing partial sums...
summing out inner parts ...
term collection truck ...
Manufacturing latex output ....
Working on group D_{2h} .

In [520]:
!beep silent

silence


In [465]:
# group_label = 'T_{h}'
# group = CPGs.get_group_by_label(group_label)
# irreps = group.irrep_labels
# irrep_combos = list(combinations_with_replacement(group.irrep_labels,2))
# total_waves_groundtruth = sum([num_waves(ir0,ir1) for ir0, ir1 in irrep_combos])

In [466]:
# total_waves_asfound = sum([len(v) for k,v in supreme_states.items()])
# total_waves_asfound

In [467]:
# class Term():
#     '''
#     Represents a term, holds the states that correspond to it,
#     and offers a display of the term symbol and enclosed wavefunctions.
#     '''
#     def __init__(self, init_dict):
#         for k,v in init_dict.items():
#             setattr(self, k, v)
#         self.term_prototype = sp.Symbol(r'{{}}^{{{M}}}{ir}')
#         self.state_prototype = sp.Symbol(r'\Psi({α}, {S}, {{{Γ}}}, {M_s}, {γ})')
#     def term_symbol(self):
#         return sp.Symbol(str(self.term_prototype).format(M=str(2*self.S+1),ir=str(sp.latex(self.irrep))))
#     def make_state_symbols(self):
#         state_symbols = []
#         for state in self.states:
#             ndict = {
#             'α' : state['qnums'][sp.Symbol(r'\alpha')],
#             'S' : self.S,
#             'Γ' : self.irrep,
#             'M_s' : state['qnums'][sp.Symbol(r'M_s')],
#             'γ' : state['qnums'][sp.Symbol(r'\gamma')]
#             }
#             state_symbols.append(sp.Symbol(str(self.state_prototype).format(**ndict)))
#         return state_symbols
#     def __str__(self):
#         return (self.term_symbol())
#     def __repr__(self):
#         return str(self.term_symbol())

In [468]:
# all_terms = []
# group_label = 'O'
# group = CPGs.get_group_by_label(group_label)
# for irrep in group.irrep_labels:
#     for S in [0,1]:
#         for mS in range(-S, S+1):
#             states = [{'qnums':{sp.Symbol(r'\alpha'):sp.Symbol(r'\alpha'), # qnum that distinguishes between states
#                                 sp.Symbol(r'\gamma'): component, # symbol for a component of a basis for the irrep
#                                 sp.Symbol(r'M_s'):mS # the Ms equal to m_1 + m_2
#                                 }} for component in group.component_labels[irrep]]
#             aterm = Term({'states':states,
#                   'group_label': group_label,
#                   'S': S,
#                   'irrep': irrep
#                  })
#             all_terms.append(aterm)

In [469]:
# for term in all_terms:
#     display(term.term_symbol())
#     for state_symb in term.make_state_symbols():
#         display(state_symb)
#     break

In [7]:
# s1, s2 = sp.S(1)/2, sp.S(1)/2
# Ss = [0,1]
# m1s = [-sp.S(1)/2,sp.S(1)/2]
# m2s = [-sp.S(1)/2,sp.S(1)/2]
# group_CGs = group.CG_coefficients
# Γ = sp.Symbol('T_1')
# summands = {}
# for Γ1, Γ2, Γ3, m1, m2, S in product(group.irrep_labels,
#                                  group.irrep_labels,
#                                  group.irrep_labels,
#                                  m1s, m2s, Ss):
#     for γ1, γ2, γ3 in product(group.component_labels[Γ1],
#                               group.component_labels[Γ2],
#                               group.component_labels[Γ3]):
#         for mSz in range(-S, S+1):
#             sCG = ClebschG(s1, s2, S, m1, m2, mSz)
#             if (γ1, γ2, γ3) not in group_CGs.keys():
#                 gCG = 0
#                 continue
#             else:
#                 gCG = group_CGs[(γ1, γ2, γ3)]
#             coeff = sCG*gCG
#             key = (Γ1, Γ2, Γ3, γ3, mSz)
#             if (Γ2,m2,γ2) == (Γ1,m1,γ1):
#                 coeff = 0
#             if coeff!=0:
#                 if key not in summands.keys():
#                     summands[key] = {}
#                 qkey = (Γ1,m1,γ1,Γ2,m2,γ2)
#                 ikey = (Γ2,m2,γ2Γ1,m1,γ1)
#                 if ikey in summands.keys():
#                     summands[ikey].append(Qet({:-coeff}))
#                 else:
#                     summands[key].append(Qet({(Γ1,m1,γ1,Γ2,m2,γ2):coeff}))

In [603]:
# this just does the work for a single group
# print("computing partial sums...")
# s1, s2 = sp.S(1)/2, sp.S(1)/2
# Ss = [0,1]
# m1s = [-sp.S(1)/2,sp.S(1)/2]
# m2s = [-sp.S(1)/2,sp.S(1)/2]
# group_CGs = group.CG_coefficients
# Γ = sp.Symbol('T_1')
# summands = {}
# for Γ1, Γ2, Γ3, m1, m2, S in product(group.irrep_labels,
#                                  group.irrep_labels,
#                                  group.irrep_labels,
#                                  m1s, m2s, Ss):
#     for γ1, γ2, γ3 in product(group.component_labels[Γ1],
#                               group.component_labels[Γ2],
#                               group.component_labels[Γ3]):
#         for mSz in range(-S, S+1):
#             sCG = ClebschG(s1, s2, S, m1, m2, mSz)
#             if (γ1, γ2, γ3) not in group_CGs.keys():
#                 gCG = 0
#                 continue
#             else:
#                 gCG = group_CGs[(γ1, γ2, γ3)]
#             coeff = sCG*gCG
#             key = (Γ1, Γ2, Γ3, γ3, S, mSz)
#             if (Γ2,m2,γ2) == (Γ1,m1,γ1):
#                 coeff = 0
#             if coeff!=0:
#                 if key not in summands.keys():
#                     summands[key] = []
#                 summands[key].append(Qet({(Γ1,γ1,m1,Γ2,γ2,m2):coeff}))
# print("summing out inner parts ...")
# total_qets = {k:sum(v,Qet({})) for k,v in summands.items()}
# best_qets = {k:det_simplify(v) for k,v in total_qets.items()}
# best_qets = {k:v for k,v in best_qets.items() if len(v.dict)!=0}
# print("term collection truck ...")
# terms = {}
# for k, v in best_qets.items():
#     (Γ1, Γ2, Γ3, γ3, S, mSz) = k
#     term_pair = (Γ3, S)
#     if term_pair not in terms.keys():
#         terms[term_pair] = {}
#     terms[term_pair][k] = v
# final_terms = {}
# for term in terms:
#     ir, S = term
#     states = terms[term]
#     one_term = Term({'irrep': ir, 'S': S, 'states': states})
#     final_terms[term] = one_term
# print("Manufacturing latex output ....")
# max_counter = 3
# print_outs = []
# for one_term_k, one_term in final_terms.items():
#     term_symb = r'{{}}^{{{M}}}\!{ir}'.format(M=(2*one_term_k[1]+1), ir = sp.latex(one_term_k[0]))
#     term_symb = sp.Symbol(term_symb)
#     print_outs.append('\n\\hrulefill\n \\begin{center} {\\Large $%s$} \n\\vspace{0.5cm}\n' % sp.latex(term_symb))
#     counter = 0
#     for state_key, state in one_term.states.items():
#         (Γ1, Γ2, Γ3, γ3, S, mSz) = state_key
#         α = Γ1 * Γ2
#         term_symb = r'{{}}^{{{M}}}\!{ir}'.format(M=(2*S+1), ir = sp.latex(Γ3))
#         state_symbol = '\\Psi(%s,%s,M\!=\!%d,%s)' % (sp.latex(α).lower(), term_symb, mSz, sp.latex(γ3))
#         state_symbol = sp.Symbol(state_symbol)
#         v_simple = Qet({(k[1],k[2],k[4],k[5]):v for k,v in state.dict.items()})
#         v_det = as_determinantal_ket(v_simple)
#         pout = "$%s = %s$\n" % (sp.latex(state_symbol), sp.latex(v_det))
#         pout = sp.Symbol(pout)
#         print_outs.append(sp.latex(pout))
#         if counter % 5 == 4:
#             print_outs.append('\n\\vspace{0.5cm}\n')
#         counter += 1
#     print_outs.append('\n\\end{center}')
# all_wavefunctions = '\n'.join(print_outs)
# print("saving to file ...")
# open('/Volumes/GoogleDrive/My Drive/Zia Lab/Theoretical Division/termwaves.tex','w').write(all_wavefunctions)

In [445]:
# # with new labels
# def num_waves(ir0,ir1):
#     if ir0 == ir1:
#         return sp.binomial(group.irrep_dims[ir0]+group.irrep_dims[ir1], 2)
#     else:
#         return group.irrep_dims[ir0]*group.irrep_dims[ir1]*4
# def full_waves():
#     done = []
#     size = 0
#     for k,v in group.product_table.odict.items():
#         if (k[1],k[0]) in done:
#             continue
#         size += sum(list(map(lambda x: group.irrep_dims[x], v)))
#         done.append(k)
#     return 2*size
# all_final_outputs = []
# tally_waves = {}
# for group_counter, group_label in enumerate(CPGs.all_group_labels):
#     if group_label != 'T_{h}':
#         continue
#     print("Working on group %s ..." % group_label)
#     group = CPGs.get_group_by_label(group_label)
#     print("computing partial sums...")
#     s1, s2 = sp.S(1)/2, sp.S(1)/2
#     Ss  = [0,1]
#     m1s = [-sp.S(1)/2, sp.S(1)/2]
#     m2s = [-sp.S(1)/2, sp.S(1)/2]
#     group_CGs = group.CG_coefficients
#     flat_labels = dict(sum([list(l.items()) for l in list(new_labels[group_label].values())],[]))
#     group_CGs = {(flat_labels[k[0]], flat_labels[k[1]], flat_labels[k[2]]):v for k,v in group_CGs.items()}
#     summands = OrderedDict()
#     group.new_component_labels = OrderedDict([(ir, list(new_labels[group_label][ir].values())) for ir in group.irrep_labels])
#     for Γ1, Γ2, Γ3, m1, m2, S in product(group.irrep_labels,
#                                      group.irrep_labels,
#                                      group.irrep_labels,
#                                      m1s,
#                                      m2s,
#                                      Ss):
#         for γ1, γ2, γ3 in product(group.new_component_labels[Γ1],
#                                   group.new_component_labels[Γ2],
#                                   group.new_component_labels[Γ3]):
#             for mSz in range(-S, S+1):
#                 sCG = ClebschG(s1, s2, S, m1, m2, mSz)
#                 if (γ1, γ2, γ3) not in group_CGs.keys():
#                     continue
#                 else:
#                     gCG = group_CGs[(γ1, γ2, γ3)]
#                 coeff = sCG*gCG
#                 key = (Γ1, Γ2, Γ3, γ3, S, mSz)
#                 if (Γ2,m2,γ2) == (Γ1,m1,γ1):
#                     continue
#                 if coeff!=0:
#                     if key not in summands.keys():
#                         summands[key] = []
#                     summands[key].append(Qet({(Γ1,γ1,m1,Γ2,γ2,m2):coeff}))
#     print("summing out inner parts ...")
#     total_qets = OrderedDict([(k, sum(v,Qet({}))) for k,v in summands.items()])
#     best_qets = OrderedDict([(k, det_simplify(v)) for k,v in total_qets.items()])
#     best_qets = OrderedDict([(k, v) for k,v in best_qets.items() if len(v.dict)!=0])
#     print("term collection truck ...")
#     terms = OrderedDict()
#     for k, v in best_qets.items():
#         (Γ1, Γ2, Γ3, γ3, S, mSz) = k
#         term_pair = (Γ3, S)
#         if term_pair not in terms.keys():
#             terms[term_pair] = OrderedDict()
#         terms[term_pair][k] = v
#     final_terms = OrderedDict()
#     for term in terms:
#         ir, S = term
#         states = terms[term]
#         one_term = Term({'irrep': ir, 'S': S, 'states': states})
#         final_terms[term] = one_term
#     print("Manufacturing latex output ....")
#     max_counter = 3
#     print_outs = ['\\subsection{Group $%s$} \n \\begin{center} \\underline{Component labels} \n\\vspace{0.2cm}\n' % group_label]
#     for irrep_symbol in group.irrep_labels:
#         components_for_printing = ','.join(list(map(sp.latex, group.new_component_labels[irrep_symbol])))
#         print_outs.append('\\noindent$%s:\\{%s\\}$\n\n' % (sp.latex(irrep_symbol), components_for_printing))
#     print_outs.append('\\end{center}')
#     supreme_states = {}
#     total_waves = 0
#     for one_term_k, one_term in final_terms.items():
#         term_symb = r'{{}}^{{{M}}}\!{ir}'.format(M=(2*one_term_k[1]+1), ir = sp.latex(one_term_k[0]))
#         term_symb = sp.Symbol(term_symb)
#         print_outs.append('\n\\hrulefill\n  \\subsubsection{$%s$} \n\\vspace{0.25cm}\n \\begin{center} \n' % sp.latex(term_symb))
#         counter = 0
#         done_keys = []
#         prev_α = ''
#         for state_key, state in one_term.states.items():
#             (Γ1, Γ2, Γ3, γ3, S, mSz) = state_key
#             equiv_key = (Γ2, Γ1, Γ3, γ3, S, mSz)
#             if equiv_key in done_keys:
#                 continue
#                 warn_mark = '**'
#             else:
#                 warn_mark = ''
#             α = Γ1 * Γ2
#             multiplicity = 2*S+1
#             term_symb = r'{{}}^{{{M}}}\!{ir}'.format(M=(2*S+1), ir = sp.latex(Γ3))
#             state_symbol = '%s \\Psi_{%d}(%s,%s,M\!=\!%d,%s)' % (warn_mark, counter+1, sp.latex(α).lower(), term_symb, mSz, sp.latex(γ3))
#             state_symbol = sp.Symbol(state_symbol)
#             v_simple = Qet({(k[1],k[2],k[4],k[5]):v for k,v in state.dict.items()})
#             v_det = as_determinantal_ket(v_simple)
#             sup_key = (α)
#             if sup_key not in supreme_states.keys():
#                 supreme_states[sup_key] = []
#             supreme_states[sup_key].append(v_det)
#             pout = "$\\textcolor{blue}{%s} = %s$\\vspace{0.1cm}\n" % (sp.latex(state_symbol), sp.latex(v_det))
#             pout = sp.Symbol(pout)
#             if prev_α != α:
#                 if prev_α == '':
#                     print_outs.append('\n\n $\\textcolor{red}{%s}$ \n\\vspace{0.25cm}\n' % (sp.latex(α).lower()))
#                 else:
#                     print_outs.append('\n\\vspace{0.25cm}\n\n $\\textcolor{red}{%s}$ \n\\vspace{0.25cm}\n' % (sp.latex(α).lower()))
#             prev_α = α
#             print_outs.append(sp.latex(pout))
#             counter += 1
#             total_waves += 1
#             done_keys.append(state_key)
#         print_outs.append('\n\\end{center}\n\n')
#     irrep_combos = list(combinations_with_replacement(group.irrep_labels,2))
#     total_waves_groundtruth = sum([num_waves(ir0,ir1) for ir0, ir1 in irrep_combos])
#     tally_waves[group_label] = (total_waves,total_waves_groundtruth,full_waves())
# #     print_outs.append("Total--- waves = %d\n\n" % total_waves)
# #     print_outs.append("Expected waves = %d\n\n" % total_waves_groundtruth)
#     all_wavefunctions = '\n'.join(print_outs)
#     all_wavefunctions = all_wavefunctions.replace("^'","^{'}").replace("^''","^{''}")
#     all_final_outputs.append(all_wavefunctions)
# print("saving to file ...")
# super_final_output = '\n'.join(all_final_outputs)
# open('/Users/juan/Library/Mobile Documents/com~apple~CloudDocs/iCloudFiles/Theoretical Division/termwaves.tex','w').write(super_final_output)
# !beep

Working on group T_{h} ...
computing partial sums...
summing out inner parts ...
term collection truck ...
Manufacturing latex output ....
saving to file ...


In [ ]:
# def full_waves():
#     done = []
#     size = 0
#     for k,v in group.product_table.odict.items():
#         if (k[1],k[0]) in done:
#             continue
#         size += sum(list(map(lambda x: group.irrep_dims[x]*4, v)))
#         done.append(k)
#     return size

In [608]:
# irrep_combos = list(combinations_with_replacement(irreps,2))
# counter = 0
# for k,v in supreme_states.items():
#     α = k
#     combo = [(ir0,ir1) for ir0, ir1 in irrep_combos if ir0*ir1 == α][0]
#     display(α)
#     print("irrep dims",group.irrep_dims[combo[0]],group.irrep_dims[combo[1]])
#     if combo[0] == combo[1]:
#         print("nchoosek",sp.binomial(group.irrep_dims[combo[0]]*2,2))
#     else:
#         print("(d1*d2)*4",group.irrep_dims[combo[0]]*group.irrep_dims[combo[1]]*4)
#     print("waves---",len(v))
#     print("altsum--",sum(list(map(lambda x: group.irrep_dims[x],group.product_table.odict[combo])))*4)
#     counter += 1

In [609]:
# all_final_outputs = []
# for group_counter, group_label in enumerate(CPGs.all_group_labels):
#     if group_label != 'T_{h}':
#         continue
#     print("Working on group %s ..." % group_label)
#     group = CPGs.get_group_by_label(group_label)
#     print("computing partial sums...")
#     s1, s2 = sp.S(1)/2, sp.S(1)/2
#     Ss = [0,1]
#     m1s = [-sp.S(1)/2,sp.S(1)/2]
#     m2s = [-sp.S(1)/2,sp.S(1)/2]
#     group_CGs = group.CG_coefficients
#     summands = OrderedDict()
#     for Γ1, Γ2, Γ3, m1, m2, S in product(group.irrep_labels,
#                                      group.irrep_labels,
#                                      group.irrep_labels,
#                                      m1s, m2s, Ss):
#         for γ1, γ2, γ3 in product(group.component_labels[Γ1],
#                                   group.component_labels[Γ2],
#                                   group.component_labels[Γ3]):
#             for mSz in range(-S, S+1):
#                 sCG = ClebschG(s1, s2, S, m1, m2, mSz)
#                 if (γ1, γ2, γ3) not in group_CGs.keys():
#                     gCG = 0
#                     continue
#                 else:
#                     gCG = group_CGs[(γ1, γ2, γ3)]
#                 coeff = sCG*gCG
#                 key = (Γ1, Γ2, Γ3, γ3, S, mSz)
#                 if (Γ2,m2,γ2) == (Γ1,m1,γ1):
#                     coeff = 0
#                 if coeff!=0:
#                     if key not in summands.keys():
#                         summands[key] = []
#                     summands[key].append(Qet({(Γ1,γ1,m1,Γ2,γ2,m2):coeff}))
#     print("summing out inner parts ...")
#     total_qets = OrderedDict([(k,sum(v,Qet({}))) for k,v in summands.items()])
#     best_qets = OrderedDict([(k,det_simplify(v)) for k,v in total_qets.items()])
#     best_qets = OrderedDict([(k,v) for k,v in best_qets.items() if len(v.dict)!=0])
#     print("term collection truck ...")
#     terms = OrderedDict()
#     for k, v in best_qets.items():
#         (Γ1, Γ2, Γ3, γ3, S, mSz) = k
#         term_pair = (Γ3, S)
#         if term_pair not in terms.keys():
#             terms[term_pair] = OrderedDict()
#         terms[term_pair][k] = v
#     final_terms = OrderedDict()
#     for term in terms:
#         ir, S = term
#         states = terms[term]
#         one_term = Term({'irrep': ir, 'S': S, 'states': states})
#         final_terms[term] = one_term
#     print("Manufacturing latex output ....")
#     max_counter = 3
# #     print_outs = ['\\begin{center} Group $%s$ \\end{center} \n' % group_label]
#     print_outs = ['\\subsection{Group $%s$} \n' % group_label]
#     for one_term_k, one_term in final_terms.items():
#         term_symb = r'{{}}^{{{M}}}\!{ir}'.format(M=(2*one_term_k[1]+1), ir = sp.latex(one_term_k[0]))
#         term_symb = sp.Symbol(term_symb)
#         print_outs.append('\n\\hrulefill\n  \\subsubsection{$%s$} \n\\vspace{0.25cm}\n \\begin{center} \n' % sp.latex(term_symb))
#         counter = 0
#         done_keys = []
#         prev_α = ''
#         for state_key, state in one_term.states.items():
#             (Γ1, Γ2, Γ3, γ3, S, mSz) = state_key
#             equiv_key = (Γ2, Γ1, Γ3, γ3, S, mSz)
#             if equiv_key in done_keys:
#                 continue
#                 warn_mark = '**'
#             else:
#                 warn_mark = ''
#             α = Γ1 * Γ2
#             multiplicity = 2*S+1
#             term_symb = r'{{}}^{{{M}}}\!{ir}'.format(M=(2*S+1), ir = sp.latex(Γ3))
#             state_symbol = '%s \\Psi_{%d}(%s,%s,M\!=\!%d,%s)' % (warn_mark, counter+1, sp.latex(α).lower(), term_symb, mSz, sp.latex(γ3))
#             state_symbol = sp.Symbol(state_symbol)
#             v_simple = Qet({(k[1],k[2],k[4],k[5]):v for k,v in state.dict.items()})
#             v_det = as_determinantal_ket(v_simple)
#             pout = "$\\textcolor{blue}{%s} = %s$\\vspace{0.1cm}\n" % (sp.latex(state_symbol), sp.latex(v_det))
#             pout = sp.Symbol(pout)
#             if prev_α != α:
#                 if prev_α == '':
#                     print_outs.append('\n\n $\\textcolor{red}{%s}$ \n\\vspace{0.25cm}\n' % (sp.latex(α).lower()))
#                 else:
#                     print_outs.append('\n\\vspace{0.25cm}\n\n $\\textcolor{red}{%s}$ \n\\vspace{0.25cm}\n' % (sp.latex(α).lower()))
#             prev_α = α
#             print_outs.append(sp.latex(pout))
#             counter += 1
#             done_keys.append(state_key)
#         print_outs.append('\n\\end{center}')
#     all_wavefunctions = '\n'.join(print_outs)
#     all_wavefunctions = all_wavefunctions.replace("^'","^{'}").replace("^''","^{''}")
#     all_final_outputs.append(all_wavefunctions)
# #     if group_counter == 10:
# #         break
# print("saving to file ...")
# super_final_output = '\n'.join(all_final_outputs)
# open('/Users/juan/Library/Mobile Documents/com~apple~CloudDocs/iCloudFiles/Theoretical Division/termwaves.tex','w').write(super_final_output)
# !beep

## $d^2$ (Oct 28)

In [32]:
class CrystalDefect():
    '''
    A CrystalDefect has the following attributes:
    
    - site          (str):
    - host          (str):
    - atom_symb     (str):
    - charge_state  (int):
    - ion           (Ion):
    - Bnm           (Bnm):
    - freeIparams  (dict):
    '''
    def __init__(self, defectDict):
        if 'Bnm' in defectDict.keys():
            self.Bnm = defectDict['Bnm']
            Bnmattrs = [a for a in dir(self.Bnm) if not a.startswith('__')]
            for Bnmattr in Bnmattrs:
                setattr(self, Bnmattr, getattr(self.Bnm, Bnmattr))
        for k, v in defectDict.items():
            if k == 'Bnm':
                continue
            setattr(self, k, v)
        self.GroupSymb = self.point_sym_group
        self.GroupNum = CPGs.all_group_labels.index(self.GroupSymb)
        self.Cfield = crystal_fields[self.GroupNum]
        self.splitter = crystal_splits[self.GroupNum]
    def __repr__(self):
        return self.Bnm.__repr__()

In [35]:
d2_df = CFparams[CFparams['nd^N'].apply(lambda x: 'd2' in x)]
Bnms = []
Cdefects = []
for row in d2_df.iterrows():
    row_data = row[1]
    Bparams = {k:v for k,v in row_data.items() if 'B_' in k}
    symmetry = row_data['Symmetry']
    if symmetry == 'C_{3i}':
        symmetry = 'S_{6}'
    Bnm_dict = {
        'host': row_data['Host'],
        'site': row_data['Site'],
        'point_sym_group': symmetry,
        'ion': row_data['Ion'],
        'params': Bparams,
        'unit' : 'cm^{-1}',
        'sources': row_data['Source'],
        'comments': '',
        'is_experimental': (row_data['Type'] == 'Experimental')
        }
    Bnms.append(Bnm(Bnm_dict))

### Term Factory

- Goal given an electron configuration, and a group, determine all of the terms that there could possibly exist.
- `term` contains
- 

In [115]:
class Term():
    '''
    Represents a term, holds the states that correspond to it,
    and offers a nice display of the term symbol.
    '''
    def __init__(self, init_dict):
        self.group_label = 'O_{h}'
        for k,v in init_dict.items():
            setattr(self, k, v)
        self.states = [] # a list of Qets
        self.term_prototype = sp.Symbol(r'\Psi({α},{S},{Γ},{M_s},{γ})')
    def term_symbol(self):
        ndict = {'α' : self.qnums[sp.Symbol(r'\alpha')],
        'S' : self.qnums[sp.Symbol('S')],
        'Γ' : self.qnums[sp.Symbol(r'\Gamma')],
        'M_s' : self.qnums[sp.Symbol(r'Ms')],
        'γ' : self.qnums[sp.Symbol(r'\gamma')]}
        return sp.Symbol(str(self.term_prototype).format(**ndict))
    def __str__(self):
        return str(self.qnums)
    def __repr__(self):
        return str(self.qnums)

In [116]:
aterm = Term({'qnums':{sp.Symbol(r'\alpha'):sp.Symbol(r'\alpha'), # qnum that distinguishes between states
                      sp.Symbol(r'S'):2, # The S in S(S+1), from eigenvalues of S^2
                      sp.Symbol(r'\Gamma'):sp.Symbol(r'T_{1}'), # symbol for irrep in group
                      sp.Symbol(r'\gamma'):sp.Symbol(r'\gamma'), # symbol for a component of a basis for the irrep
                      sp.Symbol(r'Ms'):1 # the Ms equal to m_1 + m_2
                      }
             })

In [117]:
group = CPGs.get_group_by_label('O')

In [126]:
group.product_table.odict[(sp.Symbol('T_1'),sp.Symbol('T_1'))]

[A_1, E, T_1, T_2]

In [118]:
group.component_labels

{A_1: [a_{A_1}],
 A_2: [a_{A_2}],
 E: [u_{E}, v_{E}],
 T_1: [x_{T_1}, y_{T_1}, z_{T_1}],
 T_2: [x_{T_2}, y_{T_2}, z_{T_2}]}

In [119]:
aterm.term_symbol()

\Psi(\alpha2T_{1}1\gamma)

In [120]:
aterm.term_prototype

\Psi({α},{S},{Γ},{M_s},{γ})

## $d^1$

In [3]:
crystal_splits[32]

{'matrices': [Matrix([
  [  B_{4,0}/21,             0,           0,             0, 5*B_{4,0}/21],
  [           0, -4*B_{4,0}/21,           0,             0,            0],
  [           0,             0, 2*B_{4,0}/7,             0,            0],
  [           0,             0,           0, -4*B_{4,0}/21,            0],
  [5*B_{4,0}/21,             0,           0,             0,   B_{4,0}/21]]),
  Matrix([
  [   B_{4,0}/21,             0,           0,             0, -5*B_{4,0}/21],
  [            0, -4*B_{4,0}/21,           0,             0,             0],
  [            0,             0, 2*B_{4,0}/7,             0,             0],
  [            0,             0,           0, -4*B_{4,0}/21,             0],
  [-5*B_{4,0}/21,             0,           0,             0,    B_{4,0}/21]])],
 'simple_free_parameters': [{B_{4,0}}, {B_{4,0}}],
 'free_parameters': [{B_{4,0}^r}, {B_{4,0}^r}],
 'eigen_system': [[(-4*B_{4,0}^r/21,
    3,
    [Matrix([
     [0],
     [1],
     [0],
     [0],
    

In [347]:
grup_indices = range(3,33)
matrices = {i: crystal_splits[i]['matrices'][0] for i in grup_indices}

In [349]:
grup_symbols = list(map(sp.Symbol,[CPGs.all_group_labels[i-1] for i in grup_indices]))

In [357]:
mahtrix = [grup_symbols]
umatrices = {}
for grup_index_0 in grup_indices:
    row = []
    m0 = matrices[grup_index_0]
    m0l = sp.latex(m0)
    if m0l not in umatrices.keys():
        umatrices[m0l] = []
    umatrices[m0l].append(CPGs.all_group_labels[grup_index_0-1])
    for grup_index_1 in grup_indices:
        m1 = matrices[grup_index_1]
        row.append(1 if m0==m1 else 0)
    mahtrix.append(row)
sp.Matrix(mahtrix)

Matrix([
[C_{2}, C_{s}, C_{2h}, D_{2}, C_{2v}, D_{2h}, C_{4}, S_{4}, C_{4h}, D_{4}, C_{4v}, D_{2d}, D_{4h}, C_{3}, S_{6}, D_{3}, C_{3v}, D_{3d}, C_{6}, C_{3h}, C_{6h}, D_{6}, C_{6v}, D_{3h}, D_{6h}, T, T_{h}, O, T_{d}, O_{h}],
[    1,     1,      1,     0,      0,      0,     0,     0,      0,     0,      0,      0,      0,     0,     0,     0,      0,      0,     0,      0,      0,     0,      0,      0,      0, 0,     0, 0,     0,     0],
[    1,     1,      1,     0,      0,      0,     0,     0,      0,     0,      0,      0,      0,     0,     0,     0,      0,      0,     0,      0,      0,     0,      0,      0,      0, 0,     0, 0,     0,     0],
[    1,     1,      1,     0,      0,      0,     0,     0,      0,     0,      0,      0,      0,     0,     0,     0,      0,      0,     0,      0,      0,     0,      0,      0,      0, 0,     0, 0,     0,     0],
[    0,     0,      0,     1,      1,      1,     0,     0,      0,     0,      0,      0,      0,     0,     0,     0,

In [12]:
class CrystalDefect():
    '''
    A CrystalDefect has the following attributes:
    
    - site          (str):
    - host          (str):
    - atom_symb     (str):
    - charge_state  (int):
    - ion           (Ion):
    - Bnm           (Bnm):
    - freeIparams  (dict):
    '''
    def __init__(self, defectDict):
        if 'Bnm' in defectDict.keys():
            self.Bnm = defectDict['Bnm']
            Bnmattrs = [a for a in dir(self.Bnm) if not a.startswith('__')]
            for Bnmattr in Bnmattrs:
                setattr(self, Bnmattr, getattr(self.Bnm, Bnmattr))
        for k, v in defectDict.items():
            if k == 'Bnm':
                continue
            setattr(self, k, v)
        self.GroupSymb = self.point_sym_group
        self.GroupNum = CPGs.all_group_labels.index(self.GroupSymb)
        self.Cfield = crystal_fields[self.GroupNum]
        self.splitter = crystal_splits[self.GroupNum]
    def __repr__(self):
        return self.Bnm.__repr__()

In [5]:
d1_df = CFparams[CFparams['nd^N'].apply(lambda x: 'd1' in x)]
Bnms = []
Cdefects = []
for row in d1_df.iterrows():
    row_data = row[1]
    Bparams = {k:v for k,v in row_data.items() if 'B_' in k}
    symmetry = row_data['Symmetry']
    if symmetry == 'C_{3i}':
        symmetry = 'S_{6}'
    Bnm_dict = {
        'host': row_data['Host'],
        'site': row_data['Site'],
        'point_sym_group': symmetry,
        'ion': row_data['Ion'],
        'params': Bparams,
        'unit' : 'cm^{-1}',
        'sources': row_data['Source'],
        'comments': '',
        'is_experimental': (row_data['Type'] == 'Experimental')
        }
    Bnms.append(Bnm(Bnm_dict))

In [9]:
CPGs.all_group_labels.index('S_{6}')

16

In [10]:
crystal_splits[16]['eigen_system'][0][1][0]

-B_{2,0}^r/14 - B_{4,0}^r/14 - sqrt(81*B_{2,0}^r**2 - 90*B_{2,0}^r*B_{4,0}^r + 25*B_{4,0}^r**2 + 140*B_{4,3}^i**2 + 140*B_{4,3}^r**2)/42

In [13]:
cdef = CrystalDefect({'Bnm':Bnms[0],'charge_state':4})

In [14]:
def square_field_in_a_round_hole(matrix, Bnm_params):
    # assume all params are real
    # and that they aint currently typed as either
    # assume that all that's missing is zero
    available_params = {sp.Symbol(k):v for k,v in Bnm_params.items() if not math.isnan(v)}
    matrix = matrix.subs(available_params)
    available_params = {sp.Symbol(k+'^r'):v for k,v in Bnm_params.items() if not math.isnan(v)}
    matrix_params = set(matrix.free_symbols)
    have_params = matrix_params.intersection(available_params)
    dont_have = {k: 0 for k in matrix_params.difference(available_params)}
    fudged_params = available_params
    fudged_params.update(dont_have)
    return matrix.subs(fudged_params)

In [15]:
cdef.splitter['matrices'][0].subs({sp.Symbol('B_{2,0}'):0,sp.Symbol('B_{4,3}^i'):0,sp.Symbol('B_{4,3}^r'):sp.sqrt(sp.S(10)/7) * sp.Symbol('B_{4,0}')}).eigenvals()

{2*B_{4,0}/7: 3, -3*B_{4,0}/7: 2}

In [16]:
splits = cdef.splitter['matrices'][0].subs({sp.Symbol('B_{2,0}'):0,sp.Symbol('B_{4,3}^i'):0,sp.Symbol('B_{4,3}^r'):sp.sqrt(sp.S(10)/7) * sp.Symbol('B_{4,0}')}).eigenvals()

In [18]:
cdef.host

'Y3Al5O12'

In [19]:
dE = (13200+8800.0) * UnitCon.con_factor('cm^{-1}','eV')
1240/dE

454.60338568141566

In [20]:
# splits = square_field_in_a_round_hole(cdef.splitter['matrices'][0], cdef.params).eigenvals()
splits = {-8800.0:3, 13200.0:2}
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(4,6))
ax.plot([-1,0],[0,0])
for e, mult in splits.items():
    e = e*UnitCon.con_factor('cm^{-1}','eV') 
    for i in range(mult):
        ax.plot([0,1,2],[0,e+i/50,e+i/50],'k-')
ax.set_ylabel('$\Delta$E / $cm^{-1}$')
ax.set_title('$%s^{%d\!\!+}\!\!:\!%s$' % (cdef.ion, cdef.charge_state,''.join([r'{%s}_{%d}' % (k,v) for k,v in complex_formula_parser(cdef.host)[0].items()])))
ax.xaxis.set_visible(False)
plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
V4ion = Ion('V',4)
V4ion.nist_data.loc[:,'Level (cm^{-1})'] = V4ion.nist_data['Level (eV)'] * UnitCon.con_factor('eV','cm^{-1}')

In [29]:
V4ion.nist_data.loc[:,'Level (cm^{-1})']

0          0.000000
1        624.869951
2     148143.354049
3     206393.720573
4     207660.002906
          ...      
66    496296.308675
67    497556.146639
68    500116.956840
69    500501.683285
70    526532.016519
Name: Level (cm^{-1}), Length: 71, dtype: float64

In [30]:
cdef.splitter['free_parameters']

[{B_{2,0}^r, B_{4,0}^r, B_{4,3}^i, B_{4,3}^r}]

In [31]:
cdef.splitter['matrices'][0].free_symbols

{B_{2,0}, B_{4,0}, B_{4,3}^i, B_{4,3}^r}